# Sea ice analyses (maps, timeseries, etc) for ACCESS-OM2 at multiple resolutions

To get access to the rechunked ERA5 files in `uc0`, this notebook needs to be run with `gadi_jupyter -P v45 -t 15:00:00 -n 27 -q 'normalbw' -s 'gdata/uc0'` 
using this modified `gadi_jupyter` https://github.com/aekiss/nci_scripts/blob/append-storage/gadi_jupyter

### TODO: 
- plot bias maps for runs forced by ERA5 `1deg_era5_iaf` and `025deg_era5_iaf` - see https://forum.access-hive.org.au/t/era-5-forced-access-om2-simulations/1103 (will I also need to use the JRA55 comparison runs?)
- compare thickness with cryosat or ICESat data?
- compare with SOSE ice data: http://sose.ucsd.edu/BSOSE6_iter133_solution.html
- compare with MOM6 panantarctic `/scratch/x77/amh157/mom6/archive/mom6-panan`
- compare with Pauthenet et al. SO climatology data: `/g/data/ik11/observations/Southern_Ocean_Climatology_PauthenetETAL2021/TS_Climato_Antarctic60S.nc`

In [ ]:
%matplotlib inline
import cosima_cookbook as cc
from dask.distributed import Client
from glob import glob
import os,sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import dask.array as da
import numpy as np
import xarray as xr
import netCDF4 as nc
from tqdm import tqdm_notebook

import cartopy.crs as ccrs
import cartopy.feature as cft
#from mpl_toolkits.basemap import Basemap
from calendar import month_abbr
import cmocean as cm
import pandas as pd
import calendar
import cftime
from datetime import timedelta
from datetime import datetime
import copy
import xesmf
import gzip
import yaml
from IPython.display import display, Markdown

#import sys, os
#sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
os.chdir('/g/data/v45/aek156/notebooks/github/aekiss/ice_analysis') # needed for ARE
import exptdata
#print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

# set up time units and offset to suit CICE 
# see https://github.com/OceansAus/ACCESS-OM2-1-025-010deg-report/commit/ce3b6331bc4f304d2c5f957ccd2a0ae9ca5d6970#commitcomment-31646163
# for e in exptdata.exptdict.keys():
#     exptdata.exptdict[e]['time_units'] = None

# Set up time_units to suit CICE 
# Only retain netcdf time_units for 0.1 deg.
# We need to change time_units of 1deg and 0.25 deg 
# to shift the time so the last cycle starts in 1958.
# But we don't need offset because CICE time starts at beginning of run, 
# unlike MOM which starts in year 0001.
#exptdata.exptdict['01deg']['time_units'] = "days since 1985-01-01"  # don't use None - it leaves the time as cftime

In [ ]:
exptdata.exptdict.keys()

In [ ]:
import climtas.nci
climtas.nci.GadiClient(malloc_trim_threshold='64kib')

In [ ]:
figdir = 'figs'
dpi=200
if not os.path.exists(figdir):
    os.makedirs(figdir)

def figname(fname, figdir=figdir):
    return os.path.join(figdir, fname+'_'+str(dpi)+'dpi.png')

def savefigure(fname, skip=False):
    if not(os.path.exists(fname) and skip):
        plt.savefig(fname, dpi=dpi, bbox_inches='tight')  # comment out to disable saving
        pass
    return

## Specify date range

In [ ]:
# use common start and end dates for all runs
tstart = exptdata.clim_tstart
tend = exptdata.clim_tend

In [ ]:
timerange=slice(tstart, tend)
firstyear = pd.to_datetime(tstart).year  # assumes tstart is 1 January!
lastyear = pd.to_datetime(tend).year-1  # assumes tend is 1 January!
yearrange = str(firstyear)+'-'+str(lastyear)
print('yearrange =', yearrange, 'complete years')
print('tstart =', tstart)
print('tend =', tend)

## Define data loaders

In [ ]:
def fixcicetime(da):
    '''
    Correct the time coordinate in DataArray from CICE netcdf file.
    
    CICE netcdf files unhelpfully have a time coordinate which is just after the end of the averaging period, 
    e.g. the time stamp for a January average is 1 February, which messes up groupby month etc.
    
    This function just subtracts 12 hours to put it in the correct month (and day, for daily means).
    
    PR 109 gives an option to fix this:
    https://github.com/OceansAus/cosima-cookbook/pull/109
    
    '''
    try:
        da['time'] = da.time - np.timedelta64(12, 'h')
    except:
        da['time'] = da.time - timedelta(hours=12)  # for 01deg which for some reason uses cftime
    return da

# use this for DataSet: replaces the bad time dimension with the average of time_bounds.
#     The time type is also changed to datetime64[ns]
#     ds['time'] = ds.time_bounds.astype('int64').mean(axis=1).astype('datetime64[ns]')
    

In [ ]:
def loaddata(ide, varnames=['aice_m'], timerange=timerange, ncfiles=None, model='cice', **kwargs):
# load model data
#   NB: varnames must be t-grid variables!!

    grids = [(p, xr.open_dataset(p)) for p in ide['gridpaths']]
    for k in ['xt_ocean', 'yt_ocean', 'geolon_t', 'geolat_t', 'area_t']:
        if k not in ide:
            for (p, g) in grids:
                try:
                    ide[k] = g[k]
                    print(k, 'loaded from', p)
                    break
                except:
                    continue
            try:
                ide[k] = ide[k].rename({'grid_x_T': 'xt_ocean', 'grid_y_T': 'yt_ocean'}) # fix for 01deg
            except:
                pass

    if not isinstance(ncfiles, list):
        ncfiles = [ncfiles]*len(varnames)  # use the same ncfile for all variables

    for varname, ncfile in zip(varnames, ncfiles):
        if varname not in ide.keys():
            if ncfile is None:
                print('loading', varname)
            else:
                print('loading', varname, 'from', ncfile)                
            var = cc.querying.getvar(ide['expt'], varname, ide['session'], ncfile=ncfile,
                                                 start_time=str(timerange.start),
                                                 end_time=str(timerange.stop+timedelta(hours=12)),  # tweak since fixcicetime hasn't been applied yet
                                                 decode_coords=False, **kwargs)
#             if ide['offset']:
#                 var['time'] = var['time'] + ide['offset']  # BUG: already trimmed by start_time, end_time
            if model.lower()=='cice':
                try:
                    var = fixcicetime(var).sel(time=timerange)
                except:
                    pass
                # use physical coords instead of indices - ASSUMES VARIABLES ARE ON T GRID!
#                 var.coords['ni'] = ide['xt_ocean'].data
#                 var.coords['nj'] = ide['yt_ocean'].data
                var = var.rename(({'ni': 'xt_ocean', 'nj': 'yt_ocean'}))
#                 var = var.assign_coords({'latitude': ide['geolat_t'], 'longitude': ide['geolon_t']})
            else:
                try:
                    var = var.sel(time=timerange)
                except:
                    pass
            var.coords['xt_ocean'] = ide['xt_ocean'].data
            var.coords['yt_ocean'] = ide['yt_ocean'].data
            var = var.assign_coords({'latitude': ide['geolat_t'], 'longitude': ide['geolon_t']})
            ide[varname] = var
            try:
                varmm = var.groupby('time.month').mean('time', skipna=True)
                ide[varname+'_mm'] = varmm
            except:
                pass

In [ ]:
def loadJRA55do(ide, expt='MRI-JRA55-do-1-4-0', varnames=[], timerange=timerange): #, session=JRAsession):
# load JRA55-do data
    for varname in varnames:
        if varname not in ide.keys():
            print('loading', varname)
            var = cc.querying.getvar(expt, varname, ide['session'],
                                                 start_time=str(timerange.start),
                                                 end_time=str(timerange.stop),
                                                 decode_coords=False)
            var = var.rename(({'lon': 'longitude', 'lat': 'latitude'}))
            var = var.sel(time=timerange)

            ide[varname] = var
            
            vard = var.resample(time='d').mean() # daily averages
            vard.attrs = var.attrs
            ide[varname+'_d'] = vard
            
            varm = var.resample(time='m').mean() # monthly averages
            varm.attrs = var.attrs
            ide[varname+'_m'] = varm

            varmm = var.groupby('time.month').mean('time', skipna=True)
            varmm.attrs = var.attrs
            ide[varname+'_mm'] = varmm

In [ ]:
def get_sic_obs(pattern, path='/g/data3/hh5/tmp/cosima/observations/NOAA/G02202_V3', # from http://nsidc.org/data/G02202
                variable='goddard_merged_seaice_conc_monthly', timerange=timerange): 
    '''
    Return a dataarray from the nc files in path/pattern.
    '''
    dataarrays = []
    files = glob(os.path.join(path, pattern))
    for f in tqdm_notebook(files, leave=False, desc='opening files'):
        dataarrays.append(xr.open_dataset(f, decode_times=False)[variable])
    dataarray = xr.concat(dataarrays, dim='time', coords='all')
    if 'time' in dataarray.coords:
        time_units = dataarray.time.units
        decoded_time = xr.conventions.times.decode_cf_datetime(dataarray.time, time_units)
        dataarray.coords['time'] = ('time', decoded_time,
                                    {'long_name': 'time', 'decoded_using': time_units })

    # replace values outside valid range with nan
    dataarray = dataarray.where(np.logical_and(dataarray>=0, dataarray<=1)) #, np.nan)
    
    # use the same coord names as access-om2 t grid
    dataarray = dataarray.rename(({'xgrid': 'xt_ocean', 'ygrid': 'yt_ocean'}))
    
    # sort by time and select timerange
    dataarray = dataarray.sortby('time').sel(time=timerange)

    return dataarray

In [ ]:
def get_giomas(path='/g/data/v45/aek156/giomas/Global_seaice', # from sftp anonymous@pscfiles.apl.uw.edu:/pscfiles/zhang/Global_seaice
                variable='heff', timerange=timerange, varrange=[None, None], scale=1.0): 
    '''
    Return a dataarray of GIOMAS data from the flat binary .gz files in path/variable.H????.gz
    variable defaults to 'heff' but can be any of 'heff', 'area', 'iceprod', 'snow' (and maybe others)
    GIOMAS info: http://psc.apl.washington.edu/zhang/Global_seaice/data.html
    '''
    # regridded giomas SIT, SIC is also here /g/data/v14/pas548/obs/seaice/giomas/ 
    grids = xr.open_dataset(os.path.join(path, 'heff.H1979.nc.gz'), decode_times=False)  # open this to get grid data
    shape = grids.heff.shape
    l = grids.lon_scaler
    longitude = xr.where(np.logical_and(l>l.isel(i=-1, j=0), l.i<shape[-1]/2), l-360, l) # move discontinuity to i=0
    xt_ocean = longitude.sel(j=0)  # nominal coordinate - incorrect in Arctic
    yt_ocean = xr.DataArray(np.linspace(grids.lat_scaler.min(), grids.lat_scaler.max(), shape[1]))  # nominal coordinate - incorrect in Arctic and probably elsewhere
    
    dataarrays = []
    for year in tqdm_notebook(range(timerange.start.year, timerange.stop.year+1), leave=False, desc='opening files'):
        file = os.path.join(path, variable+'.H'+str(year)+'.gz')
        with gzip.open(file, 'rb') as f:
            data=np.frombuffer(f.read(), dtype=np.single).reshape(shape)
            time = [ cftime.datetime(year, m, 15) for m in range(1,13) ]
#             time = [ datetime(year, m, 15) for m in range(1,13) ]  # workaround for type error: https://arccss.slack.com/archives/C08KM5KS6/p1625538785048700
            dataarrays.append(xr.DataArray(data, 
                                           dims=('time', 'yt_ocean', 'xt_ocean'),
                                           coords={'time': time, 'yt_ocean': yt_ocean.data, 'xt_ocean': xt_ocean.data}))
    dataarray = xr.concat(dataarrays, dim='time', coords='all')

    # sort by time and select timerange
    dataarray = dataarray.sortby('time').sel(time=timerange)
    
    dataarray = dataarray.assign_coords({ 'latitude': (['yt_ocean', 'xt_ocean'], grids.lat_scaler.data),
                                          'longitude': (['yt_ocean', 'xt_ocean'], longitude) }) #grids.lon_scaler.data) })
    
    dataarray = dataarray * scale
    
    # truncate data to varrange
    if varrange[0] != None:
        dataarray = dataarray.where(dataarray > varrange[0], varrange[0])
    if varrange[1] != None:
        dataarray = dataarray.where(dataarray < varrange[1], varrange[1])

    return dataarray

In [ ]:
def obs_mm(obs, groupby='time.month'):
    cobs = obs.groupby(groupby).mean('time', skipna=True)
    cobs = cobs.assign_coords({'latitude': obs.latitude.isel(time=0),
                              'longitude': obs.longitude.isel(time=0)})
    return cobs

In [ ]:
def obs_mstd(obs, groupby='time.month'):
    cobs = obs.groupby(groupby).std('time', skipna=True)
    cobs = cobs.assign_coords({'latitude': obs.latitude.isel(time=0),
                              'longitude': obs.longitude.isel(time=0)})
    return cobs

### JRA55 (not -do) data loader
JRA55 is available on NCI at `/g/data/ua8/synda/CREATE-IP/reanalysis/JMA/JRA-55/JRA-55/atmos/`: http://climate-cms.wikis.unsw.edu.au/JRA55

In [ ]:
JRA55path = '/g/data/ua8/synda/CREATE-IP/reanalysis/JMA/JRA-55/JRA-55/atmos/mon/v20200612/' # monthly means

In [ ]:
def get_JRA55(variable, path=JRA55path, timerange=timerange):
    '''
    Return a dataarray for an JRA55 variable.
    timerange is a slice object.
    '''
    files = glob(os.path.join(path, variable+'_*.nc'))
    files.sort()

    dataarray = xr.open_mfdataset(
                files,
                parallel=True,
                autoclose=True,
                decode_times=False)[variable]
    if 'time' in dataarray.coords:
        time_units = dataarray.time.units
        decoded_time = xr.conventions.times.decode_cf_datetime(dataarray.time, time_units)
        dataarray.coords['time'] = ('time', decoded_time,
                                    {'long_name': 'time', 'decoded_using': time_units })
    dataarray = dataarray.rename(({'lon': 'longitude', 'lat': 'latitude'}))
    
    # sort by time and select timerange
#     dataarray = dataarray.sortby('time').sel(time=timerange)
    dataarray = dataarray.sel(time=timerange)

    return dataarray

In [ ]:
def loadJRA55(ide, varnames=[], **kwargs):
    for varname in varnames:
        if varname not in ide.keys():
            print('loading', varname)
            var = get_JRA55(varname, **kwargs)

            ide[varname] = var
            
            vard = var.resample(time='d').mean() # daily averages
            vard.attrs = var.attrs
            ide[varname+'_d'] = vard
            
            varm = var.resample(time='m').mean() # monthly averages
            varm.attrs = var.attrs
            ide[varname+'_m'] = varm

            varmm = var.groupby('time.month').mean('time', skipna=True)
            varmm.attrs = var.attrs
            ide[varname+'_mm'] = varmm

### ERA5 data loader
ERA5 is available on NCI at `/g/data/rt52`: https://opus.nci.org.au/display/ERA5/ERA5+Community+Home
but some of that is badly chunked. Correctly chunked replacements are at `/g/data/uc0/era5_tmp/single-levels/reanalysis` (see https://github.com/COSIMA/access-om2/issues/242#issuecomment-939718446) and `get_ERA5` automatically uses these if present.

To get access to the rechunked ERA5 files in `uc0`, this notebook needs to be run with `gadi_jupyter -P v45 -t 15:00:00 -n 27 -q 'normalbw' -s 'gdata/uc0'` 
using this modified `gadi_jupyter` https://github.com/aekiss/nci_scripts/blob/append-storage/gadi_jupyter

Also see https://github.com/COSIMA/access-om2/issues/242

and https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form

The hourly single-level data is probably what we want. This appears to be from the "reanalysis" or "HRES" dataset (as opposed to the lower-resolution "ensemble" or "EDA" dataset).

It's 1-hourly 0.25° data (higher resolution than JRA55-do, which is 3-hourly 0.5625°), though this has been interpolated from the reanalysis grid, which is 0.28125° (31km) for HRES.

It's currently only from 1979 onwards but data from 1950 will be available soon (cf. 1958 for JRA55-do).

In [ ]:
ERA5path = '/g/data/rt52/era5/single-levels/reanalysis'
ERA5rechunkedpath = '/g/data/uc0/era5_tmp/single-levels/reanalysis' # see https://github.com/COSIMA/access-om2/issues/242#issuecomment-939718446
if len(glob(ERA5rechunkedpath))==0:
    print("ERROR: can't access", ERA5rechunkedpath, ' - try using -s option in gadi_jupyter')

In [ ]:
# ERA5vars is needed for get_ERA5 and loadERA5
ERA5vars = dict()  # keys are all ERA5 variables in ERA5path; values are dicts with keys 'fvar' and 'attrs'
ERA5varsfn = 'ERA5vars.yaml'
try:
    with open(ERA5varsfn, 'r') as f:
        ERA5vars = yaml.load(f)
except: # create file - takes about 20 min
    for p in tqdm_notebook(glob(os.path.join(ERA5path, '*')), leave=False, desc='scanning for ERA5 variable names'):
        files = glob(os.path.join(p, '2017', '*.nc'))
        files.sort()
        if len(files) > 0:
            file = files[0]
            da = xr.open_mfdataset(
                    file,
                    parallel=True,
                    autoclose=True,
                    decode_times=False)
            vname = [ v for v in da.variables if v not in list(da.coords) ][0]  # assumes only one data variable per file
            var = nc.Dataset(file).variables[vname]
            ERA5vars[vname] = {'fvar': os.path.basename(p),  # subdir of ERA5path containing this variable
                               'attrs': da[vname].attrs,
                               'dimensions': var.dimensions,
                               'chunking': var.chunking()} # could use this instead: ds.encoding['chunksizes']
    with open(ERA5varsfn, 'w') as f:
        yaml.dump(ERA5vars, f)

`ERA5vars` keys are all available variables. Potentially relevant ones include:
- `alnid`: Near IR albedo for diffuse radiation [(0 - 1)]
- `alnip`: Near IR albedo for direct radiation [(0 - 1)]
- `aluvd`: UV visible albedo for diffuse radiation [(0 - 1)]
- `aluvp`: UV visible albedo for direct radiation [(0 - 1)]
- `asn`: Snow albedo [(0 - 1)]
- `blh`: Boundary layer height [m]
- `cdww`: Coefficient of drag with waves [dimensionless]
- `chnk`: Charnock [~]
- `crr`: Convective rain rate [kg m**-2 s**-1]
- `csfr`: Convective snowfall rate water equivalent [kg m**-2 s**-1]
- `d2m`: **2 metre dewpoint temperature** [K]
- `dwi`: 10 metre wind direction [degrees]
- `fal`: Forecast albedo [(0 - 1)]
- `fg10`: 10 metre wind gust since previous post-processing [m s**-1]
- `flsr`: Forecast logarithm of surface roughness for heat [~]
- `fsr`: Forecast surface roughness [m]
- `i10fg`: Instantaneous 10 metre wind gust [m s**-1]
- `ie`: Instantaneous moisture flux [kg m**-2 s**-1]
- `iews`: Instantaneous eastward turbulent surface stress [N m**-2]
- `ilspf`: Instantaneous large-scale surface precipitation fraction [(0 - 1)]
- `inss`: Instantaneous northward turbulent surface stress [N m**-2]
- `ishf`: Instantaneous surface sensible heat flux [W m**-2]
- `istl1`: Ice temperature layer 1 [K]
- `istl2`: Ice temperature layer 2 [K]
- `istl3`: Ice temperature layer 3 [K]
- `istl4`: Ice temperature layer 4 [K]
- `lsm`: Land-sea mask [(0 - 1)]
- `lsrr`: Large scale rain rate [kg m**-2 s**-1]
- `lssfr`: Large scale snowfall rate water equivalent [kg m**-2 s**-1]
- `mbld`: Mean boundary layer dissipation [W m**-2]
- `mcpr`: Mean convective precipitation rate [kg m**-2 s**-1]
- `mcsr`: Mean convective snowfall rate [kg m**-2 s**-1]
- `mer`: Mean evaporation rate [kg m**-2 s**-1]
- `metss`: Mean eastward turbulent surface stress [N m**-2]
- `mlspf`: Mean large-scale precipitation fraction [Proportion]
- `mlspr`: Mean large-scale precipitation rate [kg m**-2 s**-1]
- `mlssr`: Mean large-scale snowfall rate [kg m**-2 s**-1]
- `mn2t`: Minimum temperature at 2 metres since previous post-processing [K]
- `mntpr`: Minimum total precipitation rate since previous post-processing [kg m**-2 s**-1]
- `mntss`: Mean northward turbulent surface stress [N m**-2]
- `mper`: Mean potential evaporation rate [kg m**-2 s**-1]
- `mror`: Mean runoff rate [kg m**-2 s**-1]
- `msdrswrf`: Mean surface direct short-wave radiation flux [W m**-2]
- `msdrswrfcs`: Mean surface direct short-wave radiation flux, clear sky [W m**-2]
- `msdwlwrf`: Mean surface downward long-wave radiation flux [W m**-2]
- `msdwlwrfcs`: Mean surface downward long-wave radiation flux, clear sky [W m**-2]
- `msdwswrf`: Mean surface downward short-wave radiation flux [W m**-2]
- `msdwswrfcs`: Mean surface downward short-wave radiation flux, clear sky [W m**-2]
- `msdwuvrf`: Mean surface downward UV radiation flux [W m**-2]
- `mser`: Mean snow evaporation rate [kg m**-2 s**-1]
- `msl`: Mean sea level pressure [Pa]
- `mslhf`: Mean surface latent heat flux [W m**-2]
- `msmr`: Mean snowmelt rate [kg m**-2 s**-1]
- `msnlwrf`: Mean surface net long-wave radiation flux [W m**-2]
- `msnlwrfcs`: Mean surface net long-wave radiation flux, clear sky [W m**-2]
- `msnswrf`: Mean surface net short-wave radiation flux [W m**-2]
- `msnswrfcs`: Mean surface net short-wave radiation flux, clear sky [W m**-2]
- `msr`: Mean snowfall rate [kg m**-2 s**-1]
- `msror`: Mean surface runoff rate [kg m**-2 s**-1]
- `msshf`: Mean surface sensible heat flux [W m**-2]
- `mssror`: Mean sub-surface runoff rate [kg m**-2 s**-1]
- `mtdwswrf`: Mean top downward short-wave radiation flux [W m**-2]
- `mtnlwrf`: Mean top net long-wave radiation flux [W m**-2]
- `mtnlwrfcs`: Mean top net long-wave radiation flux, clear sky [W m**-2]
- `mtnswrf`: Mean top net short-wave radiation flux [W m**-2]
- `mtnswrfcs`: Mean top net short-wave radiation flux, clear sky [W m**-2]
- `mtpr`: Mean total precipitation rate [kg m**-2 s**-1]
- `mx2t`: Maximum temperature at 2 metres since previous post-processing [K]
- `mxtpr`: Maximum total precipitation rate since previous post-processing [kg m**-2 s**-1]
- `p140209`: Air density over the oceans [kg m**-3]
- `ptype`: Precipitation type [code table (4.201)]
- `rsn`: Snow density [kg m**-3]
- `sd`: Snow depth [m of water equivalent]
- `siconc`: **Sea ice area fraction** [(0 - 1)]
- `skt`: Skin temperature [K]
- `sp`: Surface pressure [Pa]
- `sst`: Sea surface temperature [K]
- `t2m`: **2 metre temperature** [K]
- `tauoc`: Normalized stress into ocean [dimensionless]
- `tsn`: Temperature of snow layer [K]
- `u10`: **10 metre U wind component** [m s**-1]
- `u100`: 100 metre U wind component [m s**-1]
- `u10n`: Neutral wind at 10 m u-component [m s**-1]
- `v10`: **10 metre V wind component** [m s**-1]
- `v100`: 100 metre V wind component [m s**-1]
- `v10n`: Neutral wind at 10 m v-component [m s**-1]
- `wind`: 10 metre wind speed [m s**-1]
- `wmb`: Model bathymetry [m]
- `z`: Geopotential [m**2 s**-2]
- `zust`: Friction velocity [m s**-1]

In [ ]:
# list all available ERA5 variables
ERA5varskeys = list(ERA5vars.keys())
ERA5varskeys.sort()
md = '| name | longname [units] | dimensions | chunking | path |\n'
md += '| --- | --- | --- | --- | --- |\n'
for k in ERA5varskeys:
    fvar = ERA5vars[k]['fvar']
    md += '| '+' | '.join([ '`'+k+'`', 
                           ERA5vars[k]['attrs']['long_name']+' ['+ERA5vars[k]['attrs']['units']+']', 
                           repr(ERA5vars[k]['dimensions']),
                           repr(ERA5vars[k]['chunking']),
                           '`'+os.path.join(ERA5path, fvar, '*', fvar+'*.nc')+'`'])+' |\n'
display(Markdown(md))
# print(md)

In [ ]:
# find all ERA5 files in ERA5path with bad chunking
# this is slow! and was only needed to send info to NCI
ERA5varskeys = list(ERA5vars.keys())
ERA5varskeys.sort()
for vname in tqdm_notebook(ERA5varskeys, leave=False, desc='scanning ERA5 variables'):
    fvar = ERA5vars[vname]['fvar']
    filenames = os.path.join(ERA5path, fvar, '*', fvar+'*.nc')
    files = glob(filenames)
    files.sort()
    allchunks = []
    badchunks = []
    for f in tqdm_notebook(files, leave=False, desc='scanning '+vname+' files'):
#         print(vname, f)
#         ds = xr.open_mfdataset(
#                 f,
#                 parallel=True,
#                 autoclose=True,
#                 decode_times=False)
        var = nc.Dataset(f).variables[vname]
        chunk = var.chunking()
        allchunks.append(chunk)
        if chunk[0] > 200:
            badchunks.append(repr(chunk)+' '+f)
#         break
    unique_allchunks = [repr(list(x)) for x in set(tuple(x) for x in allchunks)]
    unique_allchunks.sort()
    print(vname, repr(ERA5vars[vname]['dimensions']), ERA5vars[vname]['attrs']['long_name'])
    print(filenames)
    print('all chunkings found in', vname, 'files:')
    for ch in unique_allchunks:
          print(ch)
    print('bad chunking:')
    for ch in badchunks:
          print(ch)
#     break
    

        
#     md += '| '+' | '.join([ '`'+k+'`', 
#                            ERA5vars[k]['attrs']['long_name']+' ['+ERA5vars[k]['attrs']['units']+']', 
#                            repr(ERA5vars[k]['dimensions']),
#                            repr(ERA5vars[k]['chunking']),
#                            '`'+os.path.join(ERA5path, fvar, '*', fvar+'*.nc')+'`'])+' |\n'
# display(Markdown(md))
# # print(md)

In [ ]:
def get_ERA5(variable, fvar=None, path=ERA5path, rechunkedpath=ERA5rechunkedpath,
                timerange=timerange): #, time_units='days since 1900-01-01'): 
    '''
    Return a dataarray for an ERA5 variable (e.g. ERA5vars keys).
    fvar is the variable name in the filename and path. 
    If fvar is omitted, it is looked up in ERA5vars.
    timerange is a slice object.
    If files with the same basename exist in both ERA5path and ERA5rechunkedpath,
    the version in ERA5rechunkedpath is used instead of that in ERA5path.
    '''
    if fvar is None:
        try:
            fvar = ERA5vars[variable]['fvar']
        except KeyError:
            fvar = variable  # wild guess, hope for the best...
    files = []
    for yr in range( timerange.start.year, 
                    (timerange.stop - pd.DateOffset(seconds=1)).year+1):
        newfiles = glob(os.path.join(path, fvar, str(yr), fvar+'*.nc'))
        rechunkedfiles = glob(os.path.join(rechunkedpath, fvar, str(yr), fvar+'*.nc'))
        newfilebns = [os.path.basename(p) for p in newfiles]
        rechunkedfilebns = [os.path.basename(p) for p in rechunkedfiles]
        for rcf, rcfbn in zip(rechunkedfiles, rechunkedfilebns):
            try:
                i = newfilebns.index(rcfbn)
                print('replacing', newfiles[i], 'with rechunked version', rcf)
                newfiles[i] = rcf
            except ValueError:
                pass   
        files.extend(newfiles)
    files.sort(key=lambda p: os.path.basename(p))

    dataarray = xr.open_mfdataset(
                files,
                # chunks=8,  
                chunks={'time': 8, 'longitude': None, 'latitude': None},  # 8 1-hourly slices per chunk, matching chunking for most ERA5 NetCDF files
                # chunks={'time': 24, 'longitude': 1, 'latitude': 1},  # 24 1-hourly slices per chunk
                parallel=True,
                autoclose=True,
                decode_times=False)[variable]
    if 'time' in dataarray.coords:
        time_units = dataarray.time.units
        decoded_time = xr.conventions.times.decode_cf_datetime(dataarray.time, time_units)
        dataarray.coords['time'] = ('time', decoded_time,
                                    {'long_name': 'time', 'decoded_using': time_units })
#     # use the same coord names as access-om2 t grid
#     dataarray = dataarray.rename(({'xgrid': 'xt_ocean', 'ygrid': 'yt_ocean'}))
    
    # sort by time and select timerange
#     dataarray = dataarray.sortby('time').sel(time=timerange)
    dataarray = dataarray.sel(time=timerange)

    return dataarray

In [ ]:
def loadERA5(ide, varnames=[], **kwargs):
    for varname in varnames:
        if varname not in ide.keys():
            print('loading', varname)
            var = get_ERA5(varname, **kwargs)

            ide[varname] = var
            
            vard = var.resample(time='d').mean() # daily averages
            vard.attrs = var.attrs
            ide[varname+'_d'] = vard
            
            varm = var.resample(time='m').mean() # monthly averages
            varm.attrs = var.attrs
            ide[varname+'_m'] = varm

            varmm = var.groupby('time.month').mean('time', skipna=True)
            varmm.attrs = var.attrs
            ide[varname+'_mm'] = varmm

### List JRA55-do variable names

In [ ]:
JRA55dopath = '/g/data/ik11/inputs/JRA-55/MRI-JRA55-do/MRI-JRA55-do-1-4-0'
JRA55dovars = dict()  # keys are all ERA5 variables in ERA5path; values are dicts with keys 'fvar' and 'attrs'
for p in tqdm_notebook(glob(os.path.join(JRA55dopath, '*', '*')), leave=False, desc='scanning for JRA55do variable names'):
    files = glob(os.path.join(p, '*.nc'))
    if len(files) > 0:
        file = files[0]
#         print(file)
        da = xr.open_mfdataset(
                file,
                parallel=True,
                autoclose=True,
                decode_times=False)
        vname = [ v for v in da.variables if v not in list(da.coords) and not v.endswith('_bnds')][0]  # assumes only one data variable per file
#         print(vname)
        JRA55dovars[vname] = {'path': os.path.dirname(file),
                           'attrs': da[vname].attrs}

In [ ]:
# list all available JRA55-do variables
JRA55dovarskeys = list(JRA55dovars.keys())
JRA55dovarskeys.sort()
md = '| name | longname [units] | path |\n'
md += '| --- | --- | --- |\n'
for k in JRA55dovarskeys:
    md += '| '+' | '.join([ '`'+k+'`', 
                           JRA55dovars[k]['attrs']['long_name']+' ['+JRA55dovars[k]['attrs']['units']+']', 
                           '`'+os.path.join(JRA55dovars[k]['path'],'*.nc')+'`'])+' |\n'
display(Markdown(md))
# print(md)

## Load data

### Load ACCESS-OM2 model data

In [ ]:
ice_data = exptdata.exptdict

In [ ]:
ensemble = [ k for k in ice_data.keys() if len(k.split('_')) > 1 ]
# ensemble = [ k for k in ensemble if ice_data[k][0]['res'] in ['0.25°'] ]

ensemble = [ k for k in ensemble if ice_data[k][0]['res'] in ['1°'] ]

# ensemble = [ k for k in ensemble if ice_data[k][0]['perturbed'] not in ['turning_angle'] ]

# ensemble = [ k for k in ensemble if ice_data[k][0]['perturbed'] in ['j09_bgmax', None] ]
# ensemble = [ k for k in ensemble if ice_data[k][0]['perturbed'] in ['albicev', None] ]


# ensemble = [ k for k in ensemble if ice_data[k][0]['perturbed'] in [None] ]  # control run only
ensemble.sort()
controlkeys = [ k for k in ensemble if ice_data[k][0]['perturbed']==None ]

In [ ]:
ensemble = ['1deg', '025deg', '01deg']
# ensemble = ['025deg', '01deg']
controlkeys = ensemble

In [ ]:
ensemble = ['1deg_albicev_0.9', '1deg_turning_angle_26', '1deg_control']
controlkeys = ['1deg_control']

In [ ]:
ensemble = ['1degERA5', '025degERA5']
controlkeys = ensemble

In [ ]:
ensemble

In [ ]:
controlkeys

In [ ]:
# for ekey in ice_data.keys():
for ekey in ensemble: # just the ensemble
    print(ekey)
    for ide in ice_data[ekey]:
        loaddata(ide, varnames=['aice_m'])

#         loaddata(ide, varnames=['daidtd_m', 'daidtt_m'])  # area tendency due to dynamics and thermo
#         loaddata(ide, varnames=['dvidtd_m', 'dvidtt_m'])  # volume tendency due to dynamics and thermo
#         loaddata(ide, varnames=['congel_m', 'frazil_m', 'meltt_m', 'meltb_m', 'meltl_m', 'snoice_m', 'evap_ai_m'])  # volume tendency due to dynamics and thermo

        # loaddata(ide, varnames=['hi_m'])
        # loaddata(ide, varnames=['hs_m'])
#         loaddata(ide, varnames=['hi'])

#         loaddata(ide, varnames=['mld'], model='mom', ncfiles=['ocean-2d-mld-1-daily-mean-ym_%'])


#         loaddata(ide, varnames=['aice'])
#         if ekey == '01deg':
# #             loaddata(ide, varnames=['sea_level', 'surface_temp', 'surface_salt', 'mld'], 
# #                      ncfiles=['ocean-2d-sea_level-1-daily-mean-ym_%', 'ocean-2d-surface_temp-1-daily-mean-ym_%', 'ocean-2d-surface_salt-1-daily-mean-ym_%', 'ocean-2d-mld-1-daily-mean-ym_%'])

#             loaddata(ide, model='mom', varnames=['surface_temp', 'surface_salt', 'mld'], 
#                      ncfiles=['ocean-2d-surface_temp-1-daily-mean-ym_%', 'ocean-2d-surface_salt-1-daily-mean-ym_%', 'ocean-2d-mld-1-daily-mean-ym_%'])
#             ide['sst'] = ide['surface_temp'] # ensure consistent names
#             ide['sss'] = ide['surface_salt'] # ensure consistent names

#         else:
#             loaddata(ide, model='mom', varnames=['sst', 'sss', 'mld'], ncfiles='ocean_daily.nc')
# #             loaddata(ide, varnames=['sea_level', 'sst', 'sss', 'mld'], ncfiles='ocean_daily.nc')

#         break  # only load cycle 1
    
#     break

### Load SIC obs data

In [ ]:
obs_NH = get_sic_obs('north/monthly/*.nc')
obs_NH_mm = obs_mm(obs_NH)

In [ ]:
obs_SH = get_sic_obs('south/monthly/*.nc')
obs_SH_mm = obs_mm(obs_SH)

In [ ]:
obs_SH_mstd = obs_mstd(obs_SH)

In [ ]:
# Total ice area timeseries
ObsDirExt = '/g/data3/hh5/tmp/cosima/observations/NOAA/G02135'  # from http://nsidc.org/data/g02135
obsExtNHFileList = glob(os.path.join(ObsDirExt, 'north/monthly/data/*.csv'))
obsExtSHFileList = glob(os.path.join(ObsDirExt, 'south/monthly/data/*.csv'))
obsExtNHFileList.sort()
obsExtSHFileList.sort()

### Load GIOMAS data

In [ ]:
GIOMAS = {}  # interannually varying monthly means
# where possible, use the same names as for access-om2 
for var, kwargs in \
           {'hi_m': {'variable': 'heff', 'varrange': [0, None]},
            'hs_m': {'variable': 'snow', 'varrange': [0, None], 'scale': 1/.33},  # convert from water equivalent using CICE snow density / water density = .330
            'aice_m': {'variable': 'area', 'varrange': [0, 1]},
            'iceprod_m': {'variable': 'iceprod', 'varrange': [None, None]}
           }.items():
    print('loading', var)
    GIOMAS[var] = get_giomas(**kwargs)
GIOMAS_mm = {k: v.groupby('time.month').mean('time', skipna=True) for k, v in GIOMAS.items()}  # monthly climatologies

### Load JRA55-do data
JRA55-do has sea ice concentration `siconc`, `siconca` in

`/g/data/qv56/replicas/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-4-0/seaIce`

which is now indexed in the cookbook https://github.com/COSIMA/master_index/issues/11

Presumably these are both from COBE-SST - see JMA2006a, MatsumotoIshiiFukudaHirahara2006a, IshiiShoujiSugimotoMatsumoto2005a
- `siconc` is daily, on the ocean grid
    - .nc metadata: :grid = "1x1 degree latitude x longitude" ;
    - .nc metadata: :nominal_resolution = "100 km" ;
    - continuous range of values
    - presumably this is the original COBE-SST data before the 55% thresholding and regridding
- `siconca` is 3-hourly, on the atmos grid, **assimilated into JRA55**
    - .nc metadata: :grid = "data regridded to the normal atmosphere TL319 gaussian grid (320x640 latxlon) from a reduced TL319 gaussian grid" ;
    - .nc metadata: :nominal_resolution = "50 km" ;
    - almost all values are 0 or 100% (there are a few of 20, 40, 60, 80% at edges of 100% regions, presumably due to iterpolation onto atmos grid, and perhaps in time to get 3-hourly data from daily siconc)
    - so presumably this is within the 55% contour as seen by the atmosphere model
   
so`siconc` is original ice data, `siconca` is thresholded - confirmed by comparing these:

`/g/data/qv56/replicas/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-4-0/seaIce/day/siconc/gn/v20190429/siconc_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-4-0_gn_20180101-20181231.nc`

`/g/data/qv56/replicas/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-4-0/seaIce/3hrPt/siconca/gr/v20190429/siconca_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-4-0_gr_201801010000-201812312100.nc`


In [ ]:
JRA55do_data = {'session': cc.database.create_session()}

In [ ]:
loadJRA55do(JRA55do_data, varnames=['siconc'])  # COBE-SST sea ice concentration

In [ ]:
loadJRA55do(JRA55do_data, varnames=['siconca'])  # COBE-SST sea ice concentration, thresholded daily at 55% and interpolated to 3-hourly - used as input to JRA55 reanalysis

In [ ]:
loadJRA55do(JRA55do_data, varnames=['uas', 'psl', 'tas', 'rlds', 'rsds']) # also 'vas', ts', 'huss', 'prra', 'prsn'

In [ ]:
loadJRA55do(JRA55do_data, varnames=['tas', 'huss']) # also 'vas', ts', 'huss', 'prra', 'prsn'

In [ ]:
loadJRA55do(JRA55do_data, varnames=['tas']) # also 'vas', ts', 'huss', 'prra', 'prsn'

In [ ]:
loadJRA55do(JRA55do_data, varnames=['uas', 'vas']) # also 'vas', ts', 'huss', 'prra', 'prsn'

### Load JRA55 (not -do) data

In [ ]:
JRA55_data = dict()

In [ ]:
loadJRA55(JRA55_data, varnames=['tas'])

In [ ]:
loadJRA55(JRA55_data, varnames=['hus'])

### Load ERA5 data

In [ ]:
ERA5_data = dict()

In [ ]:
loadERA5(ERA5_data, varnames=['t2m'])

In [ ]:
loadERA5(ERA5_data, varnames=['u10', 'v10'])

# Regridders
see https://cosima-recipes.readthedocs.io/en/latest/documented_examples/Regridding.html
and https://xesmf.readthedocs.io

In [ ]:
def mom_regridder(grid_in, grid_out, method='bilinear', periodic=True, **kwargs):
    '''Return a function that takes one dataarray argument on grid_in and returns that dataarray regridded onto grid_out.
    
    grid_in and grid_out are paths to MOM grid NetCDF files.
    The regridder maps tracer points to tracer points.
    '''
    weightfn = '_'.join(['regrid', 'weights', os.path.splitext(os.path.basename(grid_in))[0],
                         'to', os.path.splitext(os.path.basename(grid_out))[0], method])+'.nc'
    def opends(fn):
        try: # for /g/data/ik11/grids/ocean_grid_10.nc and /g/data/ik11/grids/ocean_grid_025.nc
            return xr.open_dataset(fn).rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'geolon_t': 'lon', 'geolat_t': 'lat'})         
        except ValueError: # for /g/data/ik11/grids/ocean_grid_01.nc - see https://github.com/COSIMA/access-om2/issues/236
            ds = xr.open_dataset(fn).rename({'grid_x_T': 'x', 'grid_y_T': 'y', 'geolon_t': 'lon', 'geolat_t': 'lat'})
            ds_fix = xr.open_dataset('/g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_cycle3/output488/ocean/ocean-3d-temp-1-daily-mean-ym_1958_01.nc')\
                        .rename({'xt_ocean': 'x', 'yt_ocean': 'y'})
            return ds.assign_coords({'x': ds_fix['x'], 'y': ds_fix['y']})

    ds_in = opends(grid_in)
    ds_out = opends(grid_out)

    rg = xesmf.Regridder(ds_in, ds_out, method=method, periodic=periodic, reuse_weights=os.path.exists(weightfn), filename=weightfn, **kwargs)

    def outf(da): # define regridding function to output
        outds = rg(da.chunk({'xt_ocean': None, 'yt_ocean': None}))
        outds.coords['x'] = ds_out['x']
        outds.coords['y'] = ds_out['y']
        return outds.rename({'x': 'xt_ocean', 'y': 'yt_ocean', 'lon': 'longitude', 'lat': 'latitude'})

    return outf

In [ ]:
def G02202_obs_regridder(grid_in, grid_out, method='bilinear', **kwargs):
    '''Return a function that takes one dataarray argument on grid_in and returns that dataarray regridded onto grid_out.
    
    grid_in -- path to a NSIDC G02202 NetCDF grid file
    grid_out -- path to a MOM grid file (tracer points are used)
    '''
    weightfn = '_'.join(['regrid', 'weights', os.path.splitext(os.path.basename(grid_in))[0],
                         'to', os.path.splitext(os.path.basename(grid_out))[0], method])+'.nc'
    ds_in = xr.open_dataset(grid_in)\
             .rename({'xgrid': 'x', 'ygrid': 'y',
                     'longitude': 'lon', 'latitude': 'lat'})
    try: # for /g/data/ik11/grids/ocean_grid_10.nc and /g/data/ik11/grids/ocean_grid_025.nc
        ds_out = xr.open_dataset(grid_out).rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'geolon_t': 'lon', 'geolat_t': 'lat'})
    except ValueError: # for /g/data/ik11/grids/ocean_grid_01.nc - see https://github.com/COSIMA/access-om2/issues/236
        ds_out = xr.open_dataset(grid_out).rename({'grid_x_T': 'x', 'grid_y_T': 'y', 'geolon_t': 'lon', 'geolat_t': 'lat'})
        ds_out_fix = xr.open_dataset('/g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_cycle3/output488/ocean/ocean-3d-temp-1-daily-mean-ym_1958_01.nc')\
                    .rename({'xt_ocean': 'x', 'yt_ocean': 'y'})
        ds_out = ds_out.assign_coords({'x': ds_out_fix['x'], 'y': ds_out_fix['y']})

    # make regridder
    rg = xesmf.Regridder(ds_in, ds_out, method=method, reuse_weights=os.path.exists(weightfn), filename=weightfn, **kwargs)

    def outf(da): # define regridding function to output
        outds = rg(da)
        outds.coords['x'] = ds_out['x']
        outds.coords['y'] = ds_out['y']
        return outds.rename({'x': 'xt_ocean', 'y': 'yt_ocean', 'lon': 'longitude', 'lat': 'latitude'})

    return outf

In [ ]:
def giomas_regridder(grid_out, method='bilinear', periodic=True, **kwargs):
    '''Return a function that takes one dataarray argument from get_giomas and returns that dataarray regridded onto grid_out.
    
    grid_out -- path to a MOM grid file (tracer points are used)
    '''
    weightfn = '_'.join(['regrid', 'weights', 'giomas',
                         'to', os.path.splitext(os.path.basename(grid_out))[0], method])+'.nc'

    ds_in = get_giomas(timerange=slice(pd.to_datetime('2000', format='%Y'), pd.to_datetime('2001', format='%Y')))\
                .isel(time=0).rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'longitude': 'lon', 'latitude': 'lat'})

    try: # for /g/data/ik11/grids/ocean_grid_10.nc and /g/data/ik11/grids/ocean_grid_025.nc
        ds_out = xr.open_dataset(grid_out).rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'geolon_t': 'lon', 'geolat_t': 'lat'})
    except ValueError: # for /g/data/ik11/grids/ocean_grid_01.nc - see https://github.com/COSIMA/access-om2/issues/236
        ds_out = xr.open_dataset(grid_out).rename({'grid_x_T': 'x', 'grid_y_T': 'y', 'geolon_t': 'lon', 'geolat_t': 'lat'})
        ds_out_fix = xr.open_dataset('/g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_cycle3/output488/ocean/ocean-3d-temp-1-daily-mean-ym_1958_01.nc')\
                    .rename({'xt_ocean': 'x', 'yt_ocean': 'y'})
        ds_out = ds_out.assign_coords({'x': ds_out_fix['x'], 'y': ds_out_fix['y']})

    # make regridder
    rg = xesmf.Regridder(ds_in, ds_out, method=method, periodic=periodic,
                         reuse_weights=os.path.exists(weightfn), 
                         filename=weightfn, **kwargs)

    def outf(da): # define regridding function to output
        outds = rg(da)
        outds.coords['x'] = ds_out['x']
        outds.coords['y'] = ds_out['y']
        return outds.rename({'x': 'xt_ocean', 'y': 'yt_ocean', 'lon': 'longitude', 'lat': 'latitude'})

    return outf

In [ ]:
def JRA55do_regridder(grid_in, grid_out, method='bilinear', periodic=True, **kwargs):
    '''Return a function that takes one dataarray argument on grid_in and returns that dataarray regridded onto grid_out.
    
    grid_in -- path to a JRA55-do or JRA55 NetCDF grid file
    grid_out -- path to a MOM grid file (tracer points are used)
    '''
    weightfn = '_'.join(['regrid', 'weights', os.path.splitext(os.path.basename(grid_in))[0],
                         'to', os.path.splitext(os.path.basename(grid_out))[0], method])+'.nc'
    ds_in = xr.open_dataset(grid_in)
    try: # for /g/data/ik11/grids/ocean_grid_10.nc and /g/data/ik11/grids/ocean_grid_025.nc
        ds_out = xr.open_dataset(grid_out).rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'geolon_t': 'lon', 'geolat_t': 'lat'})
    except ValueError: # for /g/data/ik11/grids/ocean_grid_01.nc - see https://github.com/COSIMA/access-om2/issues/236
        ds_out = xr.open_dataset(grid_out).rename({'grid_x_T': 'x', 'grid_y_T': 'y', 'geolon_t': 'lon', 'geolat_t': 'lat'})
        ds_out_fix = xr.open_dataset('/g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_cycle3/output488/ocean/ocean-3d-temp-1-daily-mean-ym_1958_01.nc')\
                    .rename({'xt_ocean': 'x', 'yt_ocean': 'y'})
        ds_out = ds_out.assign_coords({'x': ds_out_fix['x'], 'y': ds_out_fix['y']})

    # make regridder
    rg = xesmf.Regridder(ds_in, ds_out, method=method, reuse_weights=os.path.exists(weightfn), periodic=periodic, filename=weightfn, **kwargs)

    def outf(da): # define regridding function to output
        outds = rg(da)
        outds.coords['x'] = ds_out['x']
        outds.coords['y'] = ds_out['y']
        return outds.rename({'x': 'xt_ocean', 'y': 'yt_ocean', 'lon': 'longitude', 'lat': 'latitude'})

    return outf

In [ ]:
def JRA55do_to_G02202_obs_regridder(grid_in, grid_out, method='bilinear', **kwargs):
    '''Return a function that takes one dataarray argument on grid_in and returns that dataarray regridded onto grid_out.
    
    grid_in -- path to a JRA55-do or JRA55 NetCDF grid file
    grid_out -- path to a NSIDC G02202 NetCDF grid file
    '''
    weightfn = '_'.join(['regrid', 'weights', os.path.splitext(os.path.basename(grid_in))[0],
                         'to', os.path.splitext(os.path.basename(grid_out))[0], method])+'.nc'
    ds_in = xr.open_dataset(grid_in)

    ds_out = xr.open_dataset(grid_out)\
             .rename({'xgrid': 'x', 'ygrid': 'y',
                     'longitude': 'lon', 'latitude': 'lat'})

    # make regridder
    rg = xesmf.Regridder(ds_in, ds_out, method=method, reuse_weights=os.path.exists(weightfn), filename=weightfn, **kwargs)

    def outf(da): # define regridding function to output
        outds = rg(da)
        outds.coords['x'] = ds_out['x']
        outds.coords['y'] = ds_out['y']
        return outds.rename({'x': 'xt_ocean', 'y': 'yt_ocean', 'lon': 'longitude', 'lat': 'latitude'})

    return outf

In [ ]:
def JRA55do_to_ERA5_regridder(grid_in, grid_out, method='bilinear', **kwargs):
    '''Return a function that takes one dataarray argument on grid_in and returns that dataarray regridded onto grid_out.
    
    grid_in -- path to a JRA55-do or JRA55 NetCDF grid file
    grid_out -- path to an ERA5 NetCDF grid file
    '''
    weightfn = '_'.join(['regrid', 'weights', os.path.splitext(os.path.basename(grid_in))[0],
                         'to', os.path.splitext(os.path.basename(grid_out))[0], method])+'.nc'
    ds_in = xr.open_dataset(grid_in)

    ds_out = xr.open_dataset(grid_out)\
             .rename({'longitude': 'lon', 'latitude': 'lat'})
    ds_out.coords['x'] = ds_out['lon']
    ds_out.coords['y'] = ds_out['lat']

    # make regridder
    rg = xesmf.Regridder(ds_in, ds_out, method=method, reuse_weights=os.path.exists(weightfn), filename=weightfn, periodic=True, **kwargs)

    def outf(da): # define regridding function to output
        outds = rg(da)
        outds.coords['x'] = ds_out['x']
        outds.coords['y'] = ds_out['y']
        return outds.rename({'x': 'xt_ocean', 'y': 'yt_ocean', 'lon': 'longitude', 'lat': 'latitude'})

    return outf

In [ ]:
def ERA5_regridder(grid_in, grid_out, method='bilinear', periodic=True, **kwargs):
    '''Return a function that takes one dataarray argument on grid_in and returns that dataarray regridded onto grid_out.
    
    grid_in -- path to a ERA5 NetCDF grid file
    grid_out -- path to a MOM grid file (tracer points are used)
    '''
    weightfn = '_'.join(['regrid', 'weights', os.path.splitext(os.path.basename(grid_in))[0],
                         'to', os.path.splitext(os.path.basename(grid_out))[0], method])+'.nc'
    ds_in = xr.open_dataset(grid_in)\
             .rename({'longitude': 'lon', 'latitude': 'lat'})
    try: # for /g/data/ik11/grids/ocean_grid_10.nc and /g/data/ik11/grids/ocean_grid_025.nc
        ds_out = xr.open_dataset(grid_out).rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'geolon_t': 'lon', 'geolat_t': 'lat'})
    except ValueError: # for /g/data/ik11/grids/ocean_grid_01.nc - see https://github.com/COSIMA/access-om2/issues/236
        ds_out = xr.open_dataset(grid_out).rename({'grid_x_T': 'x', 'grid_y_T': 'y', 'geolon_t': 'lon', 'geolat_t': 'lat'})
        ds_out_fix = xr.open_dataset('/g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_cycle3/output488/ocean/ocean-3d-temp-1-daily-mean-ym_1958_01.nc')\
                    .rename({'xt_ocean': 'x', 'yt_ocean': 'y'})
        ds_out = ds_out.assign_coords({'x': ds_out_fix['x'], 'y': ds_out_fix['y']})

    # make regridder
    rg = xesmf.Regridder(ds_in, ds_out, method=method, reuse_weights=os.path.exists(weightfn), periodic=periodic, filename=weightfn, **kwargs)

    def outf(da): # define regridding function to output
        outds = rg(da)
        outds.coords['x'] = ds_out['x']
        outds.coords['y'] = ds_out['y']
        return outds.rename({'x': 'xt_ocean', 'y': 'yt_ocean', 'lon': 'longitude', 'lat': 'latitude'})

    return outf

In [ ]:
def ERA5_to_JRA55do_regridder(grid_in, grid_out, method='bilinear', **kwargs):
    '''Return a function that takes one dataarray argument on grid_in and returns that dataarray regridded onto grid_out.
    
    grid_in -- path to an ERA5 NetCDF grid file
    grid_out -- path to a JRA55-do or JRA55 NetCDF grid file
    '''
    weightfn = '_'.join(['regrid', 'weights', os.path.splitext(os.path.basename(grid_in))[0],
                         'to', os.path.splitext(os.path.basename(grid_out))[0], method])+'.nc'
    ds_in = xr.open_dataset(grid_in)\
             .rename({'longitude': 'lon', 'latitude': 'lat'})
    # ds_in.coords['x'] = ds_in['lon']
    # ds_in.coords['y'] = ds_in['lat']

    ds_out = xr.open_dataset(grid_out)
    ds_out.coords['x'] = ds_out['lon']
    ds_out.coords['y'] = ds_out['lat']

    # make regridder
    rg = xesmf.Regridder(ds_in, ds_out, method=method, reuse_weights=os.path.exists(weightfn), filename=weightfn, periodic=True, **kwargs)

    def outf(da): # define regridding function to output
        outds = rg(da)
        outds.coords['x'] = ds_out['x']
        outds.coords['y'] = ds_out['y']
        return outds.rename({'x': 'xt_ocean', 'y': 'yt_ocean', 'lon': 'longitude', 'lat': 'latitude'})

    return outf

In [ ]:
def JRA55_to_JRA55do_regridder(grid_in, grid_out, method='bilinear', **kwargs):
    '''Return a function that takes one dataarray argument on grid_in and returns that dataarray regridded onto grid_out.
    
    grid_in -- path to a JRA55 NetCDF grid file
    grid_out -- path to a JRA55-do NetCDF grid file
    '''
    weightfn = '_'.join(['regrid', 'weights', os.path.splitext(os.path.basename(grid_in))[0],
                         'to', os.path.splitext(os.path.basename(grid_out))[0], method])+'.nc'
    ds_in = xr.open_dataset(grid_in)
    # ds_in.coords['x'] = ds_in['lon']
    # ds_in.coords['y'] = ds_in['lat']

    ds_out = xr.open_dataset(grid_out)
    ds_out.coords['x'] = ds_out['lon']
    ds_out.coords['y'] = ds_out['lat']

    # make regridder
    rg = xesmf.Regridder(ds_in, ds_out, method=method, reuse_weights=os.path.exists(weightfn), filename=weightfn, periodic=True, **kwargs)

    def outf(da): # define regridding function to output
        outds = rg(da)
        outds.coords['x'] = ds_out['x']
        outds.coords['y'] = ds_out['y']
        return outds.rename({'x': 'xt_ocean', 'y': 'yt_ocean', 'lon': 'longitude', 'lat': 'latitude'})

    return outf

In [ ]:
# define regridder functions
regrid_1_to_025 = mom_regridder('/g/data/ik11/grids/ocean_grid_10.nc', '/g/data/ik11/grids/ocean_grid_025.nc')#, method='conservative')
regrid_1_to_01 = mom_regridder('/g/data/ik11/grids/ocean_grid_10.nc', '/g/data/ik11/grids/ocean_grid_01.nc')#, method='conservative')
regrid_025_to_01 = mom_regridder('/g/data/ik11/grids/ocean_grid_025.nc', '/g/data/ik11/grids/ocean_grid_01.nc')#, method='conservative')
regrid_025_to_1 = mom_regridder('/g/data/ik11/grids/ocean_grid_025.nc', '/g/data/ik11/grids/ocean_grid_10.nc')#, method='conservative')
regrid_01_to_1 = mom_regridder('/g/data/ik11/grids/ocean_grid_01.nc', '/g/data/ik11/grids/ocean_grid_10.nc')#, method='conservative')
regrid_01_to_025 = mom_regridder('/g/data/ik11/grids/ocean_grid_01.nc', '/g/data/ik11/grids/ocean_grid_025.nc')#, method='conservative')

In [ ]:
obsfile_NH = '/g/data3/hh5/tmp/cosima/observations/NOAA/G02202_V3/north/monthly/seaice_conc_monthly_nh_f08_198708_v03r01.nc'
regrid_NHobs_to_1 = G02202_obs_regridder(obsfile_NH, '/g/data/ik11/grids/ocean_grid_10.nc')
regrid_NHobs_to_025 = G02202_obs_regridder(obsfile_NH, '/g/data/ik11/grids/ocean_grid_025.nc')
regrid_NHobs_to_01 = G02202_obs_regridder(obsfile_NH, '/g/data/ik11/grids/ocean_grid_01.nc')

obsfile_SH = '/g/data3/hh5/tmp/cosima/observations/NOAA/G02202_V3/south/monthly/seaice_conc_monthly_sh_f08_198708_v03r01.nc'
regrid_SHobs_to_1 = G02202_obs_regridder(obsfile_SH, '/g/data/ik11/grids/ocean_grid_10.nc')
regrid_SHobs_to_025 = G02202_obs_regridder(obsfile_SH, '/g/data/ik11/grids/ocean_grid_025.nc')
regrid_SHobs_to_01 = G02202_obs_regridder(obsfile_SH, '/g/data/ik11/grids/ocean_grid_01.nc')

obsregridders = {'NH': 
                 {
                     '1°': regrid_NHobs_to_1,
                     '0.25°': regrid_NHobs_to_025,
                     '0.1°': regrid_NHobs_to_01
                 },
                'SH':
                 {
                     '1°': regrid_SHobs_to_1,
                     '0.25°': regrid_SHobs_to_025,
                     '0.1°': regrid_SHobs_to_01
                 }
                }

In [ ]:
regrid_giomas_to_1 = giomas_regridder('/g/data/ik11/grids/ocean_grid_10.nc')
regrid_giomas_to_025 = giomas_regridder('/g/data/ik11/grids/ocean_grid_025.nc')
regrid_giomas_to_01 = giomas_regridder('/g/data/ik11/grids/ocean_grid_01.nc')
GIOMASregridders = {
                     '1°': regrid_giomas_to_1,
                     '0.25°': regrid_giomas_to_025,
                     '0.1°': regrid_giomas_to_01
                   }

In [ ]:
JRA55dofile = '/g/data/qv56/replicas/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-4-0/seaIce/day/siconc/gn/v20190429/siconc_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-4-0_gn_19580101-19581231.nc'
JRA55do_data['siconc_mm_regridded_to_NHobs'] = JRA55do_to_G02202_obs_regridder(JRA55dofile, obsfile_NH)(JRA55do_data['siconc_mm'])/100.0  # convert from percent
JRA55do_data['siconc_mm_regridded_to_SHobs'] = JRA55do_to_G02202_obs_regridder(JRA55dofile, obsfile_SH)(JRA55do_data['siconc_mm'])/100.0  # convert from percent
JRA55do_data['siconc_mm_regridded_to_mom'] = dict()
JRA55do_data['siconc_mm_regridded_to_mom']['1°'] = JRA55do_regridder(JRA55dofile, '/g/data/ik11/grids/ocean_grid_10.nc')(JRA55do_data['siconc_mm'])/100.0  # convert from percent
JRA55do_data['siconc_mm_regridded_to_mom']['0.25°'] = JRA55do_regridder(JRA55dofile, '/g/data/ik11/grids/ocean_grid_025.nc')(JRA55do_data['siconc_mm'])/100.0  # convert from percent
JRA55do_data['siconc_mm_regridded_to_mom']['0.1°'] = JRA55do_regridder(JRA55dofile, '/g/data/ik11/grids/ocean_grid_01.nc')(JRA55do_data['siconc_mm'])/100.0  # convert from percent

In [ ]:
JRA55dofile = '/g/data/qv56/replicas/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-4-0/seaIce/3hrPt/siconca/gr/v20190429/siconca_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-4-0_gr_201801010000-201812312100.nc'
JRA55do_data['siconca_mm_regridded_to_NHobs'] = JRA55do_to_G02202_obs_regridder(JRA55dofile, obsfile_NH)(JRA55do_data['siconca_mm'])/100.0  # convert from percent
JRA55do_data['siconca_mm_regridded_to_SHobs'] = JRA55do_to_G02202_obs_regridder(JRA55dofile, obsfile_SH)(JRA55do_data['siconca_mm'])/100.0  # convert from percent
JRA55do_data['siconca_mm_regridded_to_mom'] = dict()
JRA55do_data['siconca_mm_regridded_to_mom']['1°'] = JRA55do_regridder(JRA55dofile, '/g/data/ik11/grids/ocean_grid_10.nc')(JRA55do_data['siconca_mm'])/100.0  # convert from percent
JRA55do_data['siconca_mm_regridded_to_mom']['0.25°'] = JRA55do_regridder(JRA55dofile, '/g/data/ik11/grids/ocean_grid_025.nc')(JRA55do_data['siconca_mm'])/100.0  # convert from percent
JRA55do_data['siconca_mm_regridded_to_mom']['0.1°'] = JRA55do_regridder(JRA55dofile, '/g/data/ik11/grids/ocean_grid_01.nc')(JRA55do_data['siconca_mm'])/100.0  # convert from percent

In [ ]:
ERA5file = glob(os.path.join(ERA5path, ERA5vars['t2m']['fvar'], '2017', ERA5vars['t2m']['fvar']+'*.nc'))[0]
def ERA5_to_025(**kwargs):
    return ERA5_regridder(ERA5file, '/g/data/ik11/grids/ocean_grid_025.nc', **kwargs)

In [ ]:
JRA55dofile = '/g/data/qv56/replicas/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-4-0/seaIce/3hrPt/siconca/gr/v20190429/siconca_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-4-0_gr_201801010000-201812312100.nc'
def JRA55do_to_025(**kwargs):
    return JRA55do_regridder(JRA55dofile, '/g/data/ik11/grids/ocean_grid_025.nc', **kwargs)

In [ ]:
# JRA55dofile = '/g/data/qv56/replicas/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-4-0/seaIce/day/siconc/gn/v20190429/siconc_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-4-0_gn_19580101-19581231.nc'
JRA55dofile = '/g/data/qv56/replicas/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-4-0/seaIce/3hrPt/siconca/gr/v20190429/siconca_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-4-0_gr_201801010000-201812312100.nc'
ERA5file = glob(os.path.join(ERA5path, ERA5vars['t2m']['fvar'], '2017', ERA5vars['t2m']['fvar']+'*.nc'))[0]
def JRA55do_to_ERA5(data_in):
    return JRA55do_to_ERA5_regridder(JRA55dofile, ERA5file)(data_in)

In [ ]:
# JRA55dofile = '/g/data/qv56/replicas/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-4-0/seaIce/day/siconc/gn/v20190429/siconc_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-4-0_gn_19580101-19581231.nc'
JRA55dofile = '/g/data/qv56/replicas/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-4-0/seaIce/3hrPt/siconca/gr/v20190429/siconca_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-4-0_gr_201801010000-201812312100.nc'
ERA5file = glob(os.path.join(ERA5path, ERA5vars['t2m']['fvar'], '2017', ERA5vars['t2m']['fvar']+'*.nc'))[0]
def ERA5_to_JRA55do(data_in):
    return ERA5_to_JRA55do_regridder(ERA5file, JRA55dofile)(data_in)

In [ ]:
JRA55file = os.path.join(JRA55path, 'tas_Amon_reanalysis_JRA-55_195801-201912.nc')
def JRA55_to_025(**kwargs):
    return JRA55do_regridder(JRA55file, '/g/data/ik11/grids/ocean_grid_025.nc', **kwargs)

In [ ]:
JRA55file = os.path.join(JRA55path, 'tas_Amon_reanalysis_JRA-55_195801-201912.nc')
ERA5file = glob(os.path.join(ERA5path, ERA5vars['t2m']['fvar'], '2017', ERA5vars['t2m']['fvar']+'*.nc'))[0]
def JRA55_to_ERA5(data_in):
    return JRA55do_to_ERA5_regridder(JRA55file, ERA5file)(data_in)

In [ ]:
JRA55file = os.path.join(JRA55path, 'tas_Amon_reanalysis_JRA-55_195801-201912.nc')
ERA5file = glob(os.path.join(ERA5path, ERA5vars['t2m']['fvar'], '2017', ERA5vars['t2m']['fvar']+'*.nc'))[0]
def ERA5_to_JRA55(data_in):
    return ERA5_to_JRA55do_regridder(ERA5file, JRA55file)(data_in)

In [ ]:
JRA55file = os.path.join(JRA55path, 'tas_Amon_reanalysis_JRA-55_195801-201912.nc')
JRA55dofile = '/g/data/qv56/replicas/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-4-0/seaIce/3hrPt/siconca/gr/v20190429/siconca_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-4-0_gr_201801010000-201812312100.nc'
def JRA55_to_JRA55do(data_in):
    return JRA55_to_JRA55do_regridder(JRA55file, JRA55dofile)(data_in)

# Define map plotter and views

In [ ]:
# based on https://cosima-recipes.readthedocs.io/en/latest/tutorials/Making_Maps_with_Cartopy.html#gallery-tutorials-making-maps-with-cartopy-ipynb

def plotSIC(var, legend=True, projection=ccrs.NorthPolarStereo(), extent=[-180, 180, 48, 90],
           levels=np.arange(0,1.01,.1), cmp=plt.get_cmap('nipy_spectral'),
           cmap_gamma=2., # NB: non-1 gamma only works for fields and ticklevels in range [0, 1]
           cbar_ticklevels=[0,.3,.4,.5,.6,.7,.8,.85,.9,.95,1],  # for cmap_gamma = 2.
#            cmap_gamma = 4., cbar_ticklevels = [0,.5,.6,.7,.8,.85,.9,.95,.98,1],  # for cmap_gamma = 4.
           cbar_sigfigs=2, cbar_fontsize=12, cbar_label='Sea ice concentration', extend='neither', colorbar=True,
           lon_grid=range(-360, 361, 30), lat_grid=range(-80, 81, 10), draw_gridline_labels=True,
           x='longitude', y='latitude', xtransform=ccrs.PlateCarree(),
           contourvars=[],  # list of (var, label, kwargs) for overlaid contours
           feature=cft.NaturalEarthFeature('physical', 'land', '50m',
                                   edgecolor='black', facecolor=[.6, .6, .6], 
                                   linewidth=0.25),
           featureoverplot=False):
    var = var**cmap_gamma
    plt.figure(figsize=(12,7))
    ax = plt.axes(projection=projection)
#     ax.set_facecolor(cmp(0))
    ax.set_facecolor([.4, .4, .4])
#     ax.coastlines(resolution='50m', linewidth=0.5)
    if not featureoverplot:
        ax.add_feature(feature)
#     ax.set_global()
    gl = ax.gridlines(draw_labels=draw_gridline_labels,
                      linewidth=0.5, color=[0.8, 0.8, 0.8], alpha=0.5)
    gl.ylabels_top = None

#     gl.ylabels_bottom = None # only needed for Weddell and Amundsen-Bellingshausen closeups
#     gl.ylabels_left = None # only needed for Weddell and Amundsen-Bellingshausen closeups
#     gl.ylabels_right = None # only needed for Weddell and Amundsen-Bellingshausen closeups

#     gl.ylabels_right = None
    gl.xlabels_top = None
    gl.xlabels_bottom = None
    gl.xlabels_left = None
    gl.xlabels_right = None
    gl.xlocator = mticker.FixedLocator(lon_grid)
    gl.ylocator = mticker.FixedLocator(lat_grid)

#     contourf is buggy near poles, so use pcolormesh instead
#    p1 = var.plot.contourf(ax=ax,
#                           x='longitude', y='latitude',
#                           levels=levels, #vmin=0, vmax=1,
#                           cmap=cmp,
#                           transform=ccrs.PlateCarree(),
#                           add_colorbar=False)

#     p1 = exptdata.joinseams(var).plot.pcolormesh(ax=ax,
    p1 = var.plot.pcolormesh(ax=ax,
                           x=x, y=y,
                           vmin=min(cbar_ticklevels), vmax=max(cbar_ticklevels),
                           cmap=cmp,
                           transform=xtransform,
                           add_colorbar=False)

    ax.set_extent(extent, ccrs.PlateCarree())
#     p1.cmap.set_over(color=cmp(255), alpha=None)
#     p1.cmap.set_under(color=cmp(0), alpha=None)
    
    for cv in contourvars:
        cline = cv[0].plot.contour(#, #ax=ax,
            x='longitude', y='latitude',
            transform=ccrs.PlateCarree(),
            add_colorbar=False,
            **cv[2])
        if cv[1] is not None:
            cline.collections[0].set_label(cv[1]+' '+str(int(cv[2]['levels'][0]*100))+'%')

#         SeasonSI.sel(season = season).plot.contour(x = 'xt_ocean', y = 'yt_ocean', ax = axes[i,j],
#                                                    levels = [contIce], colors = 'white', linewidths = 1,
#                                                    transform = ccrs.PlateCarree())

    if featureoverplot:
        ax.add_feature(feature)

    if legend:
        plt.legend(prop={'size':8},loc='center')

    if colorbar:
        ax_cb = plt.axes([0.3, 0.08, 0.422, 0.015])
        cbar = plt.colorbar(p1, cax=ax_cb, orientation='horizontal', extend=extend)
        cbar.set_label(cbar_label, size=cbar_fontsize)
        cbar_ticks = [ f**cmap_gamma for f in cbar_ticklevels ]  
        cbar_ticklabels = [str(round(f, cbar_sigfigs)) for f in cbar_ticklevels]
        cbar.set_ticks(cbar_ticks)
        cbar.set_ticklabels(cbar_ticklabels)
        cbar_labels = plt.getp(cbar.ax.axes,'xticklabels')
        plt.setp(cbar_labels, fontsize=cbar_fontsize)
    
    return ax

In [ ]:
views = {
#     'NH': {'projection': ccrs.NorthPolarStereo(), 'extent': [-180, 180, 48, 90]},
    'SH': {'projection': ccrs.SouthPolarStereo(), 'extent': [-180, 180, -90, -54]},

# for some reason Stereographic doesn't work with JRA55-do data
#     'Amundsen-Bellingshausen': {'projection': ccrs.Stereographic(central_longitude=-90, central_latitude=-70), 'extent': [-115, -60, -77, -63], 'lon_grid': [], 'lat_grid': [], 'draw_gridline_labels': False},

# ... but SouthPolarStereo works with JRA55-do data and looks nearly identical
#     'Amundsen-BellingshausenSP': {'projection': ccrs.SouthPolarStereo(central_longitude=-90), 'extent': [-115, -60, -77, -63], 'lon_grid': [], 'lat_grid': [], 'draw_gridline_labels': False},

#     'Amundsen-Weddell': {'projection': ccrs.Stereographic(central_longitude=-70, central_latitude=-70), 'extent': [-115, -25, -82, -60], 'draw_gridline_labels': False},
#     'Weddell': {'projection': ccrs.Stereographic(central_longitude=-60, central_latitude=-70), 'extent': [-62, -20, -76.5, -60], 'draw_gridline_labels': False},
#     'Weddell-closeup': {'projection': ccrs.Stereographic(central_longitude=-60, central_latitude=-70), 'extent': [-60, -41, -75, -70], 'lon_grid': [], 'lat_grid': [], 'draw_gridline_labels': False}
}

In [ ]:
extlevel = [0.15, 0.15+1e-15]  # SIC contour level defining extent 

# Plot maps

## Ice monthly climatologies

### Model SIC

In [ ]:
# plot monthly SIC climatologies from each cycle of each model
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for enum, ekey in enumerate(ensemble): #enumerate(ice_data.keys()): #['1deg']: #['025deg']: #['01deg']: #
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['aice_m_mm']:
                for m in range(1,13):
                    title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'mean'])
                    fname = figname('_'.join([viewname, var, 
                                              ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                              yearrange, 'mean', 'month', str(m).zfill(2)]))
                    if os.path.exists(fname):
                        print('   -- skipping', fname)
                    else:
                        print('doing', fname)
                        v = ide[var].sel(month=m)
                        obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
                        try:
                            idev1 = ice_data[ekey+'v1'][cycle-1]
                            plotSIC(v, contourvars=[
                                        (idev1[var].sel(month=m), idev1['desc'], {'levels': extlevel, 'colors': 'C'+str(enum+3)}), # 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}), # old v1 config
                                        (v, ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
        # sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                        (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                                        (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                                ],
                                        **view).set_title(title);
                        except KeyError:                        
                            plotSIC(v, contourvars=[
                                    # (idev1[var].sel(month=m), idev1['desc'], {'levels': extlevel, 'colors': 'C'+str(enum+3)}), # 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}), # old v1 config
                                    (v, ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
    # sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                    (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                                    (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                            ],
                                    **view).set_title(title);
                        savefigure(fname)
                        plt.close()
#                         break
            break
#         break
print('done')

In [ ]:
# plot monthly SIC climatologies from each cycle of each model, with SIC set to 0 for hi < hithreshold
hithreshold = .2  # thickness threshold (m)
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for enum, ekey in enumerate(ensemble): #enumerate(ice_data.keys()): #['1deg']: #['025deg']: #['01deg']: #
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['aice_m']:
                vthresh_m = ide[var].where(ide['hi_m'] > hithreshold, 0.0)
                vthresh_m_mm = vthresh_m.groupby('time.month').mean('time', skipna=True)
                for m in range(1,13):
                    title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'mean'])
                    fname = figname('_'.join([viewname, var+'_mm', 'hithreshold', str(hithreshold), 
                                              ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                              yearrange, 'mean', 'month', str(m).zfill(2)]))
                    if os.path.exists(fname):
                        print('   -- skipping', fname)
                    else:
                        v = vthresh_m_mm.sel(month=m)
#                         idev1 = ice_data[ekey+'v1'][cycle-1]
                        obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
                        
                        print('doing', fname)
                        plotSIC(v, contourvars=[
#                                 (idev1[var].sel(month=m), idev1['desc'], {'levels': extlevel, 'colors': 'C'+str(enum+3)}), # 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}), # old v1 config
                                (v, ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
# sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                                (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                        ], cbar_label='Sea ice concentration for thickness > '+str(hithreshold)+'m',
                                **view).set_title(title);
                        savefigure(fname)
                        plt.close()
#                         break
#             break
#         break
print('done')

In [ ]:
# plot monthly SIC climatology for perturbations
controlkey = controlkeys[0]  # KLUDGE! do this better
control = ice_data[controlkey][0]
for var in ['aice_m_mm']:
    for m in range(1,13):
        vcontrol = control[var].sel(month=m)
        for ekey in ensemble: #ice_data.keys():
            for viewname, view in views.items():
                if viewname in ['NH']:
                    hem = 'NH'
                else:
                    hem = 'SH'
                ide = ice_data[ekey][0]
                title = ' '.join([ide['desc'], yearrange, calendar.month_name[m], 'mean'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'], yearrange, 'mean', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    v = ide[var].sel(month=m)
                    obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)

                    print('doing', fname)
                    contourvars=[
                            (vcontrol, control['desc'], {'levels': extlevel, 'colors': 'green'}),
                            (v, ide['desc'], {'levels': extlevel, 'colors': 'y'}),
# sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                            (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'silver'}),
                            (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'silver'}),
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(v, contourvars=contourvars, **view).set_title(title);
                    savefigure(fname)
                    plt.close()
#             break
#         break
print('done')

In [ ]:
# plot monthly SIC climatology for perturbations, with SIC set to 0 for hi < hithreshold
hithreshold = .2  # thickness threshold (m)
controlkey = controlkeys[0]  # KLUDGE! do this better
control = ice_data[controlkey][0]
for var in ['aice_m']:
    vcontrolthresh_m = control[var].where(ide['hi_m'] > hithreshold, 0.0)
    vcontrolthresh_m_mm = vcontrolthresh_m.groupby('time.month').mean('time', skipna=True)
    for ekey in ensemble: #ice_data.keys():
        ide = ice_data[ekey][0]
        vthresh_m = ide[var].where(ide['hi_m'] > hithreshold, 0.0)
        vthresh_m_mm = vthresh_m.groupby('time.month').mean('time', skipna=True)
        for m in range(1,13):
            vcontrol = vcontrolthresh_m_mm.sel(month=m)
            for viewname, view in views.items():
                if viewname in ['NH']:
                    hem = 'NH'
                else:
                    hem = 'SH'
                title = ' '.join([ide['desc'], yearrange, calendar.month_name[m], 'mean'])
                fname = figname('_'.join([viewname, var+'_mm', 'hithreshold', str(hithreshold),
                                          ide['expt'], yearrange, 'mean', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    v = vthresh_m_mm.sel(month=m)
                    obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)

                    print('doing', fname)
                    contourvars=[
                            (vcontrol, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (v, ide['desc'], {'levels': extlevel, 'colors': 'y'}),
# sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                            (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'silver'}),
                            (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'silver'}),
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(v, contourvars=contourvars, 
                            cbar_label='Sea ice concentration for thickness > '+str(hithreshold)+'m',
                            **view).set_title(title);
                    savefigure(fname)
                    plt.close()
#             break
#         break
print('done')

### Model SIC differences

In [ ]:
# plot monthly SIC climatology difference between cycle n and cycle 1 for each model
for viewname, view in views.items():
    print(viewname)
    for ekey in ice_data.keys(): #['1deg']: #['01deg']: #['025deg']:
        for var in ['aice_m_mm']:
            for m in range(1,13):
                vref = ice_data[ekey][0][var].sel(month=m)
                for cycle, ide in enumerate(ice_data[ekey][1:], start=2):
                    title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'mean'])
#                     title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'mean', 'minus cycle 1'])
                    fname = figname('_'.join([viewname, var, 
                                              ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-cycle1', # FRAGLE! assumes filename cycle number = cycm2+1
                                              yearrange, 'mean', 'month', str(m).zfill(2)]))
                    if os.path.exists(fname):
                       print('   -- skipping', fname)
                    else:
                        v = ide[var].sel(month=m) - vref
                        print('doing', fname)
                        plotSIC(v, cmp=plt.get_cmap('seismic'), cbar_label='Sea ice concentration minus cycle 1',
                                cmap_gamma=1., cbar_ticklevels=np.arange(-.15,.1501,.05), cbar_sigfigs=2, **view).set_title(title);
                        savefigure(fname)
                        plt.close()
print('done')

In [ ]:
# plot monthly SIC climatology difference between control and perturbation
controlkey = controlkeys[0]  # KLUDGE! do this better
control = ice_data[controlkey][0]
for var in ['aice_m_mm']:
    for m in range(1,13):
        vcontrol = control[var].sel(month=m)
        for ekey in ensemble[1:]: #ice_data.keys():
            if ekey != controlkey:
                for viewname, view in views.items():
                    if viewname in ['NH']:
                        hem = 'NH'
                    else:
                        hem = 'SH'
                    ide = ice_data[ekey][0]
                    title = ' '.join([ide['desc'], yearrange, calendar.month_name[m], 'mean'])
                    fname = figname('_'.join([viewname, var, 
                                              ide['expt'], '-control', yearrange, 'mean', 'month', str(m).zfill(2)]))
                    if os.path.exists(fname):
                        print('   -- skipping', fname)
                    else:
                        v = ide[var].sel(month=m)
                        vdiff = v - vcontrol
                        obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)

                        print('doing', fname)
                        plotSIC(vdiff, cmp=plt.get_cmap('seismic'), cbar_label='Sea ice concentration minus control',
                                cmap_gamma=1., cbar_ticklevels=np.arange(-.15,.1501,.05), cbar_sigfigs=2, 
                                contourvars=[
                                (vcontrol, control['desc'], {'levels': extlevel, 'colors': 'lime'}),
                                (v, ide['desc'], {'levels': extlevel, 'colors': 'y'}),
# sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'silver'}),
                                (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'silver'}),
                                ], **view).set_title(title);
                        savefigure(fname)
                        plt.close()
#             break
#         break
print('done')

### Observed SIC

In [ ]:
# plot monthly climatologies from SIC obs
for viewname, view in views.items():
    print(viewname)
    for m in range(1,13):
        title = ' '.join(['Passive microwave', yearrange, calendar.month_name[m], 'mean'])
        fname = figname('_'.join([viewname, 'aice', 'obs', yearrange, 'mean', 'month', str(m).zfill(2)]))
        if False: #os.path.exists(fname):
            print('   -- skipping', fname)
        else:
            if viewname in ['NH']:
                hem = 'NH'
            else:
                hem = 'SH'
            v = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
            print('doing', fname)
            plotSIC(v, #x='xt_ocean', y='ygrid', xtransform=ccrs.SouthPolarStereo(),
                    contourvars=[
                    (v.isel(xt_ocean=slice(0, 158)), 'passive microwave', {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (v.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'white'})], #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    **view).set_title(title);
            savefigure(fname)
            plt.close()
#             break
print('done')

### Model SIC bias (model minus observed)

In [ ]:
# plot monthly SIC climatology bias for each model
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for enum, ekey in enumerate(ensemble): #['1deg']: #['025deg']: #['01deg']: #
        obsv = eval('_'.join(['obs', hem, 'mm']))
        vrefs = obsregridders[hem][ice_data[ekey][0]['res']](obsv)
        for var in ['aice_m_mm']:
            for m in range(1,13):
                vref = vrefs.sel(month=m)
                for cycle, ide in enumerate(ice_data[ekey], start=1):
                    title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'bias'])
                    fname = figname('_'.join([viewname, var, 
                                              ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-obs', # FRAGLE! assumes filename cycle number = cycm2+1
                                              yearrange, 'mean', 'month', str(m).zfill(2)]))
                    if os.path.exists(fname):
                        print('   -- skipping', fname)
                    else:
                        vcontrol = control[var].sel(month=m)
                        v = ide[var].sel(month=m)
                        bias = v - vref
                        print('doing', fname)
                        contourvars=[
                                (vcontrol, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                                (v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                                (vref, 'passive microwave',  {'levels': extlevel, 'colors': 'k'})
                                ]
                        if ekey == controlkey:
                            contourvars = contourvars[0:1]+contourvars[2:]
                        plotSIC(bias, cmp=plt.get_cmap('seismic'), cbar_label='Sea ice concentration minus passive microwave obs',
                                cmap_gamma=1., cbar_ticklevels=np.arange(-1, 1.01, 0.25), cbar_sigfigs=2,
                                contourvars=contourvars, **view).set_title(title);

                        savefigure(fname)
                        plt.close()
#                     break
#                 break
#             break
#         break
print('done')

In [ ]:
# plot monthly SIC climatology bias for each model, with model SIC set to 0 for hi < hithreshold
hithreshold = .2  # thickness threshold (m)
controlkey = controlkeys[0]  # KLUDGE! do this better
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for enum, ekey in enumerate(ensemble): #['1deg']: #['025deg']: #['01deg']: #
        ide = ice_data[ekey][0]
        obsv = eval('_'.join(['obs', hem, 'mm']))
        vrefs = obsregridders[hem][ice_data[ekey][0]['res']](obsv)
        for var in ['aice_m']:
            vthresh_m = ide[var].where(ide['hi_m'] > hithreshold, 0.0)
            vthresh_m_mm = vthresh_m.groupby('time.month').mean('time', skipna=True)
            vcontrolthresh_m = control[var].where(ide['hi_m'] > hithreshold, 0.0)
            vcontrolthresh_m_mm = vcontrolthresh_m.groupby('time.month').mean('time', skipna=True)
            for m in range(1,13):
                vref = vrefs.sel(month=m)
#                 for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], # 'cycle', str(cycle), 
                                  yearrange, calendar.month_name[m], 'bias'])
                fname = figname('_'.join([viewname, var+'_mm', 'hithreshold', str(hithreshold),
                                          ide['expt'].split('_cycle')[0]+'-obs', #'cycle'+str(cycle)+'-obs', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'mean', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                   print('   -- skipping', fname)
                else:
                    vcontrol = vcontrolthresh_m_mm.sel(month=m)
                    v = vthresh_m_mm.sel(month=m)
                    bias = v - vref
                    print('doing', fname)
                    contourvars=[
                            (vcontrol, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (vref, 'passive microwave',  {'levels': extlevel, 'colors': 'k'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(bias, cmp=plt.get_cmap('seismic'), cbar_label='SIC (for thickness > '+str(hithreshold)+'m) minus passive microwave obs',
                            cmap_gamma=1., cbar_ticklevels=np.arange(-1, 1.01, 0.25), cbar_sigfigs=2,
                            contourvars=contourvars, **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                 break
#                 break
#             break
#         break
print('done')

In [ ]:
# plot monthly SIC climatology bias for each model relative to siconca (which was assimilated into JRA55) and siconc
controlkey = controlkeys[0]  # KLUDGE! do this betvter
# # controlkey = '025deg'
control = ice_data[controlkey][0]
for JRA55dovar in ['siconca', 'siconc']:
    for viewname, view in views.items():
        print(viewname)
        if viewname in ['NH']:
            hem = 'NH'
        else:
            hem = 'SH'
        for enum, ekey in enumerate(ensemble): #['1deg']: #['025deg']: #['01deg']: #
            obss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
            vrefs = JRA55do_data[JRA55dovar+'_mm_regridded_to_mom'][ice_data[ekey][0]['res']]
            for var in ['aice_m_mm']:
                for m in range(1,13):
                    vref = vrefs.sel(month=m)
                    for cycle, ide in enumerate(ice_data[ekey], start=1):
                        title = ' '.join([ide['desc'], #'cycle', str(cycle), 
                                          yearrange, calendar.month_name[m], 'minus JRA55', JRA55dovar])
                        fname = figname('_'.join([viewname, var, 
                                                  ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-'+JRA55dovar, # FRAGLE! assumes filename cycle number = cycm2+1
                                                  yearrange, 'mean', 'month', str(m).zfill(2)]))
                        if os.path.exists(fname):
                           print('   -- skipping', fname)
                        else:
                            vcontrol = control[var].sel(month=m)
                            v = ide[var].sel(month=m)
                            bias = v - vref
                            print('doing', fname)
                            contourvars=[
                                    (vcontrol, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                                    (v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                                    (vref, JRA55dovar,  {'levels': extlevel, 'colors': 'dimgray'}),
                                    (obss.sel(month=m), 'passive microwave',  {'levels': extlevel, 'colors': 'k'})
                                    ]
                            if ekey == controlkey:
                                contourvars = contourvars[0:1]+contourvars[2:]
                            plotSIC(bias, cmp=plt.get_cmap('seismic'), cbar_label='Sea ice concentration minus JRA55 '+JRA55dovar,
                                    cmap_gamma=1., cbar_ticklevels=np.arange(-1, 1.01, 0.25), cbar_sigfigs=2,
                                    contourvars=contourvars, **view).set_title(title);

                            savefigure(fname)
                            plt.close()
#                         break
#                     break
#                 break
#             break
print('done')

### Model SIC standard deviation

In [ ]:
# plot standard deviation of monthly SIC from each cycle of each model
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for enum, ekey in enumerate(ensemble): #enumerate(ice_data.keys()): #['1deg']: #['025deg']: #['01deg']: #
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['aice_m']:
                varstd = ide[var].groupby('time.month').std('time', skipna=True)
                for m in range(1,13):
                    title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'mean standard deviation'])
                    fname = figname('_'.join([viewname, var, 
                                              ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                              yearrange, 'std', 'month', str(m).zfill(2)]))
                    if os.path.exists(fname):
                        print('   -- skipping', fname)
                    else:
                        v = varstd.sel(month=m)
                        obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
                        
                        print('doing', fname)
                        plotSIC(v, cbar_label='Standard deviation of monthly mean sea ice concentration',
                                contourvars=[
                                (ide[var+'_mm'].sel(month=m), ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
# sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                                (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                        ],
                                **view).set_title(title);
                        savefigure(fname)
                        plt.close()
#                         break
#             break
#         break
print('done')

### Observed SIC standard deviation

In [ ]:
# plot standard deviation of monthly mean of SIC obs
for viewname, view in views.items():
    print(viewname)
    for m in range(1,13):
        title = ' '.join(['Passive microwave', yearrange, calendar.month_name[m], 'mean standard deviation'])
        fname = figname('_'.join([viewname, 'aice', 'obs', yearrange, 'std', 'month', str(m).zfill(2)]))
        if os.path.exists(fname):
            print('   -- skipping', fname)
        else:
            if viewname in ['NH']:
                hem = 'NH'
            else:
                hem = 'SH'
            vstd = eval('_'.join(['obs', hem, 'mstd'])).sel(month=m)
            v = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
            print('doing', fname)
            plotSIC(vstd, cbar_label='Standard deviation of monthly mean sea ice concentration',
                    contourvars=[
                    (v.isel(xt_ocean=slice(0, 158)), 'passive microwave', {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (v.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'white'})], #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    **view).set_title(title);
            savefigure(fname)
            plt.close()
#             break
print('done')

### SIC tendency

In [ ]:
# plot monthly SIC tendency climatology from each cycle of each model
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for enum, ekey in enumerate(ensemble): #enumerate(ice_data.keys()): #['1deg']: #['025deg']: #['01deg']: #
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['aice_m_mm']:
                for m in range(1,13):
                    prevm = (m-2)%12 + 1
                    title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'mean tendency'])
                    fname = figname('_'.join([viewname, var, 
                                              ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                              yearrange, 'mean', 'tendency', 'month', str(m).zfill(2)]))
                    if os.path.exists(fname):
                        print('   -- skipping', fname)
                    else:
                        v = ide[var].sel(month=m) - ide[var].sel(month=prevm)
#                         idev1 = ice_data[ekey+'v1'][0]
                        obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
                        
                        print('doing', fname)
                        plotSIC(v, cmp=plt.get_cmap('seismic'), cbar_label='SIC change from previous month',
                                cmap_gamma=1., cbar_ticklevels=np.arange(-1, 1.01, 0.25), cbar_sigfigs=2,
                                contourvars=[
#                                 (idev1[var].sel(month=m), idev1['desc'], {'levels': extlevel, 'colors': 'C'+str(enum+3)}), # 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}), # old v1 config
                                (ide[var].sel(month=m), ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
# sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'black'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                                (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'black'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                        ],
                                **view).set_title(title);
                        savefigure(fname)
                        plt.close()
#                         break
#             break # cycle 1 only
#         break
print('done')

In [ ]:
# plot monthly tendency climatologies from SIC obs
for viewname, view in views.items():
    print(viewname)
    for m in range(1,13):
        prevm = (m-2)%12 + 1
        title = ' '.join(['Passive microwave', yearrange, calendar.month_name[m], 'mean tendency'])
        fname = figname('_'.join([viewname, 'aice', 'obs', yearrange, 'mean', 'tendency', 'month', str(m).zfill(2)]))
        if os.path.exists(fname):
            print('   -- skipping', fname)
        else:
            if viewname in ['NH']:
                hem = 'NH'
            else:
                hem = 'SH'
            obs = eval('_'.join(['obs', hem, 'mm']))
            v = obs.sel(month=m)
            tendency = obs.sel(month=m) - obs.sel(month=prevm)
            print('doing', fname)
            plotSIC(tendency, cmp=plt.get_cmap('seismic'), cbar_label='SIC change from previous month',
                    cmap_gamma=1., cbar_ticklevels=np.arange(-1, 1.01, 0.25), cbar_sigfigs=2,
                    contourvars=[
                    (v.isel(xt_ocean=slice(0, 158)), 'passive microwave', {'levels': extlevel, 'colors': 'black'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (v.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'black'})], #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    **view).set_title(title);
            savefigure(fname)
            plt.close()
#             break
print('done')

In [ ]:
# plot monthly SIC tendency climatology bias for each cycle of each model
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for enum, ekey in enumerate(ensemble): #enumerate(ice_data.keys()): #['1deg']: #['025deg']: #['01deg']: #
        obs = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['aice_m_mm']:
                for m in range(1,13):
                    prevm = (m-2)%12 + 1
                    title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'tendency bias'])
                    fname = figname('_'.join([viewname, var, 
                                              ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                              yearrange, 'tendency', 'bias', 'month', str(m).zfill(2)]))
                    if os.path.exists(fname):
                        print('   -- skipping', fname)
                    else:
                        v = ide[var].sel(month=m) - ide[var].sel(month=prevm)
                        obsv = obs.sel(month=m)
                        obstendency = obs.sel(month=m) - obs.sel(month=prevm)
                        bias = v - obstendency
                        print('doing', fname)
                        plotSIC(bias, cmp=plt.get_cmap('seismic'), cbar_label='Bias (model-obs) in SIC change from previous month',
                                cmap_gamma=1., cbar_ticklevels=np.arange(-1, 1.01, 0.25), cbar_sigfigs=2,
                                contourvars=[
                                (ide[var].sel(month=m), ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
# sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'black'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                                (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'black'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                        ],
                                **view).set_title(title);
                        savefigure(fname)
                        plt.close()
#                         break
#             break # cycle 1 only
#         break
print('done')

### SIC tendency components

- `daidtd` is %/day SIC change due to dynamics
- `daidtt` is %/day SIC change due to thermodynamics

In [ ]:
# plot monthly SIC tendency components from each cycle of each model
cause = {'daidtd_m_mm':'dynamics', 'daidtt_m_mm': 'thermo'}
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for enum, ekey in enumerate(ensemble): #enumerate(ice_data.keys()): #['1deg']: #['025deg']: #['01deg']: #
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['daidtd_m_mm', 'daidtt_m_mm']:
                for m in range(1,13):
                    title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], cause[var], 'tendency'])
                    fname = figname('_'.join([viewname, var, 
                                              ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                              yearrange, cause[var], 'tendency', 'month', str(m).zfill(2)]))
                    if os.path.exists(fname):
                        print('   -- skipping', fname)
                    else:
                        v = ide[var].sel(month=m)*calendar.monthrange(2021, m)[1]/100  # convert from %/day to SIC change over month (ignoring leap years)
                        obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
                        
                        print('doing', fname)
                        plotSIC(v, cmp=plt.get_cmap('seismic'), cbar_label='SIC change over month due to '+cause[var],
                                cmap_gamma=1., cbar_ticklevels=np.arange(-1, 1.01, 0.25), cbar_sigfigs=2,
                                contourvars=[
                                (ide['aice_m_mm'].sel(month=m), ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
# sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'black'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                                (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'black'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                        ],
                                **view).set_title(title);
                        savefigure(fname)
                        plt.close()
#                         break
#             break # cycle 1 only
#         break
print('done')

### Ice thickness

In [ ]:
# plot monthly ice thickness climatologies from each cycle of each model
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    if viewname == 'NH':
        cbar_ticklevels=np.arange(0, 5.01, 1.0)
    elif viewname == 'Amundsen-Bellingshausen':
        cbar_ticklevels=np.arange(0, 3.501, 0.5)
    else:
        cbar_ticklevels=np.arange(0, 2.501, 0.5)
    for enum, ekey in enumerate(ensemble):
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['hi_m_mm']:
                for m in range(1,13):
                    title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'mean'])
                    fname = figname('_'.join([viewname, var, 
                                              ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                              yearrange, 'mean', 'month', str(m).zfill(2)]))
                    if os.path.exists(fname):
                        print('   -- skipping', fname)
                    else:
                        v = ide[var].sel(month=m)
                        vsic = ide['aice_m_mm'].sel(month=m)
                        obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)

                        print('doing', fname)
#                         plotSIC(v, cbar_label='Sea ice thickness (m)',
#                                 cmap_gamma=1., cbar_ticklevels=cbar_ticklevels, cbar_sigfigs=1, extend='max', **view).set_title(title);
                        plotSIC(v, cbar_label='Sea ice thickness (m)',
                                cmap_gamma=1., cbar_ticklevels=cbar_ticklevels, cbar_sigfigs=1, extend='max',
                                contourvars=[
#                                 (idev1[var].sel(month=m), idev1['desc'], {'levels': extlevel, 'colors': 'C'+str(enum+3)}), # 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}), # old v1 config
                                (vsic, ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
# sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                                (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                        ],
                                **view).set_title(title);

                        savefigure(fname)
                        plt.close()
#                         break
#             break
print('done')

### Snow thickness

In [ ]:
# plot monthly snow thickness climatologies from each cycle of each model
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for enum, ekey in enumerate(ensemble):
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['hs_m_mm']:
                for m in range(1,13):
                    title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'mean'])
                    fname = figname('_'.join([viewname, var, 
                                              ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                              yearrange, 'mean', 'month', str(m).zfill(2)]))
                    if os.path.exists(fname):
                        print('   -- skipping', fname)
                    else:
                        v = ide[var].sel(month=m)
                        vsic = ide['aice_m_mm'].sel(month=m)
                        obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)

                        print('doing', fname)
                        plotSIC(v, cbar_label='Snow thickness (m)',
                                cmap_gamma=1., cbar_ticklevels=np.arange(0, 0.701, 0.1), cbar_sigfigs=1, extend='max',
#                                 cmap_gamma=1., cbar_ticklevels=np.arange(0, 0.601, 0.1), cbar_sigfigs=1, extend='max',
                                contourvars=[
#                                 (idev1[var].sel(month=m), idev1['desc'], {'levels': extlevel, 'colors': 'C'+str(enum+3)}), # 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}), # old v1 config
                                (vsic, ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
# sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                                (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                        ],
                                **view).set_title(title);

                        savefigure(fname)
                        plt.close()
#                         break
#             break
print('done')

## GIOMAS

### GIOMAS SIC

In [ ]:
# plot monthly SIC climatologies from GIOMAS
vname = 'aice_m'
vdata = GIOMAS_mm[vname]
vdata_regridded = regrid_giomas_to_025(vdata)
for viewname, view in views.items():
    print(viewname)
    for m in range(1,13):
        title = ' '.join(['GIOMAS', yearrange, calendar.month_name[m], 'mean'])
        fname = figname('_'.join([viewname, vname, 'GIOMAS', yearrange, 'mean', 'month', str(m).zfill(2)]))
        if os.path.exists(fname):
            print('   -- skipping', fname)
        else:
            if viewname in ['NH']:
                hem = 'NH'
            else:
                hem = 'SH'
            v = vdata_regridded.sel(month=m)
            obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
            print('doing', fname)
            plotSIC(v,
                    contourvars=[
                    (v, 'GIOMAS', {'levels': extlevel, 'colors': 'g'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave', {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (obsv.isel(xt_ocean=slice(158, None)), None, {'levels': extlevel, 'colors': 'white'})], #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    **view).set_title(title);
            savefigure(fname)
            plt.close()
#             break
#     break
print('done')

### GIOMAS SIT

In [ ]:
# plot monthly SIT climatologies from GIOMAS
vname = 'hi_m'
vdata = GIOMAS_mm[vname]
vdata_regridded = regrid_giomas_to_025(vdata)
sic_regridded = regrid_giomas_to_025(GIOMAS_mm['aice_m'])
for viewname, view in views.items():
    print(viewname)
    for m in range(1,13):
        title = ' '.join(['GIOMAS', yearrange, calendar.month_name[m], 'mean'])
        fname = figname('_'.join([viewname, vname, 'GIOMAS', yearrange, 'mean', 'month', str(m).zfill(2)]))
        if os.path.exists(fname):
            print('   -- skipping', fname)
        else:
            if viewname in ['NH']:
                hem = 'NH'
            else:
                hem = 'SH'
            if viewname == 'NH':
                cbar_ticklevels=np.arange(0, 5.01, 1.0)
            elif viewname == 'Amundsen-Bellingshausen':
                cbar_ticklevels=np.arange(0, 3.501, 0.5)
            else:
                cbar_ticklevels=np.arange(0, 2.501, 0.5)
            v = vdata_regridded.sel(month=m)
            sic = sic_regridded.sel(month=m)
            obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
            print('doing', fname)
            plotSIC(v, cbar_label='Sea ice thickness (m)',
                                cmap_gamma=1., cbar_ticklevels=cbar_ticklevels, cbar_sigfigs=1, extend='max',
                    contourvars=[
                    (sic, 'GIOMAS', {'levels': extlevel, 'colors': 'g'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave', {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (obsv.isel(xt_ocean=slice(158, None)), None, {'levels': extlevel, 'colors': 'white'})], #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    **view).set_title(title);
            savefigure(fname)
            plt.close()
#             break
#     break
print('done')

### GIOMAS snow thickness

In [ ]:
# plot monthly snow thickness climatologies from GIOMAS
vname = 'hs_m'
vdata = GIOMAS_mm[vname]
vdata_regridded = regrid_giomas_to_025(vdata)
sic_regridded = regrid_giomas_to_025(GIOMAS_mm['aice_m'])
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for m in range(1,13):
        title = ' '.join(['GIOMAS', yearrange, calendar.month_name[m], 'mean'])
        fname = figname('_'.join([viewname, vname, 'GIOMAS', yearrange, 'mean', 'month', str(m).zfill(2)]))
        if os.path.exists(fname):
            print('   -- skipping', fname)
        else:
            v = vdata_regridded.sel(month=m)
            sic = sic_regridded.sel(month=m)
            obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
            print('doing', fname)
            plotSIC(v, cbar_label='Snow thickness (m)',
                                cmap_gamma=1., cbar_ticklevels=np.arange(0, 0.701, 0.1), cbar_sigfigs=1, extend='max',
                    contourvars=[
                    (sic, 'GIOMAS', {'levels': extlevel, 'colors': 'g'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave', {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (obsv.isel(xt_ocean=slice(158, None)), None, {'levels': extlevel, 'colors': 'white'})], #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    **view).set_title(title);
            savefigure(fname)
            plt.close()
#             break
#     break
print('done')

### GIOMAS SIC bias

In [ ]:
# plot bias in GIOMAS SIC monthly climatologies relative to NSIDC (GIOMAS - NSIDC)
vname = 'aice_m'
vdata = GIOMAS_mm[vname]
vdata_regridded = regrid_giomas_to_025(vdata)
for viewname, view in views.items():
    print(viewname)
    for m in range(1,13):
        title = ' '.join(['GIOMAS', vname, yearrange, calendar.month_name[m], 'bias'])
        fname = figname('_'.join([viewname, vname, 'GIOMAS-obs', yearrange, 'mean', 'month', str(m).zfill(2)]))
        if os.path.exists(fname):
            print('   -- skipping', fname)
        else:
            if viewname in ['NH']:
                hem = 'NH'
            else:
                hem = 'SH'
            v = vdata_regridded.sel(month=m)
            obsv = eval('regrid_'+hem+'obs_to_025('+'_'.join(['obs', hem, 'mm'])+')').sel(month=m)
            bias = v-obsv
            print('doing', fname)
            plotSIC(bias, cmp=plt.get_cmap('seismic'), cbar_label='GIOMAS SIC minus passive microwave obs',
                                cmap_gamma=1., cbar_ticklevels=np.arange(-1, 1.01, 0.25), cbar_sigfigs=2,
                    contourvars=[
                    (v, 'GIOMAS', {'levels': extlevel, 'colors': 'g'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave', {'levels': extlevel, 'colors': 'k'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (obsv.isel(xt_ocean=slice(158, None)), None, {'levels': extlevel, 'colors': 'k'})], #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    **view).set_title(title);
            savefigure(fname)
            plt.close()
#             break
print('done')

### Model-GIOMAS SIC bias

In [ ]:
# plot monthly SIC climatology bias for each model wrt GIOMAS
vname = 'aice_m'
var = vname+'_mm'
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.sel(month=m)
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'mean bias'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'mean', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].sel(month=m)
                    bias = v - vref
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'y'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'k'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(bias, cmp=plt.get_cmap('seismic'), cbar_label='Sea ice concentration minus GIOMAS',
                            cmap_gamma=1., cbar_ticklevels=np.arange(-1, 1.01, 0.25), cbar_sigfigs=2,
                            contourvars=contourvars, **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS SIT bias

In [ ]:
# plot monthly SIT climatology bias for each model wrt GIOMAS
vname = 'hi_m'
var = vname+'_mm'
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.sel(month=m)
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'mean bias'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'mean', 'month', str(m).zfill(2)]))
                if False: #os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].sel(month=m)
                    bias = v - vref
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'y'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'k'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(bias, cmp=plt.get_cmap('seismic'), cbar_label='Sea ice thickness minus GIOMAS (m)',
                            cmap_gamma=1., cbar_ticklevels=np.arange(-1, 1.01, 0.25), cbar_sigfigs=2,
                            contourvars=contourvars, extend='both', **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS snow bias

In [ ]:
# plot monthly snow thickness climatology bias for each model wrt GIOMAS

# TODO: check if CICE data is water-equivalent I've assumed it isn't - which is consistent with the CICE manual
# CICE snow density = 330 kg/m^3, not sure what GIOMAS density is
# correction was applied in get_giomas

vname = 'hs_m'
var = vname+'_mm'
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.sel(month=m)
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'mean bias'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'mean', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].sel(month=m)
                    bias = v - vref
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'y'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'k'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(bias, cmp=plt.get_cmap('seismic'), cbar_label='Snow thickness minus GIOMAS (m)',
                            cmap_gamma=1., cbar_ticklevels=np.arange(-0.3, 0.301, 0.1), cbar_sigfigs=2,
                            contourvars=contourvars, extend='both', **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS SIC MAD

In [ ]:
# plot monthly SIC MAD for each model wrt GIOMAS
vname = 'aice_m'
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.groupby('time.month')[m]
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'MAD'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'MAD', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].groupby('time.month')[m]
                    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
                    MAD = (np.abs(va - vrefa)).mean('time')
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'c'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'w'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(MAD, cmp=plt.get_cmap('inferno'), cbar_label='Sea ice concentration MAD vs. GIOMAS',
                            cmap_gamma=1., cbar_ticklevels=np.arange(0, 1.01, 0.25), cbar_sigfigs=2,
                            contourvars=contourvars, **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS SIC MAD diference from control

In [ ]:
# plot monthly SIC MAD for each model wrt GIOMAS minus control MAD
vname = 'aice_m'
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.groupby('time.month')[m]
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'MAD - control'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'MAD-control', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].groupby('time.month')[m]
                    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
                    MAD = (np.abs(va - vrefa)).mean('time')
                    vcontrol = control[vname].groupby('time.month')[m]
                    MADcontrol = (np.abs(vcontrol - vrefa)).mean('time')
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'c'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'k'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(MAD-MADcontrol, cmp=plt.get_cmap('seismic'), cbar_label='Change in SIC MAD vs. GIOMAS due to perturbation',
                            cmap_gamma=1., cbar_ticklevels=np.arange(-0.2, 0.201, 0.05), cbar_sigfigs=2,
                            contourvars=contourvars, **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS SIT MAD

In [ ]:
# plot monthly SIT MAD for each model wrt GIOMAS
vname = 'hi_m'
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.groupby('time.month')[m]
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'MAD'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'MAD', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].groupby('time.month')[m]
                    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
                    MAD = (np.abs(va - vrefa)).mean('time')
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'c'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'w'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(MAD, cmp=plt.get_cmap('inferno'), cbar_label='Sea ice thickness MAD vs. GIOMAS (m)',
                            cmap_gamma=1., cbar_ticklevels=np.arange(0, 2.01, 0.25), cbar_sigfigs=2,
                            contourvars=contourvars, extend='max', **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS SIT MAD difference from control

In [ ]:
# plot monthly SIT MAD for each model wrt GIOMAS minus control MAD
vname = 'hi_m'
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.groupby('time.month')[m]
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'MAD - control'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'MAD-control', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].groupby('time.month')[m]
                    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
                    MAD = (np.abs(va - vrefa)).mean('time')
                    vcontrol = control[vname].groupby('time.month')[m]
                    MADcontrol = (np.abs(vcontrol - vrefa)).mean('time')
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'c'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'k'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(MAD-MADcontrol, cmp=plt.get_cmap('seismic'), cbar_label='Change in SIT MAD vs. GIOMAS due to perturbation (m)',
                            cmap_gamma=1., cbar_ticklevels=np.arange(-0.5, 0.501, 0.1), cbar_sigfigs=1,
                            contourvars=contourvars, extend='both', **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS snow MAD

In [ ]:
# plot monthly snow MAD for each model wrt GIOMAS
vname = 'hs_m'
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.groupby('time.month')[m]
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'MAD'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'MAD', 'month', str(m).zfill(2)]))
                if False: #os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].groupby('time.month')[m]
                    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
                    MAD = (np.abs(va - vrefa)).mean('time')
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'c'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'w'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(MAD, cmp=plt.get_cmap('inferno'), cbar_label='Snow thickness MAD vs. GIOMAS (m)',
                            cmap_gamma=1., cbar_ticklevels=np.arange(0, 0.501, 0.1), cbar_sigfigs=1,
                            contourvars=contourvars, extend='max', **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS snow MAD difference from control

In [ ]:
# plot monthly snow MAD for each model wrt GIOMAS minus control MAD
vname = 'hs_m'
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.groupby('time.month')[m]
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'MAD - control'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'MAD-control', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].groupby('time.month')[m]
                    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
                    MAD = (np.abs(va - vrefa)).mean('time')
                    vcontrol = control[vname].groupby('time.month')[m]
                    MADcontrol = (np.abs(vcontrol - vrefa)).mean('time')
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'c'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'k'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(MAD-MADcontrol, cmp=plt.get_cmap('seismic'), cbar_label='Change in snow thickness MAD vs. GIOMAS due to perturb (m)',
                            cmap_gamma=1., cbar_ticklevels=np.arange(-0.02, 0.0201, 0.005), cbar_sigfigs=3,
                            contourvars=contourvars, extend='both', **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS SIC RMSE

In [ ]:
# plot monthly SIC RMSE for each model wrt GIOMAS
vname = 'aice_m'
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.groupby('time.month')[m]
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'RMSE'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'RMSE', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].groupby('time.month')[m]
                    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
                    RMSE = np.sqrt(((va - vrefa)**2).mean('time'))
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'c'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'w'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(RMSE, cmp=plt.get_cmap('inferno'), cbar_label='Sea ice concentration RMSE vs. GIOMAS',
                            cmap_gamma=1., cbar_ticklevels=np.arange(0, 1.01, 0.25), cbar_sigfigs=2,
                            contourvars=contourvars, **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS SIC RMSE difference from control

In [ ]:
# plot monthly SIC RMSE for each model wrt GIOMAS minus control RMSE
vname = 'aice_m'
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.groupby('time.month')[m]
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'RMSE - control'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'RMSE-control', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].groupby('time.month')[m]
                    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
                    RMSE = np.sqrt(((va - vrefa)**2).mean('time'))
                    vcontrol = control[vname].groupby('time.month')[m]
                    RMSEcontrol = np.sqrt(((vcontrol - vrefa)**2).mean('time'))
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'c'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'k'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(RMSE-RMSEcontrol, cmp=plt.get_cmap('seismic'), cbar_label='Change in SIC RMSE vs. GIOMAS due to perturbation',
                            cmap_gamma=1., cbar_ticklevels=np.arange(-0.2, 0.201, 0.05), cbar_sigfigs=2,
                            contourvars=contourvars, **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS SIT RMSE

In [ ]:
# plot monthly SIT RMSE for each model wrt GIOMAS
vname = 'hi_m'
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.groupby('time.month')[m]
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'RMSE'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'RMSE', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].groupby('time.month')[m]
                    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
                    RMSE = np.sqrt(((va - vrefa)**2).mean('time'))
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'c'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'w'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(RMSE, cmp=plt.get_cmap('inferno'), cbar_label='Sea ice thickness RMSE vs. GIOMAS (m)',
                            cmap_gamma=1., cbar_ticklevels=np.arange(0, 2.01, 0.25), cbar_sigfigs=2,
                            contourvars=contourvars, extend='max', **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS SIT RMSE difference from control

In [ ]:
# plot monthly SIT RMSE for each model wrt GIOMAS minus control RMSE
vname = 'hi_m'
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.groupby('time.month')[m]
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'RMSE - control'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'RMSE-control', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].groupby('time.month')[m]
                    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
                    RMSE = np.sqrt(((va - vrefa)**2).mean('time'))
                    vcontrol = control[vname].groupby('time.month')[m]
                    RMSEcontrol = np.sqrt(((vcontrol - vrefa)**2).mean('time'))
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'c'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'k'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(RMSE-RMSEcontrol, cmp=plt.get_cmap('seismic'), cbar_label='Change in SIT RMSE vs. GIOMAS due to perturbation (m)',
                            cmap_gamma=1., cbar_ticklevels=np.arange(-0.2, 0.201, 0.05), cbar_sigfigs=2,
                            contourvars=contourvars, extend='both', **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS snow RMSE

In [ ]:
# plot monthly snow RMSE for each model wrt GIOMAS
vname = 'hs_m'
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.groupby('time.month')[m]
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'RMSE'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'RMSE', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].groupby('time.month')[m]
                    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
                    RMSE = np.sqrt(((va - vrefa)**2).mean('time'))
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'c'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'w'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(RMSE, cmp=plt.get_cmap('inferno'), cbar_label='Snow thickness RMSE vs. GIOMAS (m)',
                            cmap_gamma=1., cbar_ticklevels=np.arange(0, 0.501, 0.1), cbar_sigfigs=1,
                            contourvars=contourvars, extend='max', **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

### Model-GIOMAS snow RMSE difference from control

In [ ]:
# plot monthly snow RMSE for each model wrt GIOMAS minus control RMSE
vname = 'hs_m'
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        sic_vobss = obsregridders[hem][ice_data[ekey][0]['res']](eval('_'.join(['obs', hem, 'mm'])))
        sic_vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS_mm['aice_m'])
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        for m in range(1,13):
            sic_vobs = sic_vobss.sel(month=m)
            vref = vrefs.groupby('time.month')[m]
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'RMSE - control'])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, 'RMSE-control', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    sic_control = control['aice_m_mm'].sel(month=m)
                    sic_v = ide['aice_m_mm'].sel(month=m)
                    sic_vref = sic_vrefs.sel(month=m)
                    v = ide[var].groupby('time.month')[m]
                    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
                    RMSE = np.sqrt(((va - vrefa)**2).mean('time'))
                    vcontrol = control[vname].groupby('time.month')[m]
                    RMSEcontrol = np.sqrt(((vcontrol - vrefa)**2).mean('time'))
                    print('doing', fname)
                    contourvars=[
                            (sic_control, control['desc'], {'levels': extlevel, 'colors': 'g'}),
                            (sic_v, ide['desc'], {'levels': extlevel, 'colors': 'm'}),
                            (sic_vref, 'GIOMAS',  {'levels': extlevel, 'colors': 'c'}),
                            (sic_vobs, 'passive microwave',  {'levels': extlevel, 'colors': 'k'})
                            ]
                    if ekey == controlkey:
                        contourvars = contourvars[0:1]+contourvars[2:]
                    plotSIC(RMSE-RMSEcontrol, cmp=plt.get_cmap('seismic'), cbar_label='Change in snow thickness RMSE vs. GIOMAS due to perturb (m)',
                            cmap_gamma=1., cbar_ticklevels=np.arange(-0.02, 0.0201, 0.005), cbar_sigfigs=3,
                            contourvars=contourvars, extend='both', **view).set_title(title);

                    savefigure(fname)
                    plt.close()
#                     break
#                 break
#             break
#         break
print('done')

## Ice daily

### Concentration

In [ ]:
# plot daily SIC obs
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        indata = obs_NH_daily
    else:
        indata = obs_SH_daily
    for date in indata['time'].data[213:578]:
        datestr = np.datetime_as_string(date, unit='D')
        title = ' '.join(['Passive microwave', datestr])
        fname = figname('_'.join([viewname, 'aice', 'obs', datestr]))
        if os.path.exists(fname):
            print('   -- skipping', fname)
        else:
            v = indata.sel(time=date)
            print('doing', fname)
            plotSIC(v, **view).set_title(title);
            savefigure(fname)
            plt.close()
print('done')

In [ ]:
# plot daily SIC from each cycle of each model
for viewname, view in views.items():
    print(viewname)
    for ekey in ice_data.keys():
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['aice']:
                for date in ide[var]['time'].data: #[213:578]:
                    datestr = date.strftime('%Y-%m-%d')
                    if datestr[0:7] == '2017-01':
                        title = ' '.join([ide['desc'], 'cycle', str(cycle), datestr])
                        fname = figname('_'.join([viewname, var, 
                                                  ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                                  datestr]))
                        if os.path.exists(fname):
                            print('   -- skipping', fname)
                        else:
                            v = ide[var].sel(time=date)
                            print('doing', fname)
                            plotSIC(v, **view).set_title(title);
                            savefigure(fname)
                            plt.close()
    #                         break
#             break  # only do cycle 1
print('done')

### Thickness

In [ ]:
# plot daily ice thickness from each cycle of each model
for viewname, view in views.items():
    print(viewname)
    if viewname == 'NH':
        cbar_ticklevels=np.arange(0, 5.01, 1.0)
    elif viewname == 'Amundsen-Bellingshausen':
        cbar_ticklevels=np.arange(0, 3.501, 0.5)
    else:
        cbar_ticklevels=np.arange(0, 2.501, 0.5)
    for ekey in ['01deg']: #ice_data.keys():
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['hi']:
                for date in ide[var]['time'].data: #[213:578]:
                    datestr = date.strftime('%Y-%m-%d')
                    title = ' '.join([ide['desc'], 'cycle', str(cycle), datestr])
                    fname = figname('_'.join([viewname, var, 
                                              ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                              datestr]))
                    if os.path.exists(fname):
                        print('   -- skipping', fname)
                    else:
                        v = ide[var].sel(time=date)
                        print('doing', fname)
                        plotSIC(v, legend=False, cbar_label='Sea ice thickness (m)',
                                cmp=cm.cm.ice,
                                cmap_gamma=1.0, cbar_ticklevels=cbar_ticklevels, cbar_sigfigs=1, extend='max', **view).set_title(title);
                        savefigure(fname)
                        plt.close()
#                         break
            break  # only do cycle 1
print('done')

## Ocean daily

In [ ]:
# plot daily MLD from each cycle of each model
for viewname, view in views.items():
    print(viewname)
    for ekey in ['01deg']: #ice_data.keys():
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['mld']:
                for date in ide[var]['time'].data:
                    datestr = date.strftime('%Y-%m-%d')
                    if True: #datestr[0:7] == '2017-01':
                        title = ' '.join([ide['desc'], 'cycle', str(cycle), datestr])
                        fname = figname('_'.join([viewname, var, 
                                                  ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                                  datestr]))
                        if os.path.exists(fname):
                            print('   -- skipping', fname)
                        else:
                            v = ide[var].sel(time=date)
                            print('doing', fname)
                            plotSIC(v, legend=False, cbar_label='Mixed layer depth (m)', cmp=cm.cm.thermal_r,
                                cmap_gamma=1.0, cbar_ticklevels=np.arange(0, 50, 10), cbar_sigfigs=1, **view).set_title(title);
                            savefigure(fname)
                            plt.close()
#                             break
            break  # only do cycle 1
print('done')

In [ ]:
# plot daily SSS from each cycle of each model
for viewname, view in views.items():
    print(viewname)
    for ekey in list(ice_data.keys())[2:]:
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['sss']:
                for date in ide[var]['time'].data:
                    datestr = date.strftime('%Y-%m-%d')
                    if datestr[0:7] == '2017-01':
                        title = ' '.join([ide['desc'], 'cycle', str(cycle), datestr])
                        fname = figname('_'.join([viewname, var, 
                                                  ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                                  datestr]))
                        if os.path.exists(fname):
                            print('   -- skipping', fname)
                        else:
                            v = ide[var].sel(time=date)
                            print('doing', fname)
                            plotSIC(v, cbar_label='Sea surface salinity', cmp=cm.cm.haline,
                                cmap_gamma=1.0, cbar_ticklevels=np.arange(31.5, 34.501, 0.5), cbar_sigfigs=2, extend='both', **view).set_title(title);
                            savefigure(fname)
                            plt.close()
#                             break
            break  # only do cycle 1
print('done')

In [ ]:
# plot daily SST from each cycle of each model
for viewname, view in views.items():
    print(viewname)
    for ekey in list(ice_data.keys())[2:]:
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            for var in ['sst']:
                for date in ide[var]['time'].data:
                    datestr = date.strftime('%Y-%m-%d')
                    if datestr[0:7] == '2017-01':
                        title = ' '.join([ide['desc'], 'cycle', str(cycle), datestr])
                        fname = figname('_'.join([viewname, var, 
                                                  ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                                  datestr]))
                        if False: #os.path.exists(fname):
                            print('   -- skipping', fname)
                        else:
                            v = ide[var].sel(time=date) - 273.15
                            print('doing', fname)
                            plotSIC(v, cbar_label='Sea surface temperature (°C)', cmp=cm.cm.thermal,
                                cmap_gamma=1.0, cbar_ticklevels=np.arange(-2, 7, 1), cbar_sigfigs=1, extend='max', **view).set_title(title);
                            savefigure(fname)
                            plt.close()
#                             break
            break  # only do cycle 1
print('done')

## JRA55-do monthly climatology

In [ ]:
# plot monthly climatological JRA siconc and siconca
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for var in ['siconc_mm', 'siconca_mm']:
        for m in range(1,13):
            title = ' '.join(['JRA55-do', var, yearrange, calendar.month_name[m], 'mean'])
            fname = figname('_'.join([viewname, var, 
                                      yearrange, 'mean', 'month', str(m).zfill(2)]))
            if os.path.exists(fname):
                print('   -- skipping', fname)
            else:
                print('doing', fname)
                v = JRA55do_data[var].sel(month=m)/100
                siconc = JRA55do_data['siconc_mm'].sel(month=m)/100
                siconca = JRA55do_data['siconca_mm'].sel(month=m)/100
                obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
                plotSIC(v, cbar_label='Sea ice concentration',
                    contourvars=[
# sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                    (siconc.isel(longitude=slice(0, 158)), 'siconc', {'levels': extlevel, 'colors': 'g'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (siconc.isel(longitude=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'g'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    (siconca.isel(longitude=slice(0, 158)), 'siconca', {'levels': extlevel, 'colors': 'r'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (siconca.isel(longitude=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'r'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    (obsv.isel(xt_ocean=slice(0, 158)), 'NSIDC', {'levels': extlevel, 'colors': 'silver'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'silver'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    ], **view).set_title(title);
                savefigure(fname)
                plt.close()

#             break
#         break
print('done')

In [ ]:
# plot monthly climatological JRA55-do siconc and siconca bias relative to NSIDC
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for var in ['siconc_mm', 'siconca_mm']: 
        for m in range(1,13):
            title = ' '.join(['JRA55-do', var, yearrange, calendar.month_name[m], 'bias'])
            fname = figname('_'.join([viewname, var+'-obs', 
                                      yearrange, 'mean', 'month', str(m).zfill(2)]))
            if os.path.exists(fname):
                print('   -- skipping', fname)
            else:
                print('doing', fname)
                siconc = JRA55do_data['siconc_mm'].sel(month=m)/100
                siconca = JRA55do_data['siconca_mm'].sel(month=m)/100
                obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
                v = JRA55do_data[var + '_regridded_to_'+hem+'obs'].sel(month=m) - obsv
                plotSIC(v, cmp=plt.get_cmap('seismic'), cbar_label='Sea ice concentration minus NSIDC',
                    cmap_gamma=1., cbar_ticklevels=np.arange(-.15,.1501,.05), cbar_sigfigs=2,
                    contourvars=[
# sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                    (siconc.isel(longitude=slice(0, 158)), 'siconc', {'levels': extlevel, 'colors': 'g'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (siconc.isel(longitude=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'g'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    (siconca.isel(longitude=slice(0, 158)), 'siconca', {'levels': extlevel, 'colors': 'y'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (siconca.isel(longitude=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'y'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    (obsv.isel(xt_ocean=slice(0, 158)), 'NSIDC', {'levels': extlevel, 'colors': 'silver'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                    (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'silver'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                    ], 
                        **view).set_title(title);
                savefigure(fname)
                plt.close()

#             break
#         break
print('done')

## JRA55-do daily means

In [ ]:
# plot daily JRA55-do siconc
for viewname, view in views.items():
    print(viewname)
    for var in ['siconc']:
        for date in JRA55do_data[var]['time'].data:
            datestr = np.datetime_as_string(date, unit='D')
            if datestr[0:7] == '2017-01':
                title = ' '.join(['JRA55-do', datestr])
                fname = figname('_'.join([viewname, var, 'JRA55-do', datestr]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    v = JRA55do_data[var].sel(time=date)/100
                    print('doing', fname)
                    plotSIC(v, cbar_label=JRA55do_data[var].long_name.replace('Area Percentage', 'Concentration'), **view).set_title(title);
#                     savefigure(fname)
#                     plt.close()
                    break
print('done')

In [ ]:
# plot daily JRA55-do uas, vas
for viewname, view in views.items():
    print(viewname)
    for var in ['uas_d']:
        for date in JRA55do_data[var]['time'].data:
            datestr = np.datetime_as_string(date, unit='D')
            if datestr[0:7] == '2017-01':
                title = ' '.join(['JRA55-do', datestr])
                fname = figname('_'.join([viewname, var, 'JRA55-do', datestr]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    v = JRA55do_data[var].sel(time=date)
                    print('doing', fname)
                    plotSIC(v, cbar_label=JRA55do_data[var].long_name+' (m/s)', cmp=plt.get_cmap('seismic'),
                        cmap_gamma=1.0, cbar_ticklevels=np.arange(-15, 15.001, 5), cbar_sigfigs=0, extend='both', **view).set_title(title);
                    savefigure(fname)
                    plt.close()
#                     break
print('done')

In [ ]:
# plot daily JRA55-do 'psl'
for viewname, view in views.items():
    print(viewname)
    for var in ['psl_d']:
        for date in JRA55do_data[var]['time'].data:
            datestr = np.datetime_as_string(date, unit='D')
            if datestr[0:7] == '2017-01':
                title = ' '.join(['JRA55-do', datestr])
                fname = figname('_'.join([viewname, var, 'JRA55-do', datestr]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    v = JRA55do_data[var].sel(time=date)/100 # convert to hPa
                    print('doing', fname)
                    plotSIC(v, cbar_label=JRA55do_data[var].long_name+' (hPa)', cmp=cm.cm.thermal,
                        cmap_gamma=1.0, cbar_ticklevels=np.arange(960, 1021, 20), cbar_sigfigs=0, extend='both', **view).set_title(title);
                    savefigure(fname)
                    plt.close()
#                     break
print('done')

In [ ]:
# plot daily JRA55-do 'tas'
for viewname, view in views.items():
    print(viewname)
    for var in ['tas_d']:
        for date in JRA55do_data[var]['time'].data:
            datestr = np.datetime_as_string(date, unit='D')
            if datestr[0:7] == '2017-01':
                title = ' '.join(['JRA55-do', datestr])
                fname = figname('_'.join([viewname, var, 'JRA55-do', datestr]))
                if False: #os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    v = JRA55do_data[var].sel(time=date) - 273.15  # convert to C
                    print('doing', fname)
                    plotSIC(v, cbar_label=JRA55do_data[var].long_name+' (°C)', cmp=cm.cm.thermal,
                        cmap_gamma=1.0, cbar_ticklevels=np.arange(-8, 3.001, 1), cbar_sigfigs=0, extend='both', **view).set_title(title);
                    savefigure(fname)
                    plt.close()
#                     break
print('done')

In [ ]:
# plot daily JRA55-do 'rlds'
for viewname, view in views.items():
    print(viewname)
    for var in ['rlds_d']:
        for date in JRA55do_data[var]['time'].data:
            datestr = np.datetime_as_string(date, unit='D')
            if datestr[0:7] == '2017-01':
                title = ' '.join(['JRA55-do', datestr])
                fname = figname('_'.join([viewname, var, 'JRA55-do', datestr]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    v = JRA55do_data[var].sel(time=date)
                    print('doing', fname)
                    plotSIC(v, cbar_label=JRA55do_data[var].long_name+' ('+JRA55do_data[var].units+')', cmp=cm.cm.thermal,
                        cmap_gamma=1.0, cbar_ticklevels=np.arange(150, 401, 50), cbar_sigfigs=0, extend='both', **view).set_title(title);
                    savefigure(fname)
                    plt.close()
#                     break
print('done')

In [ ]:
# plot daily JRA55-do 'rsds'
for viewname, view in views.items():
    print(viewname)
    for var in ['rsds_d']:
        for date in JRA55do_data[var]['time'].data:
            datestr = np.datetime_as_string(date, unit='D')
            if datestr[0:7] == '2017-01':
                title = ' '.join(['JRA55-do', datestr])
                fname = figname('_'.join([viewname, var, 'JRA55-do', datestr]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    v = JRA55do_data[var].sel(time=date)
                    print('doing', fname)
                    plotSIC(v, cbar_label=JRA55do_data[var].long_name+' ('+JRA55do_data[var].units+')', cmp=cm.cm.thermal,
                        cmap_gamma=1.0, cbar_ticklevels=np.arange(0, 401, 50), cbar_sigfigs=0, extend='max', **view).set_title(title);
                    savefigure(fname)
                    plt.close()
#                     break
print('done')

## Misc

In [ ]:
plotSIC(v, **SH);

In [ ]:
plotSIC(obs_NH.isel(time=-1), **NH).set_title('final obs');

In [ ]:
plotSIC(obs_NH_mm.sel(month=1), **NH).set_title('Obs monthly mean');

In [ ]:
plotSIC(obs_SH_mm.sel(month=1), **SH);

## JRA55-do minus ERA5 biases
- TODO: mask out land
- plot fields for forcing variables - see https://github.com/COSIMA/access-om2/issues/242#issuecomment-918795331 and https://github.com/COSIMA/01deg_jra55_iaf/blob/master/atmosphere/forcing.json and https://github.com/COSIMA/libaccessom2/blob/242-era5-support/tests/ERA5/forcing.json and https://arccss.slack.com/archives/D8BRL3VAS/p1631531254001600
| Coupling name | JRA55-do | ERA5 | Note / TODO |
| --- | --- | --- | --- |
| `swfld_ai` | `rsds` Surface Downwelling Shortwave Radiation [W m-2] | `msdrswrf` Mean surface direct short-wave radiation flux [W m**-2] | should this be `msdwswrf` Mean surface downward short-wave radiation flux [W m**-2]? Does this include `msdwuvrf` Mean surface downward UV radiation flux [W m**-2]? I guess we don't want to use the "net" quantity `msnswrf` Mean surface net short-wave radiation flux [W m**-2]?|
| `lwfld_ai` | `rlds` Surface Downwelling Longwave Radiation [W m-2] | `msdwlwrf` Mean surface downward long-wave radiation flux [W m**-2] | I guess we don't want to use the "net" quantity `msnlwrf` Mean surface net long-wave radiation flux [W m**-2]? |
| `rain_ai` | `prra` Rainfall Flux [kg m-2 s-1] | `mcpr` Mean convective precipitation rate [kg m**-2 s**-1] **and** `mlspr` Mean large-scale precipitation rate [kg m**-2 s**-1] | does snowfall need to be subtracted? Or use `crr` Convective rain rate [kg m**-2 s**-1] and `lsrr` Large scale rain rate [kg m**-2 s**-1] instead? Or`mtpr` Mean total precipitation rate [kg m**-2 s**-1] and `ptype` Precipitation type [[code table (4.201)](https://apps.ecmwf.int/codes/grib/format/grib2/ctables/4/201)] instead? |
| `snow_ai` | `prsn` Snowfall Flux [kg m-2 s-1] | `mlssr` Mean large-scale snowfall rate [kg m**-2 s**-1] | add `mcsr` Mean convective snowfall rate [kg m**-2 s**-1]? or just use `msr`	Mean snowfall rate [kg m**-2 s**-1]? what about `csfr` Convective snowfall rate water equivalent [kg m**-2 s**-1] and `lssfr` Large scale snowfall rate water equivalent [kg m**-2 s**-1] ?|
| `press_ai` | `psl` Sea Level Pressure [Pa] | `msl` Mean sea level pressure [Pa] |  |
| `runof_ai` | `friver` Water Flux into Sea Water from Rivers [kg m-2 s-1] | `msror` Mean surface runoff rate [kg m**-2 s**-1] **and** `mssror` Mean sub-surface runoff rate [kg m**-2 s**-1] | or just use `mror` Mean runoff rate [kg m**-2 s**-1]? |
| `tair_ai` | `tas` Near-Surface (10m) Air Temperature [K] | `t2m` 2 metre temperature [K] | [Change to 10m height](https://github.com/COSIMA/access-om2/issues/242#issuecomment-1012817544)  - see appendix A2 in [Tsujino et al 2018](https://doi.org/10.1016/j.ocemod.2018.07.002)|
| `qair_ai` | `huss` Near-Surface (10m) Specific Humidity [1]| `t2m` 2 metre temperature [K] **and** `d2m` 2 metre dewpoint temperature [K] | [Convert dew point to specific humidity](https://github.com/COSIMA/access-om2/issues/242#issuecomment-918783268) (also requires `msl` Mean sea level pressure [Pa]) and [change to 10m height](https://github.com/COSIMA/access-om2/issues/242#issuecomment-1012817544) - see appendix A2 in [Tsujino et al 2018](https://doi.org/10.1016/j.ocemod.2018.07.002) |
| `uwnd_ai` | `uas` Eastward Near-Surface (10m) Wind [m s-1] | `u10` 10 metre U wind component [m s**-1] |  |
| `vwnd_ai` | `vas` Northward Near-Surface (10m) Wind [m s-1] | `v10` 10 metre V wind component [m s**-1] |  |
| `licalvf_ai` | `licalvf` Land Ice Calving Flux [kg m-2 s-1] |  | not available in ERA5 - use JRA55-do values? |

In [ ]:
varmapping = { # map variables JRA55-do: [ ERA5 ... ] based on table above
    'rsds': ['msdrswrf'],
    'rlds': ['msdwlwrf'],
    'prra': ['mcpr', 'mlspr'],
    'prsn': ['mcsr', 'mlssr'],
    'psl': ['msl'],
    'friver': ['msror', 'mssror'],
    'tas': ['t2m'],
    'huss': ['t2m', 'd2m', 'msl'],
    'uas': ['u10'],
    'vas': ['v10'] }

In [ ]:
# load JRA55do forcing variables and corresponding ERA5 variable(s)
for JRA55dovname, ERA5vararray in varmapping.items():
    loadJRA55do(JRA55do_data, varnames=[JRA55dovname])
    loadERA5(ERA5_data, varnames=ERA5vararray)

In [ ]:
loaddata(ice_data['025deg'][0], varnames=['tmask'], n=1)
landmask025 = ice_data['025deg'][0]['tmask'].fillna(0)

In [ ]:
mm025 = dict()

In [ ]:
# regrid JRA55do forcing variables and corresponding ERA5 variable(s) to model 0.25deg grid
for JRA55dovname, ERA5vararray in varmapping.items():
    print(JRA55dovname)
    mm025[JRA55dovname] = dict()
    mm025[JRA55dovname]['JRA55do'] = JRA55do_to_025(method='patch')(JRA55do_data[JRA55dovar+'_mm']).where(landmask025 != 0)  # use 'patch' method to match access-om2
    mm025[JRA55dovname]['ERA5'] = {k: ERA5_to_025(method='patch')(ERA5_data[k+'_mm']).where(landmask025 != 0)  # use 'patch' method to match access-om2
                                 for k in ERA5vararray}

In [ ]:
# plot JRA55do minus corresponding ERA5 variable(s)
for JRA55dovname, ddict in mm025.items():
    print(JRA55dovname)
    JRA55dov = ddict['JRA55do']
    for ERA5vname, ERA5v in ddict['ERA5'].items():
        plt.figure()
        (JRA55dov - ERA5v).sel(month=1).plot()
        plt.title('JRA55-do '+JRA55dovname+' minus ERA5 '+ERA5vname)
    #     break
    # break



In [ ]:
mm025

In [ ]:
JRA55do_tas_025 = JRA55do_to_025(method='patch')(JRA55do_data['tas'+'_mm']).where(landmask != 0)  # use 'patch' method to match access-om2

In [ ]:
landmasks = ['1deg', '025deg', '01deg']

In [ ]:
# get model land masks
# for ekey in ice_data.keys():
# for ekey in ensemble: # just the ensemble
for ekey in landmasks:
    print(ekey)
    for ide in ice_data[ekey]:
        loaddata(ide, varnames=['tmask'], n=1)

### Temperature
See https://github.com/COSIMA/access-om2/issues/242#issuecomment-914960099

In [ ]:
def SATplot(dataname, dataset, vname):
    for viewname, view in views.items():
        print(viewname)
        if viewname in ['NH']:
            hem = 'NH'
        else:
            hem = 'SH'
        for m in range(1, 13):
            title = ' '.join([dataname, yearrange, calendar.month_name[m], 'mean'])
            fname = figname('_'.join([viewname, vname, dataname, yearrange, 'mean', 'month', str(m).zfill(2)]))
            if os.path.exists(fname):
                print('   -- skipping', fname)
            else:
                print('doing', fname)
                plt.close()
                cbar_ticklevels=np.arange(-40, 10, 5)
                plotSIC(dataset[vname].sel(month=m)-273.15, cbar_label='Surface air temperature (°C)',
                                    cmap_gamma=1., cbar_ticklevels=cbar_ticklevels, cbar_sigfigs=1, extend='both', cmp=cm.cm.thermal,
                                    # contourvars=[], 
                                    legend=False,
                                    feature=cft.NaturalEarthFeature('physical', 'land', '50m',
                                                   edgecolor='black', facecolor='None', 
                                                   linewidth=0.5),
                                    featureoverplot=True,
                                    contourvars=[
                                        (ice_data['1deg'][0]['tmask'], '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'C0'}),
                                        (ice_data['025deg'][0]['tmask'], '0.25° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'C1'}),
                                        (ice_data['01deg'][0]['tmask'], '0.1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'C2'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(xt_ocean=slice(180, 360)), '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'r'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(xt_ocean=slice(0, 158)), '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'r'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(xt_ocean=slice(159, 360)), None, {'levels': [0, 1e-15], 'colors': 'r'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(yt_ocean=slice(-90, -50)), '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'r'})
    #                                 (JRA55do_data['tas_mm'].sel(month=m), 'bla', {'levels': extlevel, 'colors': 'C'+str(enum+3)}), # 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}), # old v1 config
    #                                 (v, ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
    # # sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
    #                                 (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave',     {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
    #                                 (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'white'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                            ],
                                    **view).set_title(title);
                savefigure(fname)
                # plt.close()
            # break

In [ ]:
SATplot('JRA55-do', JRA55do_data, 'tas_mm')

In [ ]:
SATplot('ERA5', ERA5_data, 't2m_mm')

In [ ]:
def SATbiasplot():
    for viewname, view in views.items():
        print(viewname)
        if viewname in ['NH']:
            hem = 'NH'
        else:
            hem = 'SH'
        for m in range(1, 13):
            title = ' '.join(['JRA55-do 10m SAT - ERA5 2m SAT', yearrange, calendar.month_name[m], 'mean'])
            fname = figname('_'.join([viewname, 'JRA55-do_tas_mm-ERA5_t2m_mm', yearrange, 'mean', 'month', str(m).zfill(2)]))
            if os.path.exists(fname):
                print('   -- skipping', fname)
            else:
                print('doing', fname)
                plt.close()
                cbar_ticklevels=np.arange(-5, 5.01, 1)
                obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
                plotSIC(bias[m-1], cbar_label='Surface air temperature difference (°C)',
                                    cmap_gamma=1., cbar_ticklevels=cbar_ticklevels, cbar_sigfigs=1, extend='both', cmp=plt.get_cmap('seismic'),
                                    # contourvars=[], 
                                    legend=False,
                                    feature=cft.NaturalEarthFeature('physical', 'land', '50m',
                                                   edgecolor='black', facecolor='None', 
                                                   linewidth=0.5),
                                    featureoverplot=True,
                                    contourvars=[
                                        # (ice_data['1deg'][2]['aice_m_mm'].sel(month=m), None, {'label': '1° ACCESS-OM2 SIC', 'levels': [0.15, 0.15+1e-9], 'colors': 'C0'}),
                                        # (ice_data['025deg'][2]['aice_m_mm'].sel(month=m), None, {'label': '0.25° ACCESS-OM2-025 SIC', 'levels': [0.15, 0.15+1e-9], 'colors': 'C1'}),
                                        # (ice_data['01deg'][2]['aice_m_mm'].sel(month=m), None, {'label': '0.1° ACCESS-OM2-01 SIC', 'levels': [0.15, 0.15+1e-9], 'colors': 'C2'}),
                                        (ice_data['1deg'][2]['aice_m_mm'].sel(month=m), '1° ACCESS-OM2 SIC', {'levels': [0.15, 0.15+1e-9], 'colors': 'C0'}),
                                        (ice_data['025deg'][2]['aice_m_mm'].sel(month=m), '0.25° ACCESS-OM2-025 SIC', {'levels': [0.15, 0.15+1e-9], 'colors': 'C1'}),
                                        (ice_data['01deg'][2]['aice_m_mm'].sel(month=m), '0.1° ACCESS-OM2-01 SIC', {'levels': [0.15, 0.15+1e-9], 'colors': 'C2'}),
                                        # (ice_data['1deg'][0]['tmask'], '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'C0'}),
                                        # (ice_data['025deg'][0]['tmask'], '0.25° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'C1'}),
                                        # (ice_data['01deg'][0]['tmask'], '0.1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'C2'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(xt_ocean=slice(180, 360)), '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'r'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(xt_ocean=slice(0, 158)), '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'r'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(xt_ocean=slice(159, 360)), None, {'levels': [0, 1e-15], 'colors': 'r'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(yt_ocean=slice(-90, -50)), '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'r'})
    #                                 (JRA55do_data['tas_mm'].sel(month=m), 'bla', {'levels': extlevel, 'colors': 'C'+str(enum+3)}), # 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}), # old v1 config
    #                                 (v, ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
    # # sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                        (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave SIC',     {'levels': extlevel, 'colors': 'dimgray'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                                        (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'dimgray'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                                        (ice_data['1deg'][0]['tmask'], None, {'levels': [0, 1e-15], 'colors': 'k'}),
                            ],
                                    **view).set_title(title);
                savefigure(fname)
                plt.close()
            # break

In [ ]:
bias = [None]*12

In [ ]:
# month 5 doesn't work, presumably due to bad chunking in /g/data/rt52/era5/single-levels/reanalysis/2t/2014/2t_era5_oper_sfc_20140501-20140531.nc
for m in [5]: #[6, 8, 9, 10, 11, 12]: # [5, 6, 8, 9]: #range(1, 13):
    if bias[m-1] is None:
        print('doing month', str(m))
        bias[m-1] = (JRA55do_to_ERA5(JRA55do_data['tas_mm'].sel(month=m))\
                     - ERA5_data['t2m_mm'].sel(month=m)).load()
#         break

In [ ]:
SATbiasplot()

In [ ]:
bias[0]

In [ ]:
JRA55do_data['tas_mm'].sel(month=1).plot()

In [ ]:
JRA55do_to_ERA5(JRA55do_data['tas_mm'].sel(month=1)).plot()

In [ ]:
ERA5_data['t2m_mm'].sel(month=1).plot()

In [ ]:
for vrange in [40, 10, 5, 3]:
    plt.figure(figsize=(20, 20))
    for m in range(1, 13):
        plt.subplot(4,3,m)
        ax = bias[m-1].plot(vmin=-abs(vrange), vmax=abs(vrange), cmap=plt.get_cmap('seismic'))
        plt.title(' '.join(['JRA55-do T_10 - ERA55 T_2', yearrange, calendar.month_name[m], 'mean']))
    savefigure(figname('_'.join(['JRA55-do', 'T_10', 'minus', 'ERA55', 'T_2', yearrange, 'monthly', 'mean', 'vrange', str(vrange)])))
    plt.close()

### Zonal wind

In [ ]:
bias = [None]*12

In [ ]:
for m in range(1, 13):
    if bias[m-1] is None:
        print('doing month', str(m))
        bias[m-1] = (JRA55do_to_ERA5(JRA55do_data['uas_mm'].sel(month=m))\
                     - ERA5_data['u10_mm'].sel(month=m)).load()
#         break

In [ ]:
JRA55do_data['uas_mm'].sel(month=1).plot()

In [ ]:
JRA55do_to_ERA5(JRA55do_data['uas_mm'].sel(month=1)).plot()

In [ ]:
ERA5_data['u10_mm'].sel(month=1).plot()

In [ ]:
for vrange in [8, 2]:
    plt.figure(figsize=(20, 20))
    for m in range(1, 13):
        plt.subplot(4,3,m)
#         ax = bias[m-1].plot(cmap=plt.get_cmap('seismic'))
        ax = bias[m-1].plot(vmin=-abs(vrange), vmax=abs(vrange), cmap=plt.get_cmap('seismic'))
        plt.title(' '.join(['JRA55-do u_10 - ERA55 u_10', yearrange, calendar.month_name[m], 'mean']))
#         break
#     break
    savefigure(figname('_'.join(['JRA55-do', 'u_10', 'minus', 'ERA55', 'u_10', yearrange, 'monthly', 'mean', 'vrange', str(vrange)])))
    plt.close()

### Meridional wind

In [ ]:
bias = [None]*12

In [ ]:
for m in range(1, 13):
    if bias[m-1] is None:
        print('doing month', str(m))
        bias[m-1] = (JRA55do_to_ERA5(JRA55do_data['vas_mm'].sel(month=m))\
                     - ERA5_data['v10_mm'].sel(month=m)).load()
#         break

In [ ]:
JRA55do_data['vas_mm'].sel(month=1).plot()

In [ ]:
JRA55do_to_ERA5(JRA55do_data['vas_mm'].sel(month=1)).plot()

In [ ]:
ERA5_data['v10_mm'].sel(month=1).plot()

In [ ]:
for vrange in [8, 2]:
    plt.figure(figsize=(20, 20))
    for m in range(1, 13):
        plt.subplot(4,3,m)
#         ax = bias[m-1].plot(cmap=plt.get_cmap('seismic'))
        ax = bias[m-1].plot(vmin=-abs(vrange), vmax=abs(vrange), cmap=plt.get_cmap('seismic'))
        plt.title(' '.join(['JRA55-do v_10 - ERA55 v_10', yearrange, calendar.month_name[m], 'mean']))
#         break
#     break
    savefigure(figname('_'.join(['JRA55-do', 'v_10', 'minus', 'ERA55', 'v_10', yearrange, 'monthly', 'mean', 'vrange', str(vrange)])))
    plt.close()

## JRA55-do minus JRA55 biases

### Temperature

In [ ]:
def SAT_JRA55do_JRA55_bias_plot():
    for viewname, view in views.items():
        print(viewname)
        if viewname in ['NH']:
            hem = 'NH'
        else:
            hem = 'SH'
        for m in range(1, 13):
            title = ' '.join(['JRA55-do 10m SAT - JRA55 2m SAT', yearrange, calendar.month_name[m], 'mean'])
            fname = figname('_'.join([viewname, 'JRA55-do_tas_mm-JRA55_tas_mm', yearrange, 'mean', 'month', str(m).zfill(2)]))
            if os.path.exists(fname):
                print('   -- skipping', fname)
            else:
                print('doing', fname)
                plt.close()
                cbar_ticklevels=np.arange(-5, 5.01, 1)
                obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
                plotSIC(JRA55do_JRA55_bias[m-1], cbar_label='Surface air temperature difference (°C)',
                                    cmap_gamma=1., cbar_ticklevels=cbar_ticklevels, cbar_sigfigs=1, extend='both', cmp=plt.get_cmap('seismic'),
                                    # contourvars=[], 
                                    legend=False,
                                    feature=cft.NaturalEarthFeature('physical', 'land', '50m',
                                                   edgecolor='black', facecolor='None', 
                                                   linewidth=0.5),
                                    featureoverplot=True,
                                    contourvars=[
                                        # (ice_data['1deg'][2]['aice_m_mm'].sel(month=m), None, {'label': '1° ACCESS-OM2 SIC', 'levels': [0.15, 0.15+1e-9], 'colors': 'C0'}),
                                        # (ice_data['025deg'][2]['aice_m_mm'].sel(month=m), None, {'label': '0.25° ACCESS-OM2-025 SIC', 'levels': [0.15, 0.15+1e-9], 'colors': 'C1'}),
                                        # (ice_data['01deg'][2]['aice_m_mm'].sel(month=m), None, {'label': '0.1° ACCESS-OM2-01 SIC', 'levels': [0.15, 0.15+1e-9], 'colors': 'C2'}),
                                        (ice_data['1deg'][2]['aice_m_mm'].sel(month=m), '1° ACCESS-OM2 SIC', {'levels': [0.15, 0.15+1e-9], 'colors': 'C0'}),
                                        (ice_data['025deg'][2]['aice_m_mm'].sel(month=m), '0.25° ACCESS-OM2-025 SIC', {'levels': [0.15, 0.15+1e-9], 'colors': 'C1'}),
                                        (ice_data['01deg'][2]['aice_m_mm'].sel(month=m), '0.1° ACCESS-OM2-01 SIC', {'levels': [0.15, 0.15+1e-9], 'colors': 'C2'}),
                                        # (ice_data['1deg'][0]['tmask'], '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'C0'}),
                                        # (ice_data['025deg'][0]['tmask'], '0.25° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'C1'}),
                                        # (ice_data['01deg'][0]['tmask'], '0.1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'C2'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(xt_ocean=slice(180, 360)), '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'r'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(xt_ocean=slice(0, 158)), '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'r'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(xt_ocean=slice(159, 360)), None, {'levels': [0, 1e-15], 'colors': 'r'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(yt_ocean=slice(-90, -50)), '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'r'})
    #                                 (JRA55do_data['tas_mm'].sel(month=m), 'bla', {'levels': extlevel, 'colors': 'C'+str(enum+3)}), # 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}), # old v1 config
    #                                 (v, ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
    # # sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                        (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave SIC',     {'levels': extlevel, 'colors': 'dimgray'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                                        (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'dimgray'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                                        (ice_data['1deg'][0]['tmask'], None, {'levels': [0, 1e-15], 'colors': 'k'}),
                            ],
                                    **view).set_title(title);
                savefigure(fname)
                plt.close()
            # break

In [ ]:
JRA55do_JRA55_bias = [None]*12

In [ ]:
for m in range(1, 13):
    if JRA55do_JRA55_bias[m-1] is None:
        print('doing month', str(m))
        JRA55do_JRA55_bias[m-1] = (JRA55do_data['tas_mm'] - JRA55_to_JRA55do(JRA55_data['tas_mm'])).sel(month=m).load()

In [ ]:
SAT_JRA55do_JRA55_bias_plot()

## JRA55 minus ERA5 biases

### Temperature

In [ ]:
def SAT_JRA55_ERA5_bias_plot():
    for viewname, view in views.items():
        print(viewname)
        if viewname in ['NH']:
            hem = 'NH'
        else:
            hem = 'SH'
        for m in range(1, 13):
            title = ' '.join(['JRA55 2m SAT - ERA5 2m SAT', yearrange, calendar.month_name[m], 'mean'])
            fname = figname('_'.join([viewname, 'JRA55_tas_mm-ERA5_t2m_mm', yearrange, 'mean', 'month', str(m).zfill(2)]))
            if os.path.exists(fname):
                print('   -- skipping', fname)
            else:
                print('doing', fname)
                plt.close()
                cbar_ticklevels=np.arange(-5, 5.01, 1)
                obsv = eval('_'.join(['obs', hem, 'mm'])).sel(month=m)
                plotSIC(JRA55_ERA5_bias[m-1], cbar_label='Surface air temperature difference (°C)',
                                    cmap_gamma=1., cbar_ticklevels=cbar_ticklevels, cbar_sigfigs=1, extend='both', cmp=plt.get_cmap('seismic'),
                                    # contourvars=[], 
                                    legend=False,
                                    feature=cft.NaturalEarthFeature('physical', 'land', '50m',
                                                   edgecolor='black', facecolor='None', 
                                                   linewidth=0.5),
                                    featureoverplot=True,
                                    contourvars=[
                                        # (ice_data['1deg'][2]['aice_m_mm'].sel(month=m), None, {'label': '1° ACCESS-OM2 SIC', 'levels': [0.15, 0.15+1e-9], 'colors': 'C0'}),
                                        # (ice_data['025deg'][2]['aice_m_mm'].sel(month=m), None, {'label': '0.25° ACCESS-OM2-025 SIC', 'levels': [0.15, 0.15+1e-9], 'colors': 'C1'}),
                                        # (ice_data['01deg'][2]['aice_m_mm'].sel(month=m), None, {'label': '0.1° ACCESS-OM2-01 SIC', 'levels': [0.15, 0.15+1e-9], 'colors': 'C2'}),
                                        (ice_data['1deg'][2]['aice_m_mm'].sel(month=m), '1° ACCESS-OM2 SIC', {'levels': [0.15, 0.15+1e-9], 'colors': 'C0'}),
                                        (ice_data['025deg'][2]['aice_m_mm'].sel(month=m), '0.25° ACCESS-OM2-025 SIC', {'levels': [0.15, 0.15+1e-9], 'colors': 'C1'}),
                                        (ice_data['01deg'][2]['aice_m_mm'].sel(month=m), '0.1° ACCESS-OM2-01 SIC', {'levels': [0.15, 0.15+1e-9], 'colors': 'C2'}),
                                        # (ice_data['1deg'][0]['tmask'], '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'C0'}),
                                        # (ice_data['025deg'][0]['tmask'], '0.25° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'C1'}),
                                        # (ice_data['01deg'][0]['tmask'], '0.1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'C2'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(xt_ocean=slice(180, 360)), '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'r'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(xt_ocean=slice(0, 158)), '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'r'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(xt_ocean=slice(159, 360)), None, {'levels': [0, 1e-15], 'colors': 'r'}),
                                        # (ice_data['1deg'][0]['tmask'].sel(yt_ocean=slice(-90, -50)), '1° ACCESS-OM2', {'levels': [0, 1e-15], 'colors': 'r'})
    #                                 (JRA55do_data['tas_mm'].sel(month=m), 'bla', {'levels': extlevel, 'colors': 'C'+str(enum+3)}), # 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}), # old v1 config
    #                                 (v, ide['desc'], {'levels': extlevel, 'colors': 'C'+str(enum)}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
    # # sliced to work around a cartopy bug https://stackoverflow.com/questions/55062406/cartopy-fails-to-correctly-contour-data-on-rotated-grid
                                        (obsv.isel(xt_ocean=slice(0, 158)), 'passive microwave SIC',     {'levels': extlevel, 'colors': 'dimgray'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()}),
                                        (obsv.isel(xt_ocean=slice(158, 100000)), None, {'levels': extlevel, 'colors': 'dimgray'}), #, 'x': 'longitude', 'y': 'latitude', 'transform': ccrs.PlateCarree()})],
                                        (ice_data['1deg'][0]['tmask'], None, {'levels': [0, 1e-15], 'colors': 'k'}),
                            ],
                                    **view).set_title(title);
                savefigure(fname)
                plt.close()
            # break

In [ ]:
JRA55_ERA5_bias = [None]*12

In [ ]:
for m in range(1, 13):
    if JRA55_ERA5_bias[m-1] is None:
        print('doing month', str(m))
        JRA55_ERA5_bias[m-1] = (JRA55_to_ERA5(JRA55_data['tas_mm'].sel(month=m))\
                     - ERA5_data['t2m_mm'].sel(month=m)).load()


In [ ]:
SAT_JRA55_ERA5_bias_plot()

# Histograms vs parameters - TODO: FINISH

TODO: weight by area

TODO: plot groups by perturbation variable as in 


In [ ]:
def calcerr(v, vref, method='MAD', mean=[]):
    va, vrefa = xr.align(v, vref, join='override', exclude=['xt_ocean', 'yt_ocean'])
    if method == 'MAD':
        e = (np.abs(va - vrefa)).mean(mean)
    elif method == 'RMSE': 
        e = np.sqrt(((va - vrefa)**2).mean(mean))
    elif method == 'bias':
        e = (va - vrefa).mean(mean)
    else:
        return None
    return e.where(xr.ufuncs.logical_not(np.logical_and(va==0, vrefa==0)))  # set to nan if both zero

In [ ]:
vname = 'hi_m'
method = 'bias'
mean = []
var = vname
controlkey = controlkeys[0]  # KLUDGE! do this better
# controlkey = '025deg'
control = ice_data[controlkey][0]
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    for ekey in ensemble:
        vrefs = GIOMASregridders[ice_data[ekey][0]['res']](GIOMAS[vname])
        if hem == 'NH':
            vrefs = vrefs.where(vrefs.yt_ocean>0)
        else:
            vrefs = vrefs.where(vrefs.yt_ocean<0)
        for m in range(1,13):
            vref = vrefs.groupby('time.month')[m]
# BUG: try something different from broadcast_to 
            area = xr.DataArray(np.broadcast_to(ide['area_t'], vref.shape[0:1]+ide['area_t'].shape).ravel())
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], method])
                fname = figname('_'.join([viewname, var, 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle)+'-GIOMAS', # FRAGLE! assumes filename cycle number = cycm2+1
                                          yearrange, method, 'histogram', 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
                    print('doing', fname)
                    v = ide[var].groupby('time.month')[m]
                    if hem == 'NH':
                        v = v.where(v.yt_ocean>0)
                    else:
                        v = v.where(v.yt_ocean<0)
        #                     err = calcerr(v, vref, method=method, mean=[])
                    err = xr.DataArray(calcerr(v, vref, method=method, mean=[]).data.ravel())
#                     err = np.abs(v-vref)
#                     err = v-vref
#                     histdata = 
                    xr.plot.hist(err, weights=area, bins=500, log=True, density=True, histtype='step')
#                     plt.hist(err, weights=area, bins=500, log=True, density=True, histtype='step')
                    plt.title(title)
                    plt.xlabel('')
                break
            break
        break
    break
print('done')

In [ ]:
area.shape


In [ ]:
err.shape == area.shape

In [ ]:
len(err)

In [ ]:
len(area)

In [ ]:
xr.DataArray(err).values

In [ ]:
area.shape

In [ ]:
vref

In [ ]:
np.broadcast_to(ide['area_t'], err.shape[0:1]+ide['area_t'].shape)

In [ ]:
np.broadcast_to(ide['area_t'], err.shape[0:1]+ide['area_t'].shape).shape

In [ ]:
ide['area_t'].shape

In [ ]:
err.shape[0:1]

# 2D Histograms
TODO: 
- weight count by area?
- normalise by obs count?

## SIC histogram model vs. obs
TODO: 
- weight count by area?
- normalise by obs count?

In [ ]:
for viewname, view in views.items():
    print(viewname)
    for ekey in ensemble:
        if viewname in ['NH']:
            hem = 'NH'
        else:
            hem = 'SH'
        obsv = eval('_'.join(['obs', hem]))
        vref = obsregridders[hem][ice_data[ekey][0]['res']](obsv)
        vref = vref.isel(time=slice(0, -1))            
#         vref.time.data = ice_data[ekey][0]['aice_m'].time.data
        # obs dates are first day of month but cice dates are last day of month
        # so align vref to use same dates as cice
#         vref, _ = xr.align(vref, ice_data[ekey][0]['aice_m'], join='right')
#         vref_m = vref.groupby('time.month')
    #     break
        for cycle, ide in enumerate(ice_data[ekey], start=1):
            v = ide['aice_m']
            if hem == 'NH':
                v = v.where(v.yt_ocean>0)
            else:
                v = v.where(v.yt_ocean<0)
#             bias = v - vref
#             bias_m = bias.groupby('time.month')
            for m in range(1, 13):
    #             v = ide[var]
    #             vref = vrefs.sel(month=m)
                title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'bias'])
                fname = figname('_'.join([viewname, 'aice_m', 
                                          ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycm2+1
                                          'hist_vs_obs',
                                          yearrange, 'month', str(m).zfill(2)]))
                if os.path.exists(fname):
                    print('   -- skipping', fname)
                else:
    #                     v = ide[var].sel(month=m)
    #                     bias = v - vref
                    print('doing', fname)
#                     plt.scatter(vref_m.sel(month=m), bias_m.sel(month=m))
                    vrefm = vref.sel(time=(vref['time.month']==m)).data.ravel()
                    vm = v.sel(time=(v['time.month']==m)).data.ravel()
                    fig, ax = plt.subplots(nrows=1, ncols=1)
                    ax.set_facecolor('k')
#                     fig = plt.figure() #(figsize=(7,7))
#                     fig.patch.set_facecolor('black')

#                     plt.scatter(vrefm, vm-vrefm, s=.1)
#                     plt.hist2d(vref.isel(time=8).data.ravel(), 
#                       v.where(v.yt_ocean<0).isel(time=8).data.ravel(),
#                       bins=100, range=[[0, 1], [0, 1]], norm=colors.LogNorm(),
#                       cmap=plt.cm.get_cmap('nipy_spectral_r'));
                    plt.hist2d(vrefm, vm,
                      bins=100, range=[[0, 1], [0, 1]], 
                      norm=colors.LogNorm(), 
                    # TODO: pick cmax value automatically
                      cmax=1e5,  # cmax=1e5 for 1deg, 1e6 for 0.25deg, 
#                       cmap=cm.cm.ice)
                      cmap=plt.cm.get_cmap('hot'))
#                       cmap=plt.cm.get_cmap('nipy_spectral'))
                    plt.plot([0, 1],[0, 1], 'g', linewidth=2)
                    plt.colorbar()
                    plt.title(title)
                    plt.xlabel('NSIDC passive microwave SIC')
                    plt.ylabel('Model SIC')
                    savefigure(fname)
                    plt.close()
#                     break
#             break
#         break


print('done')

## SIC histogram GIOMAS vs. obs

In [ ]:
# histograms for GIOMAS SIC
GIOMAS_aice_m_regridded = regrid_giomas_to_025(GIOMAS['aice_m'])
for viewname, view in views.items():
    print(viewname)
    if viewname in ['NH']:
        hem = 'NH'
    else:
        hem = 'SH'
    obsv = eval('_'.join(['obs', hem]))
    vref = eval('regrid_'+hem+'obs_to_025(obsv)')
    vref = vref.isel(time=slice(0, -1))            
    v = GIOMAS_aice_m_regridded
    if hem == 'NH':
        v = v.where(v.yt_ocean>0)
    else:
        v = v.where(v.yt_ocean<0)
    for m in range(1, 13):
        title = ' '.join(['GIOMAS', yearrange, calendar.month_name[m], 'bias'])
        fname = figname('_'.join([viewname, 'aice_m', 
                                  'GIOMAS',
                                  'hist_vs_obs',
                                  yearrange, 'month', str(m).zfill(2)]))
        if os.path.exists(fname):
            print('   -- skipping', fname)
        else:
            print('doing', fname)
            vrefm = vref.sel(time=(vref['time.month']==m)).data.ravel()
            vm = v.sel(time=(v['time.month']==m)).data.ravel()
            fig, ax = plt.subplots(nrows=1, ncols=1)
            ax.set_facecolor('k')
            plt.hist2d(vrefm, vm,
              bins=100, range=[[0, 1], [0, 1]], 
              norm=colors.LogNorm(), 
            # TODO: pick cmax value automatically
              cmax=1e5,  # cmax=1e5 for 1deg, 1e6 for 0.25deg, 
              cmap=plt.cm.get_cmap('hot'))
            plt.plot([0, 1],[0, 1], 'g', linewidth=2)
            plt.colorbar()
            plt.title(title)
            plt.xlabel('NSIDC passive microwave SIC')
            plt.ylabel('GIOMAS SIC')
            savefigure(fname)
            plt.close()
#             break
#         break
#     break

print('done')

## Histograms model vs. GIOMAS

In [ ]:
def hist_model_vs_GIOMAS(varname='aice_m', varrange=[0, 1], desc='sea ice concentration'):
    for viewname, view in views.items():
        print(viewname)
        for ekey in ensemble:
            if viewname in ['NH']:
                hem = 'NH'
            else:
                hem = 'SH'
            obsv = GIOMAS[varname]
            vref = GIOMASregridders[ice_data[ekey][0]['res']](obsv)
            if hem == 'NH':
                vref = vref.where(vref.yt_ocean>0)
            else:
                vref = vref.where(vref.yt_ocean<0)
            for cycle, ide in enumerate(ice_data[ekey], start=1):
                v = ide[varname]
                if hem == 'NH':
                    v = v.where(v.yt_ocean>0)
                else:
                    v = v.where(v.yt_ocean<0)
    #             bias = v - vref
    #             bias_m = bias.groupby('time.month')
                for m in range(1, 13):
        #             v = ide[var]
        #             vref = vrefs.sel(month=m)
                    title = ' '.join([ide['desc'], 'cycle', str(cycle), yearrange, calendar.month_name[m], 'bias'])
                    fname = figname('_'.join([viewname, varname, 
                                              ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycm2+1
                                              'hist_vs_GIOMAS',
                                              yearrange, 'month', str(m).zfill(2)]))
                    if os.path.exists(fname):
                        print('   -- skipping', fname)
                    else:
        #                     v = ide[var].sel(month=m)
        #                     bias = v - vref
                        print('doing', fname)
    #                     plt.scatter(vref_m.sel(month=m), bias_m.sel(month=m))
                        vrefm = vref.sel(time=(vref['time.month']==m)).data.ravel()
                        vm = v.sel(time=(v['time.month']==m)).data.ravel()
                        fig, ax = plt.subplots(nrows=1, ncols=1)
                        ax.set_facecolor('k')
                        plt.hist2d(vrefm, vm,
                          bins=100, range=[varrange, varrange], 
                          norm=colors.LogNorm(), 
                        # TODO: pick cmax value automatically
                          cmax=1e5,  # cmax=1e5 for 1deg, 1e6 for 0.25deg, 
    #                       cmap=cm.cm.ice)
                          cmap=plt.cm.get_cmap('hot'))
    #                       cmap=plt.cm.get_cmap('nipy_spectral'))
                        plt.plot([0, 2.5],[0, 2.5], 'g', linewidth=2)
                        plt.colorbar()
                        plt.title(title)
                        plt.xlabel(' '.join(['GIOMAS', desc]))
                        plt.ylabel(' '.join([ide['desc'], desc]))
                        savefigure(fname)
                        plt.close()
#                         break
#                 break
#             break
    print('done')

In [ ]:
hist_model_vs_GIOMAS()

In [ ]:
hist_model_vs_GIOMAS(varname='hi_m', varrange=[0, 2.5], desc='sea ice thickness (m)')

In [ ]:
hist_model_vs_GIOMAS(varname='hs_m', varrange=[0, 0.7], desc='snow thickness (m)')

## Histograms JRA55-do vs. ERA5

Plot ERA5 vs JRA55-do to check that we're using the right ERA5 forcing fields and also see how much they differ. 

See https://github.com/COSIMA/access-om2/issues/242#issuecomment-918795331

#### TODO
- regrid JRA55-do, JRA55, ERA5 onto 0.25deg model T grid (since ERA5 is 0.25deg and JRA55-do is 0.5625deg)
    - ideally use the same regridding as the model
    /g/data/ik11/inputs/access-om2/input_20201022/common_025deg_jra55/rmp_jra55_cice_patch.nc
    and the regridders Nic has defined?
- ignore land data (use model land mask)
- weight by 0.25 model grid cell area 
- density

In [ ]:
loaddata(ice_data['025deg'][0], varnames=['tmask'], n=1)
landmask = ice_data['025deg'][0]['tmask'].fillna(0)

In [ ]:
landmask.plot()

In [ ]:
JRA55do_tas_025 = JRA55do_to_025(method='patch')(JRA55do_data['tas'+'_mm']).where(landmask != 0)  # use 'patch' method to match access-om2

In [ ]:
JRA55_tas_025 = JRA55_to_025(method='patch')(JRA55_data['tas'+'_mm']).where(landmask != 0)  # use 'patch' method to match access-om2

In [ ]:
for vrange in [3]:
    fname = figname('_'.join(['JRA55-do', 'T_10', 'minus', 'JRA55', 'T_2', yearrange, 'monthly', 'mean', 'vrange', str(vrange)]))
    if os.path.exists(fname):
        print('-- skipped', fname)
    else:
        print('doing', fname)
        plt.figure(figsize=(20, 20))
        for m in range(1, 13):
            plt.subplot(4,3,m)
            ax = (JRA55do_tas_025 - JRA55_tas_025).sel(month=m).plot(vmin=-abs(vrange), vmax=abs(vrange), cmap=plt.get_cmap('seismic'), extend='both')
            plt.title(' '.join(['JRA55-do T_10 - JRA55 T_2', yearrange, calendar.month_name[m], 'mean']))
        savefigure(fname)
        plt.close()

# humidity - SEE IF I CAN GET JRA55 SURFACE HUMIDITY! IS IT THE BOTTOM PRESSURE LEVEL?

In [ ]:
JRA55do_huss_025 = JRA55do_to_025(method='patch')(JRA55do_data['huss'+'_mm']).where(landmask != 0)  # use 'patch' method to match access-om2

In [ ]:
JRA55_hus_025 = JRA55_to_025(method='patch')(JRA55_data['hus'+'_mm']).isel(plev=0).where(landmask != 0)  # use bottom pressure level; use 'patch' method to match access-om2

In [ ]:
JRA55do_huss_025.sel(month=1).plot()

In [ ]:
JRA55_hus_025.isel(plev=0).sel(month=1).plot()

In [ ]:
for vrange in [5]:
    plt.figure(figsize=(20, 20))
    for m in range(1, 13):
        plt.subplot(4,3,m)
        # ax = (JRA55do_huss_025 - JRA55_hus_025).sel(month=m).plot(vmin=-abs(vrange), vmax=abs(vrange), cmap=plt.get_cmap('seismic'), extend='both')
        ax = (JRA55do_huss_025 - JRA55_hus_025).sel(month=m).plot(cmap=plt.get_cmap('seismic'), extend='both')
        plt.title(' '.join(['JRA55-do q_10 - JRA55 q_2', yearrange, calendar.month_name[m], 'mean']))
        break
    # savefigure(figname('_'.join(['JRA55-do', 'huss_10', 'minus', 'JRA55', 'hus_2', yearrange, 'monthly', 'mean', 'vrange', str(vrange)])))
    # plt.close()

In [ ]:
ERA5_data['t2m']

In [ ]:
ERA5_to_JRA55do(ERA5_data['t2m'+'_mm'])

In [ ]:
plt.scatter(ERA5_to_JRA55do(ERA5_data['t2m'+'_mm'].sel(month=1)), 
                          JRA55do_data['tas'+'_mm'].sel(month=1), s=1, alpha=0.1)

In [ ]:
def hist_JRA55do_vs_ERA5(JRA55dovar, ERA5var, varrange):
    vs = JRA55do_data[JRA55dovar+'_mm'] #.isel(time=0)).groupby('time.month').mean('time', skipna=True)
    vrefs = ERA5_to_JRA55do(ERA5_data[ERA5var+'_mm']) #.isel(time=0)).groupby('time.month').mean('time', skipna=True)
    for m in range(1, 13):
        title = ' '.join(['JRA55-do', JRA55dovar, 'vs', 'ERA5', ERA5var, yearrange, calendar.month_name[m]])
        fname = figname('_'.join(['JRA55', JRA55dovar, 
                                  'vs', 'ERA5', ERA5var,
                                  yearrange, 'month', str(m).zfill(2)]))
        if False: #os.path.exists(fname):
            print('   -- skipping', fname)
        else:
            v = vs.sel(month=m)
            vref = vrefs.sel(month=m)
            print('doing', fname)
#             vrefm = vref.sel(time=(vref['time.month']==m)).data.ravel()
#             vm = v.sel(time=(v['time.month']==m)).data.ravel()
            fig, ax = plt.subplots(nrows=1, ncols=1)
            ax.set_facecolor('k')
            ax.set_aspect('equal')
#             plt.hist2d(vrefm, JRA55do_to_ERA5(vm),
            # xdata = vref.data.ravel()-273.15
            # ydata = JRA55do_to_ERA5(v).data.ravel()-273.15
            # xdata = ERA5_to_JRA55do(vref).data.ravel()-273.15
            xdata = vref.data.ravel()-273.15
            ydata = v.data.ravel()-273.15
            # xdata = ERA5_to_JRA55do(vref)-273.15
            # xdata = vref.data-273.15
            # ydata = v.data-273.15
            # plt.hist2d(
            h, xedges, yedges = da.histogram2d(
                       ydata, xdata, # this order is deliberate! 
                       # JRA55do_to_ERA5(v).data.ravel().numpy()-273.15,
                       bins=100,
                       # bins=10,
                       range=[varrange, varrange], 
#                        norm=colors.LogNorm(), 
#             # TODO: pick cmax value automatically
# #               cmax=1e5,  # cmax=1e5 for 1deg, 1e6 for 0.25deg, 
            )
            plt.pcolormesh(xedges, yedges, h, norm=matplotlib.colors.LogNorm(), cmap=plt.cm.get_cmap('hot'))
            plt.plot(varrange, varrange, 'g', linewidth=1)
            plt.colorbar()
            plt.title(title)
            plt.xlabel(' '.join(['ERA5', ERA5_data[ERA5var].attrs['long_name']]))
            plt.ylabel(' '.join(['JRA55-do', JRA55do_data[JRA55dovar].attrs['long_name']]))
            savefigure(fname)
            plt.close()
    print('done')

In [ ]:
get_ERA5('t2m').data

In [ ]:
get_ERA5('t2m').groupby('time.month').mean('time', skipna=True).sel(month=1).data

In [ ]:
get_ERA5('t2m').groupby('time.month').mean('time', skipna=True).sel(month=1).data.ravel()-273.15

In [ ]:
get_ERA5('t2m').groupby('time.month').mean('time', skipna=True).sel(month=1).compute()

In [ ]:
get_ERA5('t2m').groupby('time.month').mean('time', skipna=True).sel(month=1).plot()

In [ ]:
loadERA5(ERA5_data, varnames=['t2m'])

In [ ]:
ERA5_data['t2m'].attrs['long_name']

In [ ]:
ERA5_to_JRA55do(ERA5_data['t2m_mm'].sel(month=1)).plot()

In [ ]:
ERA5_data['t2m_mm'].sel(month=1).plot()

In [ ]:
JRA55do_to_ERA5(JRA55do_data['tas_mm'].sel(month=1)).data.ravel()-273.15

In [ ]:
JRA55do_to_ERA5(JRA55do_data['tas_mm'].sel(month=1)).plot()

In [ ]:
plt.scatter(get_ERA5('t2m').groupby('time.month').mean('time', skipna=True).sel(month=1).data.ravel()[1:10], 
           JRA55do_to_ERA5(JRA55do_data['tas_mm'].sel(month=1)).data.ravel()[1:10],
           s=1)

In [ ]:
JRA55do_data['tas_mm']

In [ ]:
hist_JRA55do_vs_ERA5('tas', 't2m', [-70, 45])

# Timeseries plots

## Time series of sea ice volume (by category), area, extent and snow volume

`vicen_m(time, nc, nj, ni)`
		has units = "m",
so need to multiply by `area_t` to get volume.
`nc` is number of ice categories.

We use kcatbound=0, so lower bound of ice categories is 0, 0.64, 1.39, 2.47, 4.57m (HunkeLipscombTurnerJefferyElliott2015a-CICE5p1, table 2).

Much of the Arctic ice volume (not area) is >4.57m thick, including in the summer minimum.

<font color="FF0000"><B>FIXME:<B> land mask area differs between the three configurations and differs from obs, especially in the Canadian Archipelago and River Ob - how to remove this bias in the total extent, area and volume? Can we at least quantify the area differences poleward of (say) 65N/S?
</font>

In [ ]:
initial_transient = False

In [ ]:
def calcvol(ide, timerange=timerange):
# WARNING - this can take several minutes
    loaddata(ide, varnames=['hi_m', 'area_t'], timerange=timerange)
    if 'NH_ice_volume' not in ide.keys():
        volume = ide['hi_m']*ide['area_t'] # hi_m(time, nc, nj, ni) has units = "m", so need to multiply by `area_t` to get volume.
        volume_zonalsum = volume.sum('xt_ocean').compute()  # do this once for both NH & SH
        ide['NH_ice_volume'] = volume_zonalsum.sel(yt_ocean=slice(0, 90)).sum('yt_ocean')
        ide['SH_ice_volume'] = volume_zonalsum.sel(yt_ocean=slice(-90,0)).sum('yt_ocean')

In [ ]:
def calcvolcat(ide, timerange=timerange):
# WARNING - this can take several minutes
    loaddata(ide, varnames=['vicen_m', 'area_t'], timerange=timerange)
    if 'NH_ice_volume_cat' not in ide.keys():
        volume = ide['vicen_m']*ide['area_t'] # vicen_m(time, nc, nj, ni) has units = "m", so need to multiply by `area_t` to get volume.
        volume_zonalsum = volume.sum('xt_ocean').compute()  # do this once for both NH & SH
        ide['NH_ice_volume_cat'] = volume_zonalsum.sel(yt_ocean=slice(0, 90)).sum('yt_ocean')
        ide['SH_ice_volume_cat'] = volume_zonalsum.sel(yt_ocean=slice(-90,0)).sum('yt_ocean')

In [ ]:
def plotvol(v):
    if not initial_transient:
        v = v.sel(time=slice(pd.to_datetime('1985', format='%Y'),tend))  # common time range for all plots
    plt.plot(v['time'],v.isel(nc=c)/1e12, color='C'+str(c), linewidth=1, label='Category '+str(c+1))
#         if not initial_transient:
    plt.plot(v['time'],
             v.rolling(time=12, center=True).mean().isel(nc=c)/1e12, # 12-month rolling mean
             color='C'+str(c), linewidth=2)
#     total
#     plt.plot(v['time'],v.sum(axis=-1)/1e12, color='k', linewidth=1, label='Total')
#     plt.plot(v['time'][6:-5],
#              v.rolling(time=12, center=True).mean().sum(axis=-1)[6:-5]/1e12,  # 12-month rolling mean
#              color='k', linewidth=2)
    plt.ylim(ymin=0)
    plt.xlabel('Year')
    plt.ylabel(r'Ice volume (10$^{12}$ m$^3$)')

In [ ]:
def plotvolcumul(v):
    if not initial_transient:
        v = v.sel(time=slice(pd.to_datetime('1985', format='%Y'),tend))  # common time range for all plots
    plt.plot(v['time'],v/1e12, linewidth=1)
#         if not initial_transient:
    plt.plot(v['time'],
             v.rolling(time=12, center=True).mean()/1e12, # 12-month rolling mean
             linewidth=2)
    plt.ylim(ymin=0)
    plt.xlabel('Year')
    plt.ylabel(r'Ice volume (10$^{12}$ m$^3$)')

In [ ]:
def plotvolcatcumul(v):
    if not initial_transient:
        v = v.sel(time=slice(pd.to_datetime('1985', format='%Y'),tend))  # common time range for all plots
    v = v.cumsum(axis=-1)
    cats = list(range(len(v['nc'])))
    cats.reverse()  # so legend is in same order as plotted data
    for c in cats:
        plt.plot(v['time'],v.isel(nc=c)/1e12, color='C'+str(c), linewidth=1, label='Category '+'+'.join([str(f+1) for f in range(c+1)]))
#         if not initial_transient:
        plt.plot(v['time'],
                 v.rolling(time=12, center=True).mean().isel(nc=c)/1e12, # 12-month rolling mean
                 color='C'+str(c), linewidth=2)
    plt.ylim(ymin=0)
    plt.xlabel('Year')
    plt.ylabel(r'Ice volume (10$^{12}$ m$^3$)')

## Seasonal cycle of sea ice extent and area
We adopt the usual definition of sea ice extent as the area in which sea ice concentration exceeds 15\%.

- obs in `/g/data3/hh5/tmp/cosima/observations/NOAA/G02135/north/monthly/data/*.csv` and  `/g/data3/hh5/tmp/cosima/observations/NOAA/G02135/south/monthly/data/*.csv`
- file names `N_mm_extent_v3.0.csv`, `S_mm_extent_v3.0.csv` where `mm` is month number
- first full year: 1979
- last full year: 2017
- missing extent data (-9999): Dec 1987, Jan 1988

CSV format:
```
year, mo,    data-type, region, extent,   area
1978, 12,      Goddard,      N,  13.67,  10.90
...
```


In [ ]:
def loadObsExt(fnlist):
    """
    Return xarray DataSet of sea ice extent and area from NOAA/G02135 csv file list
    """
    df = pd.concat([pd.read_csv(f) for f in fnlist])  # read all csv files into a pandas DataFrame
    df.columns = df.columns.str.strip()  # remove leading/trailing whitespace from headers
#     df['time'] = df.apply(lambda r: datetime(r['year'], r['mo'], 1), axis=1)  # make a date column (set day=1 to match cice output)
    df['time'] = df.apply(lambda r: cftime.DatetimeGregorian(r['year'], r['mo'], 1), axis=1)  # make a date column (set day=1 to match cice output)
#     print(df)
    df = df.drop(columns=['year', 'mo', 'data-type', 'region'])  # remove redundant columns
    num = df._get_numeric_data()
    num[num < 0] = np.nan  # replace bad data with NaN
    df = df.sort_values('time')
    ds = df.to_xarray()  # convert to xarray DataSet
    ds = ds.assign_coords(index=ds['time']).drop('time')  # set index values to time and remove time
    ds['extent'] = ds.extent.rename({'index': 'time'})  # rename extent coord to time
    ds['area'] = ds.area.rename({'index': 'time'})  # rename area coord to time
    ds = ds.drop('index')  # remove index
    ds = ds*1e12  # convert from M km^2 to m^2
    return ds

In [ ]:
obs_ext_NH = loadObsExt(obsExtNHFileList)
obs_ext_NH_mm = obs_ext_NH.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
obs_ext_SH = loadObsExt(obsExtSHFileList)
obs_ext_SH_mm = obs_ext_SH.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)

In [ ]:
def calcarea(ide, timerange=timerange):
    loaddata(ide, varnames=['aice_m', 'area_t'], timerange=timerange)
    if 'NH_area' not in ide.keys():
        area = ide['aice_m']*ide['area_t']
        area_zonalsum = area.sum('xt_ocean').compute()  # do this once for both NH & SH
        ide['NH_area'] = area_zonalsum.sel(yt_ocean=slice(0, 90)).sum('yt_ocean')
        ide['SH_area'] = area_zonalsum.sel(yt_ocean=slice(-90,0)).sum('yt_ocean')
        ide['NH_area_clim'] = ide['NH_area'].sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
        ide['SH_area_clim'] = ide['SH_area'].sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)

In [ ]:
def calcextent(ide, timerange=timerange):
    loaddata(ide, varnames=['aice_m', 'area_t'], timerange=timerange)
    if 'NH_extent' not in ide.keys():
        extent = xr.where(ide['aice_m'] > 0.15, ide['area_t'], 0)
        extent_zonalsum = extent.sum('xt_ocean').compute()  # do this once for both NH & SH
        ide['NH_extent'] = extent_zonalsum.sel(yt_ocean=slice(0, 90)).sum('yt_ocean')
        ide['SH_extent'] = extent_zonalsum.sel(yt_ocean=slice(-90,0)).sum('yt_ocean')
        ide['NH_extent_clim'] = ide['NH_extent'].sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
        ide['SH_extent_clim'] = ide['SH_extent'].sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)

In [ ]:
def plotarea(v, obs):
    v = v.sel(time=slice(pd.to_datetime('1985', format='%Y'),tend))  # common time range for all plots
    plt.plot(v['time'],v/1e12, color='r', linewidth=1, label='Model')
    plt.plot(v['time'],
             v.rolling(time=12, center=True).mean()/1e12, # 12-month rolling mean
             color='r', linewidth=2)

    obs = obs.sel(time=slice(pd.to_datetime('1985', format='%Y'),tend))  # common time range for all plots
    plt.plot(obs['time'],obs/1e12, color='k', linewidth=1, label='Obs')
    plt.plot(obs['time'],
             obs.rolling(time=12, center=True).mean()/1e12, # 12-month rolling mean
             color='k', linewidth=2)

    plt.ylim(ymin=0)
    plt.xlabel('Year',font)
    plt.ylabel(r'Ice area (10$^{12}$ m$^2$)',font)

In [ ]:
def plotareaNHSH():
    plt.figure(1,(12,5))
    plt.clf()

    plt.subplot(2,1,1)
    plotarea(NH_area, obs_ext_NH.area)
    plt.title('Arctic ice area, '+exptdata.exptdict[ekey]['desc'],font)

    plt.subplot(2,1,2)
    plotarea(SH_area, obs_ext_SH.area)
    plt.title('Antarctic ice area, '+exptdata.exptdict[ekey]['desc'],font)
    plt.legend(prop=font,loc='upper left', bbox_to_anchor=(1, 1))

    plt.tight_layout()

    savefigure('ice_area_'+ekey)

In [ ]:
def plotextent(v, obs):
    v = v.sel(time=slice(pd.to_datetime('1985', format='%Y'),tend))  # common time range for all plots
    plt.plot(v['time'],v/1e12, color='r', linewidth=1, label='Model')
    plt.plot(v['time'],
             v.rolling(time=12, center=True).mean()/1e12, # 12-month rolling mean
             color='r', linewidth=2)

    obs = obs.sel(time=slice(pd.to_datetime('1985', format='%Y'),tend))  # common time range for all plots
    plt.plot(obs['time'],obs/1e12, color='k', linewidth=1, label='Obs')
    plt.plot(obs['time'],
             obs.rolling(time=12, center=True).mean()/1e12, # 12-month rolling mean
             color='k', linewidth=2)

    plt.ylim(ymin=0)
    plt.xlabel('Year',font)
    plt.ylabel(r'Ice extent (10$^{12}$ m$^2$)',font)

In [ ]:
def plotextentNHSH():
    plt.figure(1,(12,5))
    plt.clf()

    plt.subplot(1,2,1)
    plotextent(NH_extent, obs_ext_NH.extent)
    plt.title('Arctic ice extent, '+exptdata.exptdict[ekey]['desc'],font)

    plt.subplot(1,2,2)
    plotextent(SH_extent, obs_ext_SH.extent)
    plt.title('Antarctic ice extent, '+exptdata.exptdict[ekey]['desc'],font)
    plt.legend(prop=font,loc='upper left', bbox_to_anchor=(1, 1))

    plt.tight_layout()

    savefigure('ice_extent_'+ekey)

In [ ]:
# for ekey in ice_data.keys():
for ekey in ensemble:
    print(ekey)
    for ide in ice_data[ekey]:
#         calcvol(ide)
        calcarea(ide)
#         calcextent(ide)
#         break  # only load cycle 1
#     break

## 12-mo running mean min, mean and max

In [ ]:
ice_data[controlkeys[0]][0]['params']

In [ ]:
all_keys = set().union(*(ice_data[k][0]['params'].keys() for k in controlkeys))
all_keys

In [ ]:
# https://stackoverflow.com/questions/65806135/overflowerror-fig-savefig-triggered-by-dpi
matplotlib.use('Agg')
matplotlib.rcParams['agg.path.chunksize'] = 200


In [ ]:
matplotlib.style.use('fast')


In [ ]:
allparams

### separated by perturbation

In [ ]:
# 12-mo running mean minimum, mean and maximum of area -- separate plots for each perturbed parameter

def running(op, v):
    return eval("v['time'][6:-5],v.rolling(time=12, center=True)."+op+"()[6:-5]")

# cols = plt.get_cmap('nipy_spectral')
# cols = plt.get_cmap('hsv')
cols = plt.get_cmap('plasma')
# cols = cm.cm.thermal
linestyles = ['-', '--', ':', '-.']
lwidth = 0.5

allparams = set().union(*(ice_data[k][0]['params'].keys() for k in controlkeys))
allparams = ['albicev']

for cycle in range(1, 2):
    for param in allparams:  # NB: assumes the same params at both resolutions!
        subensemble = [ k for k in ensemble if ice_data[k][0]['perturbed']==param or (ice_data[k][0]['perturbed']==None and param in ice_data[k][0]['params']) ]
        print(subensemble)
        vals = [ ice_data[k][0]['params'][param] for k in subensemble ] #if param in ice_data[k][0]['params'] ]
        # sort vals and subensemble by value
        vs = sorted(zip(vals, subensemble))
        vals = [ v for v, _ in vs ]
        subensemble = [ s for _, s in vs ]
        labels = [ param+'='+str(v) for v in vals ]
        for controlkey in controlkeys:
            try:                                                                ## KLUDGE! shouldn't need this
                labels[subensemble.index(controlkey)] += ' (control)'
            except:
                pass
        if len(subensemble) > 1:
            colsteps = min(20, len(subensemble))
            lsstep = colsteps
            plt.figure(1,(12,5))                                                                                               
            for c, k in enumerate(subensemble):               
                ide = ice_data[k][cycle-1]
                cval = ((c)/colsteps)%1
                if k.split('_')[0] == '1deg':
                    label = labels[c]+', 1°'
                    linestyle = '-'
                else:
                    label = labels[c]+', 0.25°'
                    linestyle = '--'
#                 linestyle = linestyles[(c//lsstep) % len(linestyles)]
                lw = lwidth
                if k in controlkeys:
                    lw *= 3
                plt.subplot(1,2,1)
                v = ide['NH_area']/1e12
                plt.plot(*running('max', v), color=cols(cval), linewidth=lw, linestyle=linestyle, label=label)
                plt.plot(*running('mean', v), color=cols(cval), linewidth=lw, linestyle=linestyle)
                plt.plot(*running('min', v), color=cols(cval), linewidth=lw, linestyle=linestyle)

                plt.subplot(1,2,2)
                v = ide['SH_area']/1e12
                plt.plot(*running('max', v), color=cols(cval), linewidth=lw, linestyle=linestyle, label=label)
                plt.plot(*running('mean', v), color=cols(cval), linewidth=lw, linestyle=linestyle)
                plt.plot(*running('min', v), color=cols(cval), linewidth=lw, linestyle=linestyle)

            plt.subplot(1,2,1)
            v = obs_ext_NH.area/1e12
            plt.plot(*running('max', v), color='k', linewidth=2*lwidth, label='Observations')
            plt.plot(*running('mean', v), color='k', linewidth=2*lwidth)
            plt.plot(*running('min', v), color='k', linewidth=2*lwidth)

            plt.subplot(1,2,2)
            v = obs_ext_SH.area/1e12
            plt.plot(*running('max', v), color='k', linewidth=2*lwidth, label='Observations')
            plt.plot(*running('mean', v), color='k', linewidth=2*lwidth)
            plt.plot(*running('min', v), color='k', linewidth=2*lwidth)

            plt.subplot(1,2,1)
            plt.ylim(ymin=0,ymax=22)
            plt.grid(axis='y')
            plt.xlabel('Year')
            plt.ylabel(r'Sea ice area (10$^{12}$ m$^2$)')
        #     plt.title('Arctic ice area min, mean and max, cycle ' + str(cycle))
            plt.title('Arctic ice area min, mean and max')
            plt.subplot(1,2,2)
            plt.ylim(ymin=0,ymax=22)
            plt.grid(axis='y')
            plt.xlabel('Year')
        #     plt.ylabel(r'Sea ice area (10$^{12}$ m$^2$)')
        #     plt.title('Antarctic ice area min, mean and max, cycle ' + str(cycle))
            plt.title('Antarctic ice area min, mean and max')
            plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=-(-len(subensemble)//25))

#             plt.tight_layout()

        #     fname = os.path.join('figs', 'ice_area_min_mean_max_all_cycle'+str(cycle)+'.pdf')
            fname = os.path.join('figs', 'ice_area_min_mean_max_'+param+'.pdf')
            fname = os.path.join('figs', 'ice_area_min_mean_max_'+param+'_new.pdf')

            fname = os.path.join('figs', 'ice_area_min_mean_max_'+param+'_new.png')

            if os.path.exists(fname):
                print('   -- skipping', fname)
#                 break
            else:
#                     v = ide[var].sel(month=m)
#                     bias = v - vref
                print('saving', fname)
#                 plt.savefig(fname, bbox_inches='tight')
#                 plt.close()
            break

In [ ]:
# 12-mo running mean minimum, mean and maximum of area for all models

# TODO: different colour for each perturbation variable, then different line pattern for perturbation values


def running(op, v):
    return eval("v['time'][6:-5],v.rolling(time=12, center=True)."+op+"()[6:-5]")


cols = plt.get_cmap('nipy_spectral')
cols = plt.get_cmap('hsv')

cols = plt.get_cmap('plasma')

# cols = cm.cm.thermal
colsteps = min(20, len(ensemble))
linestyles = ['-', '--', ':', '-.']
lsstep = colsteps
lwidth = 1
for cycle in range(1, 2):
    plt.figure(1,(12,5))
    for c, k in enumerate(ensemble):
        ide = ice_data[k][cycle-1]
        cval = ((c)/colsteps)%1
        linestyle = linestyles[(c//lsstep) % len(linestyles)]
        plt.subplot(1,2,1)
        v = ide['NH_area']/1e12
        plt.plot(*running('max', v), color=cols(cval), linewidth=lwidth, linestyle=linestyle, label=ide['desc'])
        plt.plot(*running('mean', v), color=cols(cval), linewidth=lwidth, linestyle=linestyle)
        plt.plot(*running('min', v), color=cols(cval), linewidth=lwidth, linestyle=linestyle)

        plt.subplot(1,2,2)
        v = ide['SH_area']/1e12
        plt.plot(*running('max', v), color=cols(cval), linewidth=lwidth, linestyle=linestyle, label=ide['desc'])
        plt.plot(*running('mean', v), color=cols(cval), linewidth=lwidth, linestyle=linestyle)
        plt.plot(*running('min', v), color=cols(cval), linewidth=lwidth, linestyle=linestyle)

    plt.subplot(1,2,1)
    v = obs_ext_NH.area/1e12
    plt.plot(*running('max', v), color='k', linewidth=2*lwidth, label='Observations')
    plt.plot(*running('mean', v), color='k', linewidth=2*lwidth)
    plt.plot(*running('min', v), color='k', linewidth=2*lwidth)

    plt.subplot(1,2,2)
    v = obs_ext_SH.area/1e12
    plt.plot(*running('max', v), color='k', linewidth=2*lwidth, label='Observations')
    plt.plot(*running('mean', v), color='k', linewidth=2*lwidth)
    plt.plot(*running('min', v), color='k', linewidth=2*lwidth)

    plt.subplot(1,2,1)
    plt.ylim(ymin=0,ymax=22)
    plt.grid(axis='y')
    plt.xlabel('Year')
    plt.ylabel(r'Sea ice area (10$^{12}$ m$^2$)')
#     plt.title('Arctic ice area min, mean and max, cycle ' + str(cycle))
    plt.title('Arctic ice area min, mean and max')
    plt.subplot(1,2,2)
    plt.ylim(ymin=0,ymax=22)
    plt.grid(axis='y')
    plt.xlabel('Year')
#     plt.ylabel(r'Sea ice area (10$^{12}$ m$^2$)')
#     plt.title('Antarctic ice area min, mean and max, cycle ' + str(cycle))
    plt.title('Antarctic ice area min, mean and max')
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=-(-len(ensemble)//25))

    plt.tight_layout()
    
#     fname = os.path.join('figs', 'ice_area_min_mean_max_all_cycle'+str(cycle)+'.pdf')
    fname = 'ice_area_min_mean_max_param.pdf'

#     plt.savefig(fname, bbox_inches='tight')
#     plt.close()

In [ ]:
v

In [ ]:
ide['NH_area'] #- obs_ext_NH.area

In [ ]:
obs_ext_NH

In [ ]:
da

In [ ]:
        da = ide['NH_area'].copy(deep=True)



In [ ]:
ide['NH_area']

In [ ]:
da

In [ ]:
da = ide['NH_area'].copy(deep=True)
for i, d in enumerate(da['time'].data):
    da['time'].data[i] = d.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
# da['time'].data = [d.replace(day=1, hour=0, minute=0, second=0, microsecond=0) for d in da['time'].data]

In [ ]:
da

In [ ]:
da

In [ ]:
obs_ext_NH.area

In [ ]:
da['time'].data

In [ ]:
# 12-mo running mean minimum, mean and maximum of total area bias for all models

def running(op, v):
    return eval("v['time'][6:-5], v.rolling(time=12, center=True)."+op+"()[6:-5]")


cols = plt.get_cmap('nipy_spectral')
cols = plt.get_cmap('hsv')

cols = plt.get_cmap('plasma')

# cols = cm.cm.thermal
colsteps = min(20, len(ensemble))
linestyles = ['-', '--', ':', '-.']
lsstep = colsteps
lwidth = 1
for cycle in range(1, 2):
    plt.figure(1,(12,5))
    for c, k in enumerate(ensemble):
        ide = ice_data[k][cycle-1]
        cval = ((c)/colsteps)%1
        linestyle = linestyles[(c//lsstep) % len(linestyles)]
        plt.subplot(1,2,1)
        da = ide['NH_area'].copy(deep=True)
        for i, d in enumerate(da['time'].data):
            da['time'].data[i] = d.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
        v = da/1e12 - obs_ext_NH.area/1e12
        plt.plot(*running('max', v), color=cols(cval), linewidth=lwidth, linestyle=linestyle, label=ide['desc'])
        plt.plot(*running('mean', v), color=cols(cval), linewidth=lwidth, linestyle=linestyle)
        plt.plot(*running('min', v), color=cols(cval), linewidth=lwidth, linestyle=linestyle)

        plt.subplot(1,2,2)
        v = ide['SH_area']/1e12 - obs_ext_SH.area/1e12
        plt.plot(*running('max', v), color=cols(cval), linewidth=lwidth, linestyle=linestyle, label=ide['desc'])
        plt.plot(*running('mean', v), color=cols(cval), linewidth=lwidth, linestyle=linestyle)
        plt.plot(*running('min', v), color=cols(cval), linewidth=lwidth, linestyle=linestyle)

#     plt.subplot(1,2,1)
#     v = obs_ext_NH.area/1e12
#     plt.plot(*running('max', v), color='k', linewidth=2*lwidth, label='Observations')
#     plt.plot(*running('mean', v), color='k', linewidth=2*lwidth)
#     plt.plot(*running('min', v), color='k', linewidth=2*lwidth)

#     plt.subplot(1,2,2)
#     v = obs_ext_SH.area/1e12
#     plt.plot(*running('max', v), color='k', linewidth=2*lwidth, label='Observations')
#     plt.plot(*running('mean', v), color='k', linewidth=2*lwidth)
#     plt.plot(*running('min', v), color='k', linewidth=2*lwidth)

    plt.subplot(1,2,1)
    plt.ylim(ymin=0,ymax=22)
    plt.grid(axis='y')
    plt.xlabel('Year')
    plt.ylabel(r'Sea ice area (10$^{12}$ m$^2$)')
    plt.title('Arctic ice area min, mean and max, cycle ' + str(cycle))
    plt.subplot(1,2,2)
    plt.ylim(ymin=0,ymax=22)
    plt.grid(axis='y')
    plt.xlabel('Year')
    plt.ylabel(r'Sea ice area (10$^{12}$ m$^2$)')
    plt.title('Antarctic ice area min, mean and max, cycle ' + str(cycle))
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=-(-len(ensemble)//25))

    plt.tight_layout()
    
#     fname = os.path.join('figs', 'ice_area_min_mean_max_all_cycle'+str(cycle)+'.pdf')
    fname = 'ice_area_min_mean_max_param.pdf'

#     plt.savefig(fname, bbox_inches='tight')
#     plt.close()

In [ ]:
da['time']

In [ ]:
# 12-mo running mean minimum, mean and maximum of extent for all models
for cycle in range(1, 2):
    plt.figure(1,(12,5))
    for c, e in enumerate(ice_data.values()):
        try:
            ide = e[cycle-1]
            plt.subplot(1,2,1)
            v = ide['NH_extent']/1e12
            plt.plot(v['time'][6:-5],v.rolling(time=12, center=True).max()[6:-5], color='C'+str(c), linewidth=.1, label=ide['desc'])
            plt.plot(v['time'][6:-5],v.rolling(time=12, center=True).mean()[6:-5], color='C'+str(c), linewidth=.1)
            plt.plot(v['time'][6:-5],v.rolling(time=12, center=True).min()[6:-5], color='C'+str(c), linewidth=.1)

            plt.subplot(1,2,2)
            v = ide['SH_extent']/1e12
            plt.plot(v['time'][6:-5],v.rolling(time=12, center=True).max()[6:-5], color='C'+str(c), linewidth=.1, label=ide['desc'])
            plt.plot(v['time'][6:-5],v.rolling(time=12, center=True).mean()[6:-5], color='C'+str(c), linewidth=.1)
            plt.plot(v['time'][6:-5],v.rolling(time=12, center=True).min()[6:-5], color='C'+str(c), linewidth=.1)
        except:
            pass

    plt.subplot(1,2,1)
    v = obs_ext_NH.extent/1e12
    plt.plot(v['time'][6:-5],v.rolling(time=12, center=True).max()[6:-5], color='k', linewidth=.2, label='Observations')
    plt.plot(v['time'][6:-5],v.rolling(time=12, center=True).mean()[6:-5], color='k', linewidth=.2)
    plt.plot(v['time'][6:-5],v.rolling(time=12, center=True).min()[6:-5], color='k', linewidth=.2)

    plt.subplot(1,2,2)
    v = obs_ext_SH.extent/1e12
    plt.plot(v['time'][6:-5],v.rolling(time=12, center=True).max()[6:-5], color='k', linewidth=.2, label='Observations')
    plt.plot(v['time'][6:-5],v.rolling(time=12, center=True).mean()[6:-5], color='k', linewidth=.2)
    plt.plot(v['time'][6:-5],v.rolling(time=12, center=True).min()[6:-5], color='k', linewidth=.2)

    plt.subplot(1,2,1)
    plt.ylim(ymin=0,ymax=22)
    plt.grid(axis='y')
    plt.xlabel('Year')
    plt.ylabel(r'Sea ice extent (10$^{12}$ m$^2$)')
    plt.title('Arctic ice extent min, mean and max, cycle ' + str(cycle))
    plt.subplot(1,2,2)
    plt.ylim(ymin=0,ymax=22)
    plt.grid(axis='y')
    plt.xlabel('Year')
    plt.ylabel(r'Sea ice extent (10$^{12}$ m$^2$)')
    plt.title('Antarctic ice extent min, mean and max, cycle ' + str(cycle))
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=-(-len(ice_data.values())//25))

    plt.tight_layout()
    
#     fname = os.path.join('figs', 'ice_extent_min_mean_max_all_cycle'+str(cycle)+'.pdf')

    fname = 'ice_extent_min_mean_max_param.pdf'

    plt.savefig(fname, bbox_inches='tight')
#     plt.close()

## Timeseries of each month
Formatted to match left two columns of [Tsujino et al., 2020](https://doi.org/10.5194/gmd-13-3643-2020) fig 22:

![](https://gmd.copernicus.org/articles/13/3643/2020/gmd-13-3643-2020-f22.png)

In [ ]:
# timeseries for each month for all models

for var, obs in zip(['SH_extent', 'NH_extent'], [obs_ext_SH.extent, obs_ext_NH.extent]):
    hem = var.split('_')[0]
    for m in range(1,13):
        fname = os.path.join('figs', '_'.join([var, yearrange, 'month', str(m).zfill(2)])+'.pdf')
        if False: #os.path.exists(fname):
            print('   -- skipping', fname)
        else:
            print('doing', fname)
            plt.figure(1,(6,3))
            for cycle in range(1, 7):
                for c, e in enumerate(ice_data.values()):
                    try:
                        ide = e[cycle-1]
                        v = ide[var]/1e12
                        if cycle == 1:
                            plt.plot(v['time'].sel(time=(v['time.month']==m)),
                                     v.sel(time=(v['time.month']==m)),
                                     color='C'+str(c), linewidth=1, label=ide['desc'])
                        else:
                            plt.plot(v['time'].sel(time=(v['time.month']==m)),
                                     v.sel(time=(v['time.month']==m)),
                                     color='C'+str(c), linewidth=1) #, label=ide['desc'])
                    except:
                        pass

            v = obs/1e12
            plt.plot(v['time'].sel(time=(v['time.month']==m)),
                     v.sel(time=(v['time.month']==m)),
                     color='k', linewidth=4, label='NSIDC_SII obs')
            

# set axis limits to match Tsujino et al (2020) fig 22
            plt.xlim([cftime.DatetimeGregorian(1948, 1, 1), cftime.DatetimeGregorian(2019, 1, 1)])
            if hem == 'NH':
                if m == 3:
                    plt.yticks(np.arange(10, 21, 2))
                    plt.ylim(ymin=10, ymax=20)
                elif m == 9:
                    plt.yticks(np.arange(0, 11, 2))
                    plt.ylim(ymin=0, ymax=10)
            else:
                if m == 3:
                    plt.yticks(np.arange(0, 7, 2))
                    plt.ylim(ymin=0, ymax=7)
                elif m == 9:
                    plt.yticks(np.arange(10, 26, 5))
                    plt.ylim(ymin=12, ymax=27)
            plt.grid(axis='x')
            plt.grid(axis='y')
            plt.xlabel('Year')
            plt.ylabel(r'Sea ice extent (10$^{6}$ km$^2$)')
            plt.title(' '.join([calendar.month_name[m], hem]))
            plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

            plt.tight_layout()

            plt.savefig(fname, bbox_inches='tight')
            plt.close()
#             break
#     break

### Mean seasonal cycle of area and extent

In [ ]:
# seasonal cycle of area for all models
for cycle in range(1, 7):
    plt.figure(1,(12,5))
    for c, e in enumerate(ice_data.values()):
        try:
            ide = e[cycle-1]
            plt.subplot(2,1,1)
            v = ide['NH_area']/1e12
            v = v.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
            plt.plot(v['month'],v, color='C'+str(c), linewidth=1, label=ide['desc'])

            plt.subplot(2,1,2)
            v = ide['SH_area']/1e12
            v = v.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
            plt.plot(v['month'],v, color='C'+str(c), linewidth=1, label=ide['desc'])
        except:
            pass
    plt.subplot(2,1,1)
    v = obs_ext_NH.area/1e12
    v = v.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
    plt.plot(v['month'],v, color='k', linewidth=1, label='Observations')

    plt.subplot(2,1,2)
    v = obs_ext_SH.area/1e12
    v = v.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
    plt.plot(v['month'],v, color='k', linewidth=1, label='Observations')

    plt.subplot(2,1,1)
    plt.ylim(ymin=0,ymax=22)
    plt.xlabel('Month')
    plt.ylabel(r'Sea ice area (10$^{12}$ m$^2$)')
    plt.title('Arctic sea ice area '+yearrange+' mean annual cycle, for JRA55-do cycle ' + str(cycle))
    plt.subplot(2,1,2)
    plt.ylim(ymin=0,ymax=22)
    plt.xlabel('Month')
    plt.ylabel(r'Sea ice area (10$^{12}$ m$^2$)')
    plt.title('Antarctic sea ice area '+yearrange+' mean annual cycle, for JRA55-do cycle ' + str(cycle))
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.tight_layout()

    fname = os.path.join('figs', 'ice_area_seasonal_clim_cycle'+str(cycle)+'.pdf')
    plt.savefig(fname, bbox_inches='tight')
    plt.close()

In [ ]:
# seasonal cycle of extent for all models
for cycle in range(1, 7):

    plt.figure(1,(12,5))
    for c, e in enumerate(list(ice_data.values())[0:3]):
        try:
            ide = e[cycle-1]
            plt.subplot(2,1,1)
            v = ide['NH_extent']/1e12
            v = v.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
            plt.plot(v['month'],v, color='C'+str(c), linewidth=1, label=ide['desc'])

            plt.subplot(2,1,2)
            v = ide['SH_extent']/1e12
            v = v.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
            plt.plot(v['month'],v, color='C'+str(c), linewidth=1, label=ide['desc'])
        except:
            pass

    plt.subplot(2,1,1)
    v = obs_ext_NH.extent/1e12
    v = v.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
    plt.plot(v['month'],v, color='k', linewidth=2, label='Observations')

    plt.subplot(2,1,2)
    v = obs_ext_SH.extent/1e12
    v = v.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
    plt.plot(v['month'],v, color='k', linewidth=2, label='Observations')

    plt.subplot(2,1,1)
    plt.ylim(ymin=0,ymax=22)
    plt.xlabel('Month')
    plt.ylabel(r'Sea ice extent (10$^{12}$ m$^2$)')
    plt.title('Arctic sea ice extent '+yearrange+' mean annual cycle, for JRA55-do cycle ' + str(cycle))
    plt.subplot(2,1,2)
    plt.ylim(ymin=0,ymax=22)
    plt.xlabel('Month')
    plt.ylabel(r'Sea ice extent (10$^{12}$ m$^23$)')
    plt.title('Antarctic sea ice extent '+yearrange+' mean annual cycle, for JRA55-do cycle ' + str(cycle))
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.tight_layout()

    fname = os.path.join('figs', 'ice_extent_seasonal_clim_cycle'+str(cycle)+'.pdf')
    plt.savefig(fname, bbox_inches='tight')
    plt.close()

In [ ]:
# seasonal cycle of extent for all models
for cycle in range(1, 7):

    plt.figure(1,(12,5))
    for c, e in enumerate(list(ice_data.values())[0:3]):
        try:
            ide = e[cycle-1]
            plt.subplot(2,1,1)
            v = ide['NH_extent']/1e12
            v = v.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
            plt.plot(v['month'],v, color='C'+str(c), linewidth=1, label=ide['desc'])

            plt.subplot(2,1,2)
            v = ide['SH_extent']/1e12
            v = v.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
            plt.plot(v['month'],v, color='C'+str(c), linewidth=1, label=ide['desc'])
        except:
            pass

    plt.subplot(2,1,1)
    v = obs_ext_NH.extent/1e12
    v = v.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
    plt.plot(v['month'],v, color='k', linewidth=2, label='Observations')

    plt.subplot(2,1,2)
    v = obs_ext_SH.extent/1e12
    v = v.sel(time=slice(tstart,tend)).groupby('time.month').mean('time', skipna=True)
    plt.plot(v['month'],v, color='k', linewidth=2, label='Observations')

    plt.subplot(2,1,1)
    plt.ylim(ymin=0,ymax=22)
    plt.xlabel('Month')
    plt.ylabel(r'Sea ice extent (10$^{12}$ m$^2$)')
    plt.title('Arctic sea ice extent '+yearrange+' mean annual cycle, for JRA55-do cycle ' + str(cycle))
    plt.subplot(2,1,2)
    plt.ylim(ymin=0,ymax=22)
    plt.xlabel('Month')
    plt.ylabel(r'Sea ice extent (10$^{12}$ m$^23$)')
    plt.title('Antarctic sea ice extent '+yearrange+' mean annual cycle, for JRA55-do cycle ' + str(cycle))
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.tight_layout()

    fname = os.path.join('figs', 'ice_extent_seasonal_clim_cycle'+str(cycle)+'.pdf')
    plt.savefig(fname, bbox_inches='tight')
    plt.close()

### Mean seasonal cycle of volume terms
units: cm/day for all of
- dvidtd_m
- dvidtt_m
- congel_m
- frazil_m
- meltt_m
- meltb_m
- meltl_m
- snoice_m
- evap_ai_m           ** NB: ai - so this is a grid-cell mean, like all the others **



In [ ]:
# volume tendency terms due to thermo (and dynamics)
terms = ['congel_m', 'frazil_m', 'meltt_m', 'meltb_m', 'meltl_m', 'snoice_m', 'evap_ai_m', 'dvidtt_m'] #, 'dvidtd_m']

In [ ]:
monthdays = [ calendar.monthrange(2001, m)[1] for m in range(1, 13) ] # not a leap year
rhoi = 917.0 # CICE ice density (kg/m^3)

In [ ]:
def calcterms(ide, varnames=terms, timerange=timerange):
    loaddata(ide, varnames=varnames, timerange=timerange)
    for var in varnames:
        print(var)
        if 'NH_'+var not in ide.keys():
            vol = ide[var]*ide['area_t']
            vol_zonalsum = vol.sum('xt_ocean').compute()  # do this once for both NH & SH
            vol_zonalsum = vol_zonalsum/100*rhoi  # convert to kg/day
            if var in ['meltt_m', 'meltb_m', 'meltl_m']:
                vol_zonalsum = -vol_zonalsum
            ide['NH_'+var] = vol_zonalsum.sel(yt_ocean=slice(0, 90)).sum('yt_ocean')
            ide['SH_'+var] = vol_zonalsum.sel(yt_ocean=slice(-90,0)).sum('yt_ocean')
            # monthly climatologies are in kg/month
            ide['NH_'+var+'_clim'] = ide['NH_'+var].sel(time=timerange).groupby('time.month').mean('time', skipna=True)*monthdays
            ide['SH_'+var+'_clim'] = ide['SH_'+var].sel(time=timerange).groupby('time.month').mean('time', skipna=True)*monthdays

In [ ]:
for ekey in ensemble:
    for ide in ice_data[ekey]:
        calcterms(ide)

In [ ]:
control = controlkeys[0]  # KLUDGE! do this better!
for ekey in ensemble:
    for cycle, ide in enumerate(ice_data[ekey], start=1):
        idecontrol = ice_data[control][cycle-1]
        title = 'Antarctic sea ice mass balance '+yearrange+' mean annual cycle for '+ide['desc']
        fname = '_'.join(['SH', ide['expt'].split('_cycle')[0], 'cycle'+str(cycle), # FRAGLE! assumes filename cycle number = cycle
                                          yearrange, 'mass_terms_seasonal_clim'])
        pngfname = os.path.join(figdir, fname+'_'+str(dpi)+'dpi.png')
        pdffname = os.path.join(figdir, fname+'.pdf')
        if os.path.exists(pngfname):
            print('   -- skipping', fname)
        else:
            print('doing '+fname)
            plt.figure(figsize=(10, 8))
            if not ekey == control:
                for n, var in enumerate(terms):
                    v = idecontrol['SH_'+var+'_clim']/1e15
                    plt.plot(v['month'], v, '--', color='C'+str(n), linewidth=1)
            for n, var in enumerate(terms):
                if var == 'dvidtt_m':
                    label = 'net thermo tendency'
                else:
                    label = ide[var].long_name
                v = ide['SH_'+var+'_clim']/1e15
                plt.plot(v['month'], v, color='C'+str(n), linewidth=2, label=label)
            plt.plot([1,12], [0, 0], '-k', linewidth=0.5)  # zero line
            plt.xlabel('Month')
            plt.ylabel(r'Sea ice mass tendency (10$^{15}$ kg/month)')
            plt.xlim(xmin=1, xmax=12)
            plt.ylim(ymin=-6, ymax=3)
            plt.legend(loc='lower center')
            plt.title(title)
            plt.savefig(pngfname, dpi=dpi, bbox_inches='tight')
            plt.savefig(pdffname, dpi=dpi, bbox_inches='tight')
            plt.close()


## TODO: Mean seasonal cycle of area terms